# Import libraries

In [26]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012 as preprocess_physionet2012
from missingData.toolkits import toolkits
import math
import scipy.stats as st

# Loading database

### Standard Scaler 

In [27]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-05-12 21:04:44 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-05-12 21:04:44 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-12 21:04:44 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-12 21:04:44 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-12 21:04:44 [INFO]: Loaded successfully!
2025-05-12 21:05:00 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-12 21:05:00 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-12 21:05:00 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 'test_ICUType_classificao_baixo_peso', 'classificacao_normal_peso_test_X', 'classificacao_normal_peso_test_y', 'test_ICUType_classificacao_normal_peso', 'classificacao_sobrepeso_test_X', 'classificacao_sobrepeso_test_y', 'test_ICUTy

In [28]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [29]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

In [30]:
teste = "teste"

# Creating indicating mask and get test_X_ori

### Standard Scaler

In [31]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm_general = indicating_mask_variable_standard_norm
test_X_ori_variable_standard_norm_general = test_X_ori_variable_standard_norm
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

# Inicialize the models

### SAITS

<h4>Inicialize new model</h4>

In [7]:
saits = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-02 20:32:04 [INFO]: No given device, using default device: cpu
2025-05-02 20:32:04 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250502_T203204
2025-05-02 20:32:04 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250502_T203204/tensorboard
2025-05-02 20:32:04 [INFO]: Using customized MAE as the training loss function.
2025-05-02 20:32:04 [INFO]: Using customized MSE as the validation metric function.
2025-05-02 20:32:04 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize model</h4>

In [32]:
saits = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-12 21:05:27 [INFO]: No given device, using default device: cpu
2025-05-12 21:05:27 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-12 21:05:27 [INFO]: Using customized MAE as the training loss function.
2025-05-12 21:05:27 [INFO]: Using customized MSE as the validation metric function.
2025-05-12 21:05:27 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


### BRITS

<h4>Inicialize new model</h4>

In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: Using customized MAE as the training loss function.
2025-04-22 18:16:43 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 18:16:43 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize model</h4>

In [33]:
brits = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-12 21:05:31 [INFO]: No given device, using default device: cpu
2025-05-12 21:05:31 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-12 21:05:31 [INFO]: Using customized MAE as the training loss function.
2025-05-12 21:05:31 [INFO]: Using customized MSE as the validation metric function.
2025-05-12 21:05:31 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


### USGAN

<h4>Inicialize new model</h4>

In [8]:
us_gan = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize model</h4>

In [34]:
us_gan = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-12 21:05:35 [INFO]: No given device, using default device: cpu
2025-05-12 21:05:35 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-12 21:05:35 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


### GPVAE

<h4>Inicialize new model</h4>

In [9]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize model</h4>

In [35]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-12 21:05:38 [INFO]: No given device, using default device: cpu
2025-05-12 21:05:38 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-12 21:05:38 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


### MRNN

<h4>Inicialize new model</h4>

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: Using customized RMSE as the training loss function.
2025-04-22 18:16:43 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 18:16:43 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize model</h4>

In [36]:
mrnn = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-05-12 21:05:42 [INFO]: No given device, using default device: cpu
2025-05-12 21:05:42 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-12 21:05:42 [INFO]: Using customized RMSE as the training loss function.
2025-05-12 21:05:42 [INFO]: Using customized MSE as the validation metric function.
2025-05-12 21:05:42 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


# Train/Load the models

### SAITS

<h4>Train</h4>

In [7]:
saits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 21:54:58 [INFO]: Epoch 001 - training loss (MAE): 0.7233, validation MSE: 6.7652
2025-04-22 21:55:03 [INFO]: Epoch 002 - training loss (MAE): 0.5383, validation MSE: 6.7408
2025-04-22 21:55:08 [INFO]: Epoch 003 - training loss (MAE): 0.4956, validation MSE: 6.7218
2025-04-22 21:55:12 [INFO]: Epoch 004 - training loss (MAE): 0.4643, validation MSE: 6.6987
2025-04-22 21:55:16 [INFO]: Epoch 005 - training loss (MAE): 0.4392, validation MSE: 6.6806
2025-04-22 21:55:21 [INFO]: Epoch 006 - training loss (MAE): 0.4207, validation MSE: 6.6686
2025-04-22 21:55:24 [INFO]: Epoch 007 - training loss (MAE): 0.4065, validation MSE: 6.6527
2025-04-22 21:55:28 [INFO]: Epoch 008 - training loss (MAE): 0.3953, validation MSE: 6.6548
2025-04-22 21:55:32 [INFO]: Epoch 009 - training loss (MAE): 0.3866, validation MSE: 6.6511
2025-04-22 21:55:36 [INFO]: Epoch 010 - training loss (MAE): 0.3798, validation MSE: 6.6584
2025-04-22 21:55:36 [INFO]: Finished training. The best model is from epoch#9.
2

<h4> Load</h4>

In [37]:
saits.load("tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots")

2025-05-12 21:05:46 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots


### BRITS

<h4>Train</h4>

In [12]:
brits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 18:20:14 [INFO]: Epoch 001 - training loss (MAE): 0.9371, validation MSE: 6.7840
2025-04-22 18:22:18 [INFO]: Epoch 002 - training loss (MAE): 0.7328, validation MSE: 6.7456
2025-04-22 18:24:23 [INFO]: Epoch 003 - training loss (MAE): 0.6822, validation MSE: 6.7326
2025-04-22 18:26:17 [INFO]: Epoch 004 - training loss (MAE): 0.6576, validation MSE: 6.7294
2025-04-22 18:28:36 [INFO]: Epoch 005 - training loss (MAE): 0.6427, validation MSE: 6.7280
2025-04-22 18:30:47 [INFO]: Epoch 006 - training loss (MAE): 0.6318, validation MSE: 6.7267
2025-04-22 18:32:52 [INFO]: Epoch 007 - training loss (MAE): 0.6229, validation MSE: 6.7294
2025-04-22 18:35:07 [INFO]: Epoch 008 - training loss (MAE): 0.6149, validation MSE: 6.7316
2025-04-22 18:37:26 [INFO]: Epoch 009 - training loss (MAE): 0.6087, validation MSE: 6.7332
2025-04-22 18:37:26 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-04-22 18:37:26 [INFO]: Finished training. The best model is from epoc

<h4>Load</h4>

In [38]:
brits.load("tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots")

2025-05-12 21:05:48 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots


### USGAN

<h4>Train</h4>

In [13]:
us_gan.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 18:41:35 [INFO]: Epoch 001 - generator training loss: 0.4346, discriminator training loss: 0.1907, validation loss: 6.7750
2025-04-22 18:45:20 [INFO]: Epoch 002 - generator training loss: 0.3701, discriminator training loss: 0.0566, validation loss: 6.7282
2025-04-22 18:49:07 [INFO]: Epoch 003 - generator training loss: 0.3347, discriminator training loss: 0.0374, validation loss: 6.7004
2025-04-22 18:52:56 [INFO]: Epoch 004 - generator training loss: 0.3165, discriminator training loss: 0.0311, validation loss: 6.6837
2025-04-22 18:56:43 [INFO]: Epoch 005 - generator training loss: 0.3020, discriminator training loss: 0.0281, validation loss: 6.6737
2025-04-22 19:00:32 [INFO]: Epoch 006 - generator training loss: 0.2923, discriminator training loss: 0.0265, validation loss: 6.6685
2025-04-22 19:04:19 [INFO]: Epoch 007 - generator training loss: 0.2877, discriminator training loss: 0.0252, validation loss: 6.6691
2025-04-22 19:08:07 [INFO]: Epoch 008 - generator training los

<h4>Load</h4>

In [39]:
us_gan.load("tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots")

2025-05-12 21:05:51 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots


### GPVAE

<h4>Train</h4>

In [14]:
gp_vae.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 19:15:55 [INFO]: Epoch 001 - training loss (default): 26027.5423, validation loss: 22882.8125
2025-04-22 19:16:01 [INFO]: Epoch 002 - training loss (default): 22877.1633, validation loss: 22807.7179
2025-04-22 19:16:07 [INFO]: Epoch 003 - training loss (default): 22842.6190, validation loss: 22788.8947
2025-04-22 19:16:14 [INFO]: Epoch 004 - training loss (default): 22832.1135, validation loss: 22783.6572
2025-04-22 19:16:20 [INFO]: Epoch 005 - training loss (default): 22827.5731, validation loss: 22780.0771
2025-04-22 19:16:27 [INFO]: Epoch 006 - training loss (default): 22821.9797, validation loss: 22773.8286
2025-04-22 19:16:34 [INFO]: Epoch 007 - training loss (default): 22814.3062, validation loss: 22767.3447
2025-04-22 19:16:40 [INFO]: Epoch 008 - training loss (default): 22810.7426, validation loss: 22771.6522
2025-04-22 19:16:46 [INFO]: Epoch 009 - training loss (default): 22808.6126, validation loss: 22762.6405
2025-04-22 19:16:52 [INFO]: Epoch 010 - training loss (

<h4>Load</h4>

In [40]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots")

2025-05-12 21:05:53 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots


### MRNN

<h4>Train</h4>

In [15]:
mrnn.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 19:17:30 [INFO]: Epoch 001 - training loss (RMSE): 0.7504, validation MSE: 7.3129
2025-04-22 19:17:47 [INFO]: Epoch 002 - training loss (RMSE): 0.5341, validation MSE: 7.2684
2025-04-22 19:18:05 [INFO]: Epoch 003 - training loss (RMSE): 0.4959, validation MSE: 7.2556
2025-04-22 19:18:22 [INFO]: Epoch 004 - training loss (RMSE): 0.4612, validation MSE: 7.2489
2025-04-22 19:18:40 [INFO]: Epoch 005 - training loss (RMSE): 0.4541, validation MSE: 7.2464
2025-04-22 19:18:59 [INFO]: Epoch 006 - training loss (RMSE): 0.4473, validation MSE: 7.2449
2025-04-22 19:19:16 [INFO]: Epoch 007 - training loss (RMSE): 0.4298, validation MSE: 7.2458
2025-04-22 19:19:34 [INFO]: Epoch 008 - training loss (RMSE): 0.4340, validation MSE: 7.2485
2025-04-22 19:19:51 [INFO]: Epoch 009 - training loss (RMSE): 0.4229, validation MSE: 7.2492
2025-04-22 19:19:51 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-04-22 19:19:51 [INFO]: Finished training. The best model is 

<h4>Load</h4>

In [41]:
mrnn.load("tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots")

2025-05-12 21:05:55 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots


# Testing stage

### SAITS

#### Standard Scaler

In [42]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_general = saits_imputation_standard
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

### BRITS

#### Standard Scaler

In [43]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

### USGAN

In [44]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

### GPVAE

In [45]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori)

### MRNN

In [46]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori)

# Calculate mean absolute error

In [49]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

In [50]:
subgroups = ["General", "Female", "Male", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

## SAITS

### Calculate MAE and AE

In [52]:
testing_mae_saits_variables_standard, testing_ae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

In [53]:
from MAEModify.error import calc_mae
testing_mae_saits_variables_standard_general = []
testing_ae_saits_variables_standard_general = []
for i in range(len(saits_imputation_standard_general)):
    aux1, aux2 = calc_mae(saits_imputation_standard_general[i], test_X_ori_variable_standard_norm_general[i], indicating_mask_variable_standard_norm_general[i])
    testing_mae_saits_variables_standard_general.append(aux1) 
    testing_ae_saits_variables_standard_general.append(aux2)

### Taking the MAE difference for SAITS model

#### Difference between gender group

In [23]:
diff_saits_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 1, 2, variables)
diff_saits_gender_subgroup

[('Cholesterol', 0.5369465600078649),
 ('TroponinI', 0.21670827441781393),
 ('SaO2', 0.2006324035650055),
 ('Lactate', 0.10301878818965454),
 ('Urine', 0.06260751107865509)]

#### Difference between age group

In [24]:
diff_saits_age_subgroup = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 4, 5, variables)
diff_saits_age_subgroup

[('Cholesterol', 0.7145901970213923),
 ('TroponinI', 0.5566432434151988),
 ('ALP', 0.3592549477972564),
 ('SaO2', 0.1956973703724636),
 ('Bilirubin', 0.19557828811797667)]

#### Difference between ICUType group

In [25]:
diff_saits_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 7, variables)
diff_saits_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 8, variables)
diff_saits_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 9, variables)
diff_saits_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 7, 8, variables)
diff_saits_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 7, 9, variables)
diff_saits_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 8, 9, variables)

In [26]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_saits_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_saits_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_saits_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_saits_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_saits_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_saits_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('TroponinI', 0.9325070778805344), ('Cholesterol', 0.9139836206325719), ('TroponinT', 0.610061672941423), ('Albumin', 0.3227603119355845), ('AST', 0.2527107766945585)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('TroponinI', 1.0714226157212936), ('SaO2', 1.0588494232986556), ('ALP', 0.7533389720997652), ('TroponinT', 0.5510963137613376), ('Albumin', 0.20918888526467827)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.6292154153589171), ('TroponinT', 0.5631873029767689), ('Albumin', 0.2424261680183718), ('Bilirubin', 0.2032288517692434), ('Glucose', 0.18363430222468535)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.2740858586846673), ('Cholesterol', 0.8991669157791677), ('ALP', 0.8536263954002238), ('Temp', 0.2721045789688794), ('Glucose', 0.22842305459952772)]
----------------------------
Diferença(MAE)

In [27]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[31])
print(variables[6])
print(variables[0])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
TroponinI
Cholesterol
ALP
TroponinT


#### Difference between BMI group

In [28]:
diff_saits_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 11, variables)
diff_saits_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 12, variables)
diff_saits_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 13, variables)
diff_saits_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 14, variables)
diff_saits_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 15, variables)
diff_saits_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 16, variables)

diff_saits_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 12, variables)
diff_saits_classification_low_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 13, variables)
diff_saits_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 14, variables)
diff_saits_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 15, variables)
diff_saits_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 16, variables)

diff_saits_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 13, variables)
diff_saits_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 14, variables)
diff_saits_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 15, variables)
diff_saits_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 16, variables)

diff_saits_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 14, variables)
diff_saits_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 15, variables)
diff_saits_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 16, variables)


diff_saits_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 14, 15, variables)
diff_saits_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 14, 16, variables)

diff_saits_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 15, 16, variables)

In [29]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_saits_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_saits_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_saits_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_saits_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_saits_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_saits_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.0695110496684588), ('Bilirubin', 0.9915184933318177), ('TroponinI', 0.6146236310366883), ('TroponinT', 0.44333031899542663), ('Glucose', 0.416639242489911)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 0.2895575098137716), ('SaO2', 0.2683085879682801), ('TroponinI', 0.1622792886545023), ('Bilirubin', 0.1397830646769168), ('PaCO2', 0.1369607780183822)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.4194546739320397), ('TroponinI', 0.2691988036465801), ('SaO2', 0.24053120129581373), ('TroponinT', 0.22793862706114962), ('Temp', 0.15260043693171418)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8884223546988217), ('SaO2', 0.25746554212335293), ('Albumin', 0.20186192626912858), ('Glucose', 0.1748223376573

In [30]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_saits_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_saits_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_saits_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_saits_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_saits_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 0.9982780836518492), ('Bilirubin', 0.8517354286549008), ('Cholesterol', 0.6024235381405012), ('Glucose', 0.45734829458249626), ('TroponinI', 0.45234434238218596)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.2036099992152576), ('Bilirubin', 1.1366820218459297), ('TroponinI', 0.8838224346832684), ('Cholesterol', 0.7323207022587693), ('Albumin', 0.45063840953236967)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.2290137796990863), ('Bilirubin', 1.0050968827848537), ('Ph', 0.8831229129347288), ('Albumin', 0.5967439438309785), ('Glucose', 0.5914615801472558)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.2728403997417839), ('Bilirubin', 1.1380893342352523), ('TroponinT', 0.7408628093945846), ('Albumin', 0.6101654410062806), ('Glucose', 0.395

In [31]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_saits_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_saits_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_saits_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_saits_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('TroponinI', 0.4314780923010824), ('Bilirubin', 0.2849465931910288), ('ALP', 0.2053319155634084), ('TroponinT', 0.19265691687489067), ('Cholesterol', 0.12989716411826813)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8949578059134441), ('ALP', 0.23073569604723718), ('Albumin', 0.18762873907869004), ('Mg', 0.17643656092671073), ('Cholesterol', 0.16955198480248967)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.1489114182037523), ('Cholesterol', 0.8242084172433273), ('TroponinI', 0.45234434238218596), ('Bilirubin', 0.28635390558035134), ('ALP', 0.27456231608993475)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('TroponinT', 0.9380846965551555), ('Cholesterol', 0.8267873356681623), ('ALP', 0.23797857377517134), ('ALT', 0.22522062790349448), 

In [32]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_saits_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_saits_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_saits_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Ph', 0.8907623608787905), ('TroponinI', 0.3704914364449188), ('Cholesterol', 0.2994491489207578), ('Glucose', 0.15595710055083023), ('Albumin', 0.14610553429860884)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinT', 0.9562545013288617), ('Cholesterol', 0.9541055813615954), ('TroponinI', 0.8838224346832684), ('Albumin', 0.1595270314739109), ('K', 0.11861301604813246)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 0.9566844997864304), ('TroponinT', 0.7454277796802649), ('TroponinI', 0.5011837233592029), ('Glucose', 0.17570701271287492), ('FiO2', 0.16851930435322476)]
----------------------------


In [33]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_saits_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_saits_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.1014465142680878), ('Ph', 0.8916481947347921), ('Cholesterol', 0.6546564324408376), ('TroponinI', 0.5133309982383496), ('Glucose', 0.19586706902845652)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Ph', 0.8969041099117874), ('TroponinT', 0.890619792619491), ('Cholesterol', 0.6572353508656726), ('Glucose', 0.33166411326370515), ('ALT', 0.18581162118574304)]
----------------------------


In [34]:

print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_saits_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('TroponinI', 0.38263871132406546), ('TroponinT', 0.21082672164859684), ('K', 0.16598073639207245), ('Glucose', 0.13579704423524863), ('HCO3', 0.1312658019106418)]
----------------------------


In [35]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[32])
print(variables[0])
print(variables[6])
print(variables[31])
print(variables[5])

Top 5 variáveis BMI
-----------------------
TroponinT
ALP
Cholesterol
TroponinI
Bilirubin


### Confidence Interval (General)(subgroup)

In [ ]:
results_bootstrap_saits_general_subgroup = []

for i in range(len(subgroups)):
    results_bootstrap_saits_general_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard_general, indicating_mask_variable_standard_norm_general))

In [21]:
teste = "testw-e"

### Confidence Interval (General)(variable)

#### Distribution Bootstrap per variable

In [24]:
results_bootstrap_saits_general = []

for i in range(len(variables)):
       results_bootstrap_saits_general.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 0, i, 9000))

#### Calculating lower bound and upper bound

In [25]:
lower_bounds_saits_general, upper_bounds_saits_general = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_general)

print(lower_bounds_saits_general)
print(upper_bounds_saits_general)

[np.float64(0.2596281057729566), np.float64(0.1112169907702202), np.float64(0.09949139341087465), np.float64(0.5223233752623047), np.float64(0.17760603494641872), np.float64(0.27366120673807326), np.float64(0.6631323201661581), np.float64(0.15261946123918443), np.float64(0.17140566932331605), np.float64(0.40250615530593536), np.float64(0.21712587911182318), np.float64(0.4862783315866672), np.float64(0.4066374786039976), np.float64(0.5095917845984383), np.float64(0.23823871445282752), np.float64(0.5780308760529224), np.float64(0.3402109659972705), np.float64(0.14454482230655688), np.float64(0.004060595869510555), np.float64(0.5026499766120278), np.float64(0.12697301347690718), np.float64(0.11049653990748827), np.float64(0.16754623860623757), np.float64(0.408117332885036), np.float64(0.477918144474126), np.float64(0.45175884519225656), np.float64(0.31559009590229986), np.float64(0.45503263748354783), np.float64(0.27492416924952034), np.float64(0.19586839617850932), np.float64(0.259151256

#### Mean values of lower bound and upper bound


In [26]:
mean_values_ci_saits_general = toolkits.calc_mean_values_ci(lower_bounds_saits_general, upper_bounds_saits_general)

print(mean_values_ci_saits_general)

[np.float64(0.4166398475217067), np.float64(0.1865519913329063), np.float64(0.19241246135912216), np.float64(0.594209326138409), np.float64(0.1909421167411644), np.float64(0.3683471385763335), np.float64(0.9075044774157551), np.float64(0.17555044040538534), np.float64(0.17785917743795066), np.float64(0.426983480708703), np.float64(0.22741745578734018), np.float64(0.5318135670717179), np.float64(0.43316131466810287), np.float64(0.5374377925934044), np.float64(0.2432502182765221), np.float64(0.618085364974358), np.float64(0.38048023409496456), np.float64(0.1545837206699681), np.float64(0.004250779514051089), np.float64(0.5572594475031175), np.float64(0.1331053237819329), np.float64(0.11577085698966622), np.float64(0.17694711257878304), np.float64(0.4359770555763244), np.float64(0.5065294803812029), np.float64(0.48268719939478977), np.float64(0.3415796344515422), np.float64(0.47092299958295686), np.float64(0.3402989443248696), np.float64(0.20224582261134733), np.float64(0.2740741303215406

### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [36]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_saits_female = []

for i in indices:
       results_bootstrap_saits_female.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [37]:
means_bootstraps_saits_female,standards_deviations_saits_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_female)

print(means_bootstraps_saits_female)
print(standards_deviations_saits_female)


[np.float64(1.3709565954271359), np.float64(0.37340746641577754), np.float64(0.5514148109724911), np.float64(0.33219783151411125), np.float64(0.3127126006634792), np.float64(0.1344058768679122), np.float64(0.15004390659670197), np.float64(0.6290111725113285), np.float64(0.25990652535575487), np.float64(0.134371002704543), np.float64(0.08328852545934053), np.float64(0.5210149778023866)]
[np.float64(0.28141828979738265), np.float64(0.07163803672687734), np.float64(0.13740379564534702), np.float64(0.02541877537794542), np.float64(0.008793741630019346), np.float64(0.04882230345968876), np.float64(0.010893276810424193), np.float64(0.033966982819504714), np.float64(0.056886855698661046), np.float64(0.048427337503807684), np.float64(0.004729043421165223), np.float64(0.02245024712647019)]


#### Calculating lower bound and upper bound

In [38]:
lower_bounds_saits_female, upper_bounds_saits_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_female, means_bootstraps_saits_female,standards_deviations_saits_female)

print(lower_bounds_saits_female)
print(upper_bounds_saits_female)

[np.float64(0.8193868828334101), np.float64(0.2329994945079403), np.float64(0.28210832016850945), np.float64(0.28237794724222476), np.float64(0.2954771837792908), np.float64(0.03871592044463695), np.float64(0.1286934763746452), np.float64(0.5624371095216085), np.float64(0.1484103369926521), np.float64(0.039455165329914096), np.float64(0.0740197706725306), np.float64(0.4770133019904812)]
[np.float64(1.9225263080208617), np.float64(0.5138154383236148), np.float64(0.8207213017764727), np.float64(0.38201771578599775), np.float64(0.32994801754766767), np.float64(0.23009583329118743), np.float64(0.17139433681875874), np.float64(0.6955852355010486), np.float64(0.37140271371885764), np.float64(0.22928684007917188), np.float64(0.09255728024615045), np.float64(0.565016653614292)]


In [39]:
lower_bounds_saits_female, upper_bounds_saits_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_female)

print(lower_bounds_saits_female)
print(upper_bounds_saits_female)

[np.float64(0.7956492099657536), np.float64(0.23760147981080015), np.float64(0.31828644884334323), np.float64(0.28378240468009003), np.float64(0.29572219940107847), np.float64(0.06067526607776592), np.float64(0.1300153151859639), np.float64(0.564655620050617), np.float64(0.16172599007870753), np.float64(0.05922063882042538), np.float64(0.07466409156145183), np.float64(0.47796719947937777)]
[np.float64(1.9226152106244694), np.float64(0.5194834637979123), np.float64(0.8484750064650793), np.float64(0.3837919745568725), np.float64(0.3300071456578719), np.float64(0.24500733036190417), np.float64(0.17308137833531811), np.float64(0.6969997681732706), np.float64(0.3813886456340512), np.float64(0.24560340589051607), np.float64(0.09309534886068198), np.float64(0.5657639504442826)]


#### Mean values of lower bound and upper bound


In [40]:
mean_values_ci_saits_female = toolkits.calc_mean_values_ci(lower_bounds_saits_female, upper_bounds_saits_female)

print(mean_values_ci_saits_female)

[np.float64(1.3591322102951116), np.float64(0.37854247180435624), np.float64(0.5833807276542113), np.float64(0.33378718961848125), np.float64(0.3128646725294752), np.float64(0.15284129821983505), np.float64(0.151548346760641), np.float64(0.6308276941119437), np.float64(0.2715573178563794), np.float64(0.15241202235547072), np.float64(0.0838797202110669), np.float64(0.5218655749618302)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [41]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_saits_male = []

for i in indices:
       results_bootstrap_saits_male.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [42]:
means_bootstraps_saits_male,standards_deviations_saits_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_male)

print(means_bootstraps_saits_male)
print(standards_deviations_saits_male)


[np.float64(0.8438058109897287), np.float64(0.5869423566271345), np.float64(0.3518805600283484), np.float64(0.43558126847157913), np.float64(0.375552431719391), np.float64(0.09281901372823773), np.float64(0.1690444660224031), np.float64(0.5757740658688472), np.float64(0.27776270400965464), np.float64(0.09338513088853735), np.float64(0.07236860789377567), np.float64(0.4871324217581555)]
[np.float64(0.14484001062154944), np.float64(0.1704863043219284), np.float64(0.03277833988488178), np.float64(0.03375522091077515), np.float64(0.011914226378457826), np.float64(0.019806856883581282), np.float64(0.009005653419694348), np.float64(0.02569829910524861), np.float64(0.04482147852835932), np.float64(0.01992827195446364), np.float64(0.0030149581872297346), np.float64(0.019178377337048934)]


#### Calculating lower bound and upper bound

In [43]:
lower_bounds_saits_male, upper_bounds_saits_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_male, means_bootstraps_saits_male,standards_deviations_saits_male)

print(lower_bounds_saits_male)
print(upper_bounds_saits_male)

[np.float64(0.559924606651093), np.float64(0.2527953402988194), np.float64(0.2876361943809673), np.float64(0.3694222511962665), np.float64(0.3522009771139566), np.float64(0.05399828758947916), np.float64(0.15139370966255222), np.float64(0.525406325158622), np.float64(0.18991422036023503), np.float64(0.05432643558366898), np.float64(0.06645939843191122), np.float64(0.4495434928956204)]
[np.float64(1.1276870153283645), np.float64(0.9210893729554495), np.float64(0.41612492567572945), np.float64(0.5017402857468918), np.float64(0.39890388632482543), np.float64(0.13163973986699629), np.float64(0.186695222382254), np.float64(0.6261418065790724), np.float64(0.36561118765907424), np.float64(0.13244382619340572), np.float64(0.07827781735564011), np.float64(0.5247213506206906)]


In [44]:
lower_bounds_saits_male, upper_bounds_saits_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_male)

print(lower_bounds_saits_male)
print(upper_bounds_saits_male)

[np.float64(0.5543691095812918), np.float64(0.24383298697774514), np.float64(0.2913003458887765), np.float64(0.3728388718325378), np.float64(0.3534882648294865), np.float64(0.059506104688290054), np.float64(0.15194163170799502), np.float64(0.5272282213020713), np.float64(0.1988940094151244), np.float64(0.059478934499948836), np.float64(0.06681734315554012), np.float64(0.4504542166091075)]
[np.float64(1.1198583711659658), np.float64(0.9200887406982938), np.float64(0.4211647434429169), np.float64(0.5048942135005), np.float64(0.40002559431022744), np.float64(0.1365678947406039), np.float64(0.18747326919939217), np.float64(0.6277415611360948), np.float64(0.3747574600297154), np.float64(0.1377540376898181), np.float64(0.07844891936116019), np.float64(0.5255574881262816)]


#### Mean values of lower bound and upper bound


In [45]:
mean_values_ci_saits_male = toolkits.calc_mean_values_ci(lower_bounds_saits_male, upper_bounds_saits_male)

print(mean_values_ci_saits_male)

[np.float64(0.8371137403736288), np.float64(0.5819608638380195), np.float64(0.3562325446658467), np.float64(0.4388665426665189), np.float64(0.37675692956985696), np.float64(0.09803699971444699), np.float64(0.1697074504536936), np.float64(0.5774848912190831), np.float64(0.2868257347224199), np.float64(0.09861648609488347), np.float64(0.07263313125835016), np.float64(0.48800585236769456)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [46]:
indices = [6, 32, 0, 28, 5, 31, 3]


results_bootstrap_saits_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_saits_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [47]:
means_bootstraps_saits_more_than_or_equal_to_65_subgroup,standards_deviations_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_saits_more_than_or_equal_to_65_subgroup)
print(standards_deviations_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.6840107251428338), np.float64(0.306890478696312), np.float64(0.5410854493791742), np.float64(0.3593861740725029), np.float64(0.2174941018946004), np.float64(0.9223744805456358), np.float64(0.5744216340030307)]
[np.float64(0.17674009842842184), np.float64(0.06812227123612309), np.float64(0.11748070729527743), np.float64(0.033110838324512136), np.float64(0.024756910288949063), np.float64(0.323474177873407), np.float64(0.060833523112653914)]


#### Calculating lower bound and upper bound


In [48]:
lower_bounds_saits_more_than_or_equal_to_65_subgroup, upper_bounds_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_more_than_or_equal_to_65_subgroup, means_bootstraps_saits_more_than_or_equal_to_65_subgroup,standards_deviations_saits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_saits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_saits_more_than_or_equal_to_65_subgroup)


[np.float64(0.33760649759906275), np.float64(0.17337328052844184), np.float64(0.3108274942021385), np.float64(0.29449012345853054), np.float64(0.16897144935977113), np.float64(0.2883767419850548), np.float64(0.45519011964954403)]
[np.float64(1.0304149526866049), np.float64(0.44040767686418214), np.float64(0.77134340455621), np.float64(0.42428222468647525), np.float64(0.2660167544294296), np.float64(1.5563722191062168), np.float64(0.6936531483565174)]


In [49]:
lower_bounds_saits_more_than_or_equal_to_65_subgroup, upper_bounds_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_saits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.35107048872115837), np.float64(0.19405457858945557), np.float64(0.3309408378621302), np.float64(0.3012529987658585), np.float64(0.1722652547233647), np.float64(0.3744696407841734), np.float64(0.4631869896193011)]
[np.float64(1.0522101190096362), np.float64(0.45790638712920423), np.float64(0.7899029574949497), np.float64(0.43035416737196197), np.float64(0.26915957286882153), np.float64(1.6354705428625909), np.float64(0.6978495924750472)]


#### Mean values of lower bound and upper bound

In [50]:
mean_values_ci_saits_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_more_than_or_equal_to_65_subgroup , upper_bounds_saits_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.7016403038653973), np.float64(0.3259804828593299), np.float64(0.5604218976785399), np.float64(0.36580358306891025), np.float64(0.22071241379609313), np.float64(1.0049700918233822), np.float64(0.5805182910471741)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [51]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_saits_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_saits_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [52]:
means_bootstraps_saits_less_than_65_subgroup,standards_deviations_saits_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_less_than_65_subgroup)

print(means_bootstraps_saits_less_than_65_subgroup)
print(standards_deviations_saits_less_than_65_subgroup)

[np.float64(1.2071296148172286), np.float64(0.2725344788786177), np.float64(0.8978045284833425), np.float64(0.5526726596231104), np.float64(0.4140160720931158), np.float64(0.3646116414487031), np.float64(0.6601403948860949)]
[np.float64(0.7955134454995609), np.float64(0.08230259752908997), np.float64(0.3725300533810867), np.float64(0.13485242331156377), np.float64(0.07105143502438786), np.float64(0.08783285426784743), np.float64(0.05310376481708998)]


#### Calculating lower bound and upper bound

In [53]:
lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_less_than_65_subgroup, means_bootstraps_saits_less_than_65_subgroup, standards_deviations_saits_less_than_65_subgroup)

print(lower_bounds_saits_less_than_65_subgroup)
print(upper_bounds_saits_less_than_65_subgroup)


[np.float64(-0.35204808757927797), np.float64(0.11122435188750612), np.float64(0.16765904069762871), np.float64(0.2883667667044958), np.float64(0.2747578183954278), np.float64(0.192462410424367), np.float64(0.5560589284011133)]
[np.float64(2.766307317213735), np.float64(0.4338446058697293), np.float64(1.627950016269056), np.float64(0.816978552541725), np.float64(0.5532743257908038), np.float64(0.5367608724730393), np.float64(0.7642218613710765)]


In [54]:
lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_less_than_65_subgroup)

print(lower_bounds_saits_less_than_65_subgroup)
print(upper_bounds_saits_less_than_65_subgroup)


[np.float64(0.0), np.float64(0.1376943592692674), np.float64(0.3233972572576607), np.float64(0.32445719658454286), np.float64(0.286189075230866), np.float64(0.1889877196050605), np.float64(0.5601331906484173)]
[np.float64(2.297978957925913), np.float64(0.4524516282420147), np.float64(1.730549831346401), np.float64(0.8480803514414874), np.float64(0.5624647859409228), np.float64(0.5277707171008981), np.float64(0.7670367606413029)]


#### Mean values of lower bound and upper bound

In [55]:
mean_values_ci_saits_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup)

print(mean_values_ci_saits_less_than_65_subgroup)

[np.float64(1.1489894789629564), np.float64(0.29507299375564106), np.float64(1.026973544302031), np.float64(0.5862687740130151), np.float64(0.4243269305858944), np.float64(0.35837921835297926), np.float64(0.6635849756448601)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [56]:
indices = [28, 31, 6, 0, 32, 24]




results_bootstrap_saits_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [57]:
means_bootstraps_saits_ICUType_1_subgroup, standards_deviations_saits_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_1_subgroup)

print(means_bootstraps_saits_ICUType_1_subgroup)
print(standards_deviations_saits_ICUType_1_subgroup)


[np.float64(0.5219567254652908), np.float64(1.4191329972717064), np.float64(0.9139329648049112), np.float64(0.21569407166429788), np.float64(0.7211401097401062), np.float64(0.4648222222352919)]
[np.float64(0.18500220990963184), np.float64(0.4711921029067144), np.float64(0.19021476797505155), np.float64(0.03900834305972772), np.float64(0.1525881619809211), np.float64(0.04182740358682482)]


#### Calculating lower bound and upper bound

In [58]:
lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_1_subgroup, means_bootstraps_saits_ICUType_1_subgroup, standards_deviations_saits_ICUType_1_subgroup)

print(lower_bounds_saits_ICUType_1_subgroup)
print(upper_bounds_saits_ICUType_1_subgroup)

[np.float64(0.1593590569820933), np.float64(0.4956134457748552), np.float64(0.5411188702461672), np.float64(0.1392391241706486), np.float64(0.42207280779033696), np.float64(0.3828420176382938)]
[np.float64(0.8845543939484883), np.float64(2.3426525487685574), np.float64(1.286747059363655), np.float64(0.29214901915794717), np.float64(1.0202074116898756), np.float64(0.54680242683229)]


In [59]:
lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_1_subgroup)

print(lower_bounds_saits_ICUType_1_subgroup)
print(upper_bounds_saits_ICUType_1_subgroup)

[np.float64(0.2724862865316178), np.float64(0.5714906008460912), np.float64(0.5423973514465497), np.float64(0.14321453098769518), np.float64(0.4430609084144976), np.float64(0.38418463195425406)]
[np.float64(0.9495419093825632), np.float64(2.412294909547965), np.float64(1.2983912355171199), np.float64(0.2973851173252235), np.float64(1.0472720745698492), np.float64(0.5491950726501879)]


#### Mean values of lower bound and upper bound

In [60]:
mean_values_ci_saits_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup)

print(mean_values_ci_saits_ICUType_1_subgroup)

[np.float64(0.6110140979570905), np.float64(1.491892755197028), np.float64(0.9203942934818348), np.float64(0.22029982415645932), np.float64(0.7451664914921734), np.float64(0.46668985230222093)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [61]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_saits_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [62]:
means_bootstraps_saits_ICUType_2_subgroup, standards_deviations_saits_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_2_subgroup)

print(means_bootstraps_saits_ICUType_2_subgroup)
print(standards_deviations_saits_ICUType_2_subgroup)


[np.float64(0.30234261908205756), np.float64(0.4822451154959138), np.float64(0.0), np.float64(0.11505758221413943), np.float64(0.11150250301424608), np.float64(0.44450325827242426)]
[np.float64(0.02262483990234257), np.float64(0.14270434095330978), np.float64(0.0), np.float64(0.015155524044065732), np.float64(0.011272872453300223), np.float64(0.018251955560383098)]


#### Calculating lower bound and upper bound

In [63]:
lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_2_subgroup, means_bootstraps_saits_ICUType_2_subgroup, standards_deviations_saits_ICUType_2_subgroup)

print(lower_bounds_saits_ICUType_2_subgroup)
print(upper_bounds_saits_ICUType_2_subgroup)

[np.float64(0.25799874771748144), np.float64(0.20254974678990234), np.float64(0.0), np.float64(0.08535330092093976), np.float64(0.08940807900346395), np.float64(0.4087300827266478)]
[np.float64(0.3466864904466337), np.float64(0.7619404842019253), np.float64(0.0), np.float64(0.1447618635073391), np.float64(0.1335969270250282), np.float64(0.48027643381820073)]


In [64]:
lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_2_subgroup)

print(lower_bounds_saits_ICUType_2_subgroup)
print(upper_bounds_saits_ICUType_2_subgroup)

[np.float64(0.2612377146330772), np.float64(0.18986491055554502), np.float64(0.0), np.float64(0.08598441095084189), np.float64(0.09026797114338445), np.float64(0.40965400315649914)]
[np.float64(0.3491110665417255), np.float64(0.7859344037034782), np.float64(0.0), np.float64(0.14570947953394509), np.float64(0.1345600276637829), np.float64(0.4812821650776725)]


#### Mean values of lower bound and upper bound

In [65]:
mean_values_ci_saits_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup)

print(mean_values_ci_saits_ICUType_2_subgroup)

[np.float64(0.30517439058740137), np.float64(0.4878996571295116), np.float64(0.0), np.float64(0.11584694524239349), np.float64(0.11241399940358367), np.float64(0.4454680841170858)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [66]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_saits_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [67]:
means_bootstraps_saits_ICUType_3_subgroup, standards_deviations_saits_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_3_subgroup)

print(means_bootstraps_saits_ICUType_3_subgroup)
print(standards_deviations_saits_ICUType_3_subgroup)


[np.float64(1.5873176168992222), np.float64(0.3451376351349188), np.float64(0.8838193565096376), np.float64(0.9719416794769047), np.float64(0.17008102185635626), np.float64(0.6029704811331233)]
[np.float64(0.4492088863497037), np.float64(0.0655364408499348), np.float64(0.17397706126823817), np.float64(0.3872023228794582), np.float64(0.04196961293018973), np.float64(0.03460610203052623)]


#### Calculating lower bound and upper bound

In [68]:
lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_3_subgroup, means_bootstraps_saits_ICUType_3_subgroup, standards_deviations_saits_ICUType_3_subgroup)

print(lower_bounds_saits_ICUType_3_subgroup)
print(upper_bounds_saits_ICUType_3_subgroup)

[np.float64(0.7068843781184567), np.float64(0.21668857139410702), np.float64(0.5428305822877724), np.float64(0.21303907190291727), np.float64(0.08782209206809784), np.float64(0.5351437675079735)]
[np.float64(2.467750855679988), np.float64(0.47358669887573057), np.float64(1.2248081307315029), np.float64(1.7308442870508922), np.float64(0.2523399516446147), np.float64(0.6707971947582732)]


In [69]:
lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_3_subgroup)

print(lower_bounds_saits_ICUType_3_subgroup)
print(upper_bounds_saits_ICUType_3_subgroup)

[np.float64(0.780404067787305), np.float64(0.2202083405208857), np.float64(0.6727157881964088), np.float64(0.3642860059565915), np.float64(0.09971476394281487), np.float64(0.5375214962234229)]
[np.float64(2.54731502169647), np.float64(0.4801916645249752), np.float64(1.2507688888161221), np.float64(1.8416487085107636), np.float64(0.2615151655889945), np.float64(0.6728665632862576)]


#### Mean values of lower bound and upper bound

In [70]:
mean_values_ci_saits_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup)

print(mean_values_ci_saits_ICUType_3_subgroup)

[np.float64(1.6638595447418876), np.float64(0.35020000252293043), np.float64(0.9617423385062654), np.float64(1.1029673572336776), np.float64(0.1806149647659047), np.float64(0.6051940297548403)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [71]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_saits_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [72]:
means_bootstraps_saits_ICUType_4_subgroup, standards_deviations_saits_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_4_subgroup)

print(means_bootstraps_saits_ICUType_4_subgroup)
print(standards_deviations_saits_ICUType_4_subgroup)


[np.float64(0.4374924981118807), np.float64(0.7710783833108681), np.float64(0.7361433475539954), np.float64(0.3833519769442785), np.float64(0.15810549637325846), np.float64(0.4660701136770505)]
[np.float64(0.09361222958290319), np.float64(0.3506626777117994), np.float64(0.17848971361751265), np.float64(0.06867994998652105), np.float64(0.026035599351315757), np.float64(0.021997324882899502)]


#### Calculating lower bound and upper bound

In [73]:
lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_4_subgroup, means_bootstraps_saits_ICUType_4_subgroup, standards_deviations_saits_ICUType_4_subgroup)

print(lower_bounds_saits_ICUType_4_subgroup)
print(upper_bounds_saits_ICUType_4_subgroup)

[np.float64(0.2540158996168954), np.float64(0.083792164273365), np.float64(0.3863099372528021), np.float64(0.2487417485106851), np.float64(0.10707665932876519), np.float64(0.4229561491503407)]
[np.float64(0.620969096606866), np.float64(1.4583646023483712), np.float64(1.0859767578551887), np.float64(0.517962205377872), np.float64(0.20913433341775173), np.float64(0.5091840782037602)]


In [74]:
lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_4_subgroup)

print(lower_bounds_saits_ICUType_4_subgroup)
print(upper_bounds_saits_ICUType_4_subgroup)

[np.float64(0.27930576874390706), np.float64(0.3728340243467206), np.float64(0.36871716024740075), np.float64(0.26153937912889297), np.float64(0.10962410035351977), np.float64(0.4232686475006617)]
[np.float64(0.6447007036079485), np.float64(1.8003804990136956), np.float64(1.0708848629739434), np.float64(0.527720487030236), np.float64(0.21165193489381803), np.float64(0.5101341787631254)]


#### Mean values of lower bound and upper bound

In [75]:
mean_values_ci_saits_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup)

print(mean_values_ci_saits_ICUType_4_subgroup)

[np.float64(0.46200323617592776), np.float64(1.086607261680208), np.float64(0.7198010116106721), np.float64(0.3946299330795645), np.float64(0.16063801762366892), np.float64(0.4667014131318935)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [76]:
indices = [32, 0, 6, 31, 5, 1, 35]


results_bootstrap_saits_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [77]:
means_bootstraps_saits_undefined_classification_subgroup, standards_deviations_saits_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_undefined_classification_subgroup)

print(means_bootstraps_saits_undefined_classification_subgroup)
print(standards_deviations_saits_undefined_classification_subgroup)


[np.float64(0.2220789243699558), np.float64(0.36322793707051365), np.float64(0.5358393333607179), np.float64(0.6174529701974534), np.float64(0.34997728226605834), np.float64(0.1486205100765959), np.float64(0.09019084920226916)]
[np.float64(0.04570653322260835), np.float64(0.05263916241599928), np.float64(0.1666043195868283), np.float64(0.21180064268718604), np.float64(0.10245179483312185), np.float64(0.04951555561227327), np.float64(0.005528848192846053)]


#### Calculating lower bound and upper bound

In [78]:
lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_undefined_classification_subgroup, means_bootstraps_saits_undefined_classification_subgroup, standards_deviations_saits_undefined_classification_subgroup)

print(lower_bounds_saits_undefined_classification_subgroup)
print(upper_bounds_saits_undefined_classification_subgroup)

[np.float64(0.13249576539545999), np.float64(0.26005707455880067), np.float64(0.2093008673017333), np.float64(0.20233133862813196), np.float64(0.1491754542416527), np.float64(0.051571804402050156), np.float64(0.07935450586830153)]
[np.float64(0.3116620833444516), np.float64(0.46639879958222663), np.float64(0.8623777994197024), np.float64(1.0325746017667747), np.float64(0.550779110290464), np.float64(0.24566921575114165), np.float64(0.10102719253623679)]


In [79]:
lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_undefined_classification_subgroup)

print(lower_bounds_saits_undefined_classification_subgroup)
print(upper_bounds_saits_undefined_classification_subgroup)

[np.float64(0.14861351880442525), np.float64(0.2681309927056473), np.float64(0.21667597714838674), np.float64(0.22678503532965885), np.float64(0.19076055650846094), np.float64(0.07058041202715513), np.float64(0.079863334033074)]
[np.float64(0.32444401367727815), np.float64(0.4722891131963517), np.float64(0.8712429101228379), np.float64(1.0597512498483057), np.float64(0.5819828141296248), np.float64(0.26020473146627615), np.float64(0.10139542073969701)]


#### Mean values of lower bound and upper bound

In [80]:
mean_values_ci_saits_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup)

print(mean_values_ci_saits_undefined_classification_subgroup)

[np.float64(0.2365287662408517), np.float64(0.3702100529509995), np.float64(0.5439594436356123), np.float64(0.6432681425889822), np.float64(0.38637168531904287), np.float64(0.16539257174671565), np.float64(0.0906293773863855)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [81]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [82]:
means_bootstraps_saits_low_weight_classification_subgroup, standards_deviations_saits_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_low_weight_classification_subgroup)

print(means_bootstraps_saits_low_weight_classification_subgroup)
print(standards_deviations_saits_low_weight_classification_subgroup)


[np.float64(0.4179372777585539), np.float64(1.2434660978146057), np.float64(0.14389198214777257), np.float64(0.0), np.float64(1.3355777932607396), np.float64(0.05999720428400326), np.float64(0.08402665695116406)]
[np.float64(0.3211289804440317), np.float64(1.086221728773829), np.float64(0.10760691902498223), np.float64(0.0), np.float64(1.11933454327662), np.float64(0.020827781538153973), np.float64(0.008895762289132428)]


#### Calculating lower bound and upper bound

In [83]:
lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_low_weight_classification_subgroup, means_bootstraps_saits_low_weight_classification_subgroup, standards_deviations_saits_low_weight_classification_subgroup)

print(lower_bounds_saits_low_weight_classification_subgroup)
print(upper_bounds_saits_low_weight_classification_subgroup)

[np.float64(-0.21146395830381565), np.float64(-0.8854893698069339), np.float64(-0.06701370362851056), np.float64(0.0), np.float64(-0.858277598213026), np.float64(0.01917550259135322), np.float64(0.06659128324943492)]
[np.float64(1.0473385138209235), np.float64(3.372421565436145), np.float64(0.3547976679240557), np.float64(0.0), np.float64(3.529433184734505), np.float64(0.1008189059766533), np.float64(0.10146203065289319)]


In [84]:
lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_low_weight_classification_subgroup)

print(lower_bounds_saits_low_weight_classification_subgroup)
print(upper_bounds_saits_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.06532619020593648), np.float64(0.02636130159392695), np.float64(0.06807568231807692)]
[np.float64(0.6646820109257148), np.float64(2.8082561340301457), np.float64(0.22436379752781074), np.float64(0.0), np.float64(3.898138116424551), np.float64(0.10568388666059643), np.float64(0.10248588150567063)]


#### Mean values of lower bound and upper bound

In [85]:
mean_values_ci_saits_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup)

print(mean_values_ci_saits_low_weight_classification_subgroup)

[np.float64(0.3323410054628574), np.float64(1.4041280670150729), np.float64(0.11218189876390537), np.float64(0.0), np.float64(1.9817321533152439), np.float64(0.06602259412726169), np.float64(0.08528078191187377)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [86]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [87]:
means_bootstraps_saits_normal_weight_classification_subgroup, standards_deviations_saits_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_normal_weight_classification_subgroup)

print(means_bootstraps_saits_normal_weight_classification_subgroup)
print(standards_deviations_saits_normal_weight_classification_subgroup)


[np.float64(0.25744368744577145), np.float64(0.4349773922115265), np.float64(0.717382366934424), np.float64(0.4541860461731961), np.float64(0.49248182712618804), np.float64(0.061875693671140285), np.float64(0.061840634043035705)]
[np.float64(0.06530091398193434), np.float64(0.1130191030039274), np.float64(0.36496621546347674), np.float64(0.17155187474732164), np.float64(0.16924381363494684), np.float64(0.017466578077892432), np.float64(0.0036097147752741235)]


#### Calculating lower bound and upper bound

In [88]:
lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_normal_weight_classification_subgroup, means_bootstraps_saits_normal_weight_classification_subgroup, standards_deviations_saits_normal_weight_classification_subgroup)

print(lower_bounds_saits_normal_weight_classification_subgroup)
print(upper_bounds_saits_normal_weight_classification_subgroup)

[np.float64(0.12945624788363208), np.float64(0.21346402075880616), np.float64(0.002061729052124317), np.float64(0.11795055018811929), np.float64(0.16077004779548332), np.float64(0.027641829705314272), np.float64(0.05476572308903633)]
[np.float64(0.3854311270079108), np.float64(0.6564907636642467), np.float64(1.4327030048167237), np.float64(0.7904215421582729), np.float64(0.8241936064568928), np.float64(0.0961095576369663), np.float64(0.06891554499703509)]


In [89]:
lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_normal_weight_classification_subgroup)

print(lower_bounds_saits_normal_weight_classification_subgroup)
print(upper_bounds_saits_normal_weight_classification_subgroup)

[np.float64(0.14427118713539758), np.float64(0.24249607025212516), np.float64(0.0), np.float64(0.12762180966123823), np.float64(0.22082274351885345), np.float64(0.03289849328793306), np.float64(0.05517902856700568)]
[np.float64(0.39878917968234284), np.float64(0.6804758208233153), np.float64(1.157952311730471), np.float64(0.810228541120111), np.float64(0.8680464036506664), np.float64(0.10027582513098275), np.float64(0.06920380693573645)]


#### Mean values of lower bound and upper bound

In [90]:
mean_values_ci_saits_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup)

print(mean_values_ci_saits_normal_weight_classification_subgroup)

[np.float64(0.2715301834088702), np.float64(0.4614859455377202), np.float64(0.5789761558652355), np.float64(0.46892517539067463), np.float64(0.54443457358476), np.float64(0.0665871592094579), np.float64(0.06219141775137106)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [91]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [92]:
means_bootstraps_saits_overweight_classification_subgroup, standards_deviations_saits_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_overweight_classification_subgroup)

print(means_bootstraps_saits_overweight_classification_subgroup)
print(standards_deviations_saits_overweight_classification_subgroup)


[np.float64(0.44788379857909943), np.float64(0.22978055068985156), np.float64(0.9617003448767828), np.float64(0.8690059501858922), np.float64(0.20889000167879582), np.float64(0.1554328496222451), np.float64(0.05987518990506586)]
[np.float64(0.1963573758845288), np.float64(0.046913847941276524), np.float64(0.21814397668322832), np.float64(0.5515070410248044), np.float64(0.06296380705055063), np.float64(0.04989517332353856), np.float64(0.0029144577918172316)]


#### Calculating lower bound and upper bound

In [93]:
lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_overweight_classification_subgroup, means_bootstraps_saits_overweight_classification_subgroup, standards_deviations_saits_overweight_classification_subgroup)

print(lower_bounds_saits_overweight_classification_subgroup)
print(upper_bounds_saits_overweight_classification_subgroup)

[np.float64(0.06303041374662921), np.float64(0.137831098348761), np.float64(0.53414600713331), np.float64(-0.21192798744297847), np.float64(0.08548320753018745), np.float64(0.05764010690572585), np.float64(0.054162957598641946)]
[np.float64(0.8327371834115697), np.float64(0.3217300030309421), np.float64(1.3892546826202556), np.float64(1.949939887814763), np.float64(0.3322967958274042), np.float64(0.25322559233876435), np.float64(0.06558742221148976)]


In [94]:
lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_overweight_classification_subgroup)

print(lower_bounds_saits_overweight_classification_subgroup)
print(upper_bounds_saits_overweight_classification_subgroup)

[np.float64(0.12413702513626826), np.float64(0.14433416587788625), np.float64(0.5280093967897358), np.float64(0.10499430382568216), np.float64(0.1190599458217266), np.float64(0.07927384479062116), np.float64(0.05455167757690384)]
[np.float64(0.8832924474115974), np.float64(0.32829721086014874), np.float64(1.3861176353592872), np.float64(2.0863204489780642), np.float64(0.35445178174038233), np.float64(0.2663539917461887), np.float64(0.06601488658930574)]


#### Mean values of lower bound and upper bound

In [95]:
mean_values_ci_saits_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup)

print(mean_values_ci_saits_overweight_classification_subgroup)

[np.float64(0.5037147362739328), np.float64(0.2363156883690175), np.float64(0.9570635160745116), np.float64(1.095657376401873), np.float64(0.23675586378105445), np.float64(0.17281391826840492), np.float64(0.06028328208310479)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [96]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [97]:
means_bootstraps_saits_obs1_classification_subgroup, standards_deviations_saits_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs1_classification_subgroup)

print(means_bootstraps_saits_obs1_classification_subgroup)
print(standards_deviations_saits_obs1_classification_subgroup)


[np.float64(0.3029890874167423), np.float64(0.20407255566287066), np.float64(0.6208529973138541), np.float64(0.4395335826294013), np.float64(0.3425874360473905), np.float64(0.10179621402459223), np.float64(0.06688625776136924)]
[np.float64(0.08587512838896379), np.float64(0.060811315475714646), np.float64(0.3138483810115808), np.float64(0.21299261450393206), np.float64(0.1145905761579488), np.float64(0.04148364623779175), np.float64(0.0034668575994258057)]


#### Calculating lower bound and upper bound

In [98]:
lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs1_classification_subgroup, means_bootstraps_saits_obs1_classification_subgroup, standards_deviations_saits_obs1_classification_subgroup)

print(lower_bounds_saits_obs1_classification_subgroup)
print(upper_bounds_saits_obs1_classification_subgroup)

[np.float64(0.13467692860662012), np.float64(0.08488456747796673), np.float64(0.005721473924951148), np.float64(0.022075729228670904), np.float64(0.11799403381011664), np.float64(0.020489761451119887), np.float64(0.06009134172696567)]
[np.float64(0.4713012462268645), np.float64(0.3232605438477746), np.float64(1.235984520702757), np.float64(0.8569914360301316), np.float64(0.5671808382846644), np.float64(0.18310266659806457), np.float64(0.0736811737957728)]


In [99]:
lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs1_classification_subgroup)

print(lower_bounds_saits_obs1_classification_subgroup)
print(upper_bounds_saits_obs1_classification_subgroup)

[np.float64(0.15418243606681148), np.float64(0.09379579326862329), np.float64(0.0), np.float64(0.0), np.float64(0.15843812547099442), np.float64(0.04089258082946061), np.float64(0.060593833475975654)]
[np.float64(0.4890280355937019), np.float64(0.3325682569395985), np.float64(1.2406693076044368), np.float64(0.6809092237281027), np.float64(0.6005876011055841), np.float64(0.1972119507960664), np.float64(0.07412772674871662)]


#### Mean values of lower bound and upper bound

In [100]:
mean_values_ci_saits_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup)

print(mean_values_ci_saits_obs1_classification_subgroup)

[np.float64(0.3216052358302567), np.float64(0.2131820251041109), np.float64(0.6203346538022184), np.float64(0.34045461186405135), np.float64(0.37951286328828926), np.float64(0.11905226581276351), np.float64(0.06736078011234614)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [101]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [102]:
means_bootstraps_saits_obs2_classification_subgroup, standards_deviations_saits_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs2_classification_subgroup)

print(means_bootstraps_saits_obs2_classification_subgroup)
print(standards_deviations_saits_obs2_classification_subgroup)


[np.float64(1.3202318318531734), np.float64(0.16018320474459607), np.float64(0.0016333150023958836), np.float64(0.0), np.float64(0.20565140281883998), np.float64(0.1904748878318178), np.float64(0.0748960300784776)]
[np.float64(1.0284769556193523), np.float64(0.04163536032079147), np.float64(0.0012427663723273534), np.float64(0.0), np.float64(0.14065179783514387), np.float64(0.09377300495839148), np.float64(0.006743679991352325)]


#### Calculating lower bound and upper bound

In [103]:
lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs2_classification_subgroup, means_bootstraps_saits_obs2_classification_subgroup, standards_deviations_saits_obs2_classification_subgroup)

print(lower_bounds_saits_obs2_classification_subgroup)
print(upper_bounds_saits_obs2_classification_subgroup)

[np.float64(-0.6955459600901566), np.float64(0.07857939803249675), np.float64(-0.0008024623285632243), np.float64(0.0), np.float64(-0.07002105529885072), np.float64(0.006683175391274582), np.float64(0.06167866017216366)]
[np.float64(3.336009623796503), np.float64(0.2417870114566954), np.float64(0.004069092333354991), np.float64(0.0), np.float64(0.4813238609365307), np.float64(0.374266600272361), np.float64(0.08811339998479154)]


In [104]:
lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs2_classification_subgroup)

print(lower_bounds_saits_obs2_classification_subgroup)
print(upper_bounds_saits_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.08663420909169421), np.float64(0.0), np.float64(0.0), np.float64(0.05801357838495656), np.float64(0.05135770937952164), np.float64(0.062486301173007935)]
[np.float64(3.5585249882396806), np.float64(0.2493556649833264), np.float64(0.0025789184248367356), np.float64(0.0), np.float64(0.5359872611109567), np.float64(0.39997488231726197), np.float64(0.08915991716810763)]


#### Mean values of lower bound and upper bound

In [105]:
mean_values_ci_saits_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup)

print(mean_values_ci_saits_obs2_classification_subgroup)

[np.float64(1.7792624941198403), np.float64(0.1679949370375103), np.float64(0.0012894592124183678), np.float64(0.0), np.float64(0.29700041974795666), np.float64(0.22566629584839182), np.float64(0.07582310917055779)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [106]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [107]:
means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs3_classification_subgroup)

print(means_bootstraps_saits_obs3_classification_subgroup)
print(standards_deviations_saits_obs3_classification_subgroup)


[np.float64(1.174122801941639), np.float64(0.19605819975450417), np.float64(0.0), np.float64(0.24114741895895714), np.float64(0.2807075619561548), np.float64(0.27174246354233467), np.float64(0.12161519628120666)]
[np.float64(0.8152811692023881), np.float64(0.05438973613686845), np.float64(0.0), np.float64(0.18471670189518583), np.float64(0.14691368485557318), np.float64(0.18967925991644569), np.float64(0.01616573041190024)]


#### Calculating lower bound and upper bound

In [108]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs3_classification_subgroup, means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(-0.42379892696874766), np.float64(0.08945627579760532), np.float64(0.0), np.float64(-0.12089066409862864), np.float64(-0.007237969196836225), np.float64(-0.1000220545081108), np.float64(0.08993094689009834)]
[np.float64(2.7720445308520256), np.float64(0.30266012371140305), np.float64(0.0), np.float64(0.6031855020165429), np.float64(0.5686530931091458), np.float64(0.6435069815927801), np.float64(0.153299445672315)]


In [109]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(0.09197122917451109), np.float64(0.10864533313994013), np.float64(0.0), np.float64(0.0), np.float64(0.041572934649777814), np.float64(0.0), np.float64(0.09330420152930746)]
[np.float64(3.00022367963562), np.float64(0.31846198470855486), np.float64(0.0), np.float64(0.38263871132432054), np.float64(0.5959809854230184), np.float64(0.6848965195437268), np.float64(0.15631952632364246)]


#### Mean values of lower bound and upper bound

In [110]:
mean_values_ci_saits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup)

print(mean_values_ci_saits_obs3_classification_subgroup)

[np.float64(1.5460974544050654), np.float64(0.2135536589242475), np.float64(0.0), np.float64(0.19131935566216027), np.float64(0.3187769600363981), np.float64(0.3424482597718634), np.float64(0.12481186392647496)]


### Confidence Interval (Obesity subgroup)

In [54]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_saits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



In [55]:
means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs3_classification_subgroup)

print(means_bootstraps_saits_obs3_classification_subgroup)
print(standards_deviations_saits_obs3_classification_subgroup)


[np.float64(0.3034675566699227), np.float64(0.37128772502346685), np.float64(1.227328997203076), np.float64(0.5441540087161867), np.float64(0.8005086131995904), np.float64(0.13224206350809875), np.float64(0.058576248661024316)]
[np.float64(0.07366508116898239), np.float64(0.13777977709293526), np.float64(0.5815363479168615), np.float64(0.1695030965515837), np.float64(0.26507686121034296), np.float64(0.08729338124447372), np.float64(0.002934270640258915)]


In [56]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(0.1672897025433754), np.float64(0.17955651746199433), np.float64(0.0), np.float64(0.252581169122886), np.float64(0.33651928199329634), np.float64(0.03595718783553412), np.float64(0.0530949042269714)]
[np.float64(0.455034747405991), np.float64(0.6929829025219347), np.float64(2.3429705405575723), np.float64(0.9093829874099302), np.float64(1.3662332160682222), np.float64(0.3368713899817319), np.float64(0.06464601922419128)]


In [57]:
mean_values_ci_saits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup)

print(mean_values_ci_saits_obs3_classification_subgroup)

[np.float64(0.3111622249746832), np.float64(0.4362697099919645), np.float64(1.1714852702787861), np.float64(0.5809820782664081), np.float64(0.8513762490307593), np.float64(0.186414288908633), np.float64(0.05887046172558134)]


## BRITS

### Calculate MAE and AE

In [60]:
testing_mae_brits_variables_standard, testing_ae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for BRITS model

#### Difference between gender group

In [112]:
diff_brits_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 1, 2, variables)
diff_brits_gender_subgroup

[('Cholesterol', 0.33467020046017315),
 ('AST', 0.17888955772137866),
 ('TroponinI', 0.17526927063842523),
 ('Creatinine', 0.15555507367732185),
 ('Lactate', 0.0961695135309471)]

#### Difference between age group

In [113]:
diff_brits_age_subgroup = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 4, 5, variables)
diff_brits_age_subgroup

[('ALP', 0.4455670614286247),
 ('TroponinI', 0.3868195082859647),
 ('Bilirubin', 0.23810434213767862),
 ('Cholesterol', 0.22772024967991167),
 ('Albumin', 0.13833296504538817)]

#### Difference between ICUType group

In [114]:
diff_brits_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 7, variables)
diff_brits_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 8, variables)
diff_brits_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 9, variables)
diff_brits_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 7, 8, variables)
diff_brits_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 7, 9, variables)
diff_brits_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 8, 9, variables)

In [115]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_brits_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_brits_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_brits_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_brits_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_brits_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_brits_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.9649941105882632), ('TroponinI', 0.9581594407990047), ('TroponinT', 0.5486699079166011), ('Platelets', 0.2518916923226591), ('AST', 0.24124108598094132)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 1.0707510714622313), ('ALP', 0.9106143557497219), ('TroponinI', 0.7970523312899177), ('TroponinT', 0.4839761749137556), ('Cholesterol', 0.25934743317704234)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.6343111242413422), ('TroponinT', 0.5409005868767687), ('Bilirubin', 0.2897210435032669), ('ALP', 0.26571689159730005), ('Cholesterol', 0.23623932465153985)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.030567265690355), ('ALP', 0.88221080881332), ('Cholesterol', 0.7056466774112209), ('BUN', 0.33074673635065266), ('Platelets', 0.31153489001028306)]
----------------------------
Difere

In [116]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[0])
print(variables[6])
print(variables[31])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
ALP
Cholesterol
TroponinI
TroponinT


#### Difference between BMI group

In [117]:
diff_brits_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 11, variables)
diff_brits_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 12, variables)
diff_brits_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 13, variables)
diff_brits_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 14, variables)
diff_brits_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 15, variables)
diff_brits_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 16, variables)

diff_brits_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 12, variables)
diff_brits_classification_low_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 13, variables)
diff_brits_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 14, variables)
diff_brits_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 15, variables)
diff_brits_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 16, variables)

diff_brits_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 13, variables)
diff_brits_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 14, variables)
diff_brits_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 15, variables)
diff_brits_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 16, variables)

diff_brits_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 14, variables)
diff_brits_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 15, variables)
diff_brits_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 16, variables)


diff_brits_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 14, 15, variables)
diff_brits_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 14, 16, variables)

diff_brits_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 15, 16, variables)

In [118]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_brits_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_brits_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_brits_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_brits_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_brits_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_brits_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.9340447366419231), ('Bilirubin', 1.0634205379618589), ('TroponinI', 0.8461866959272529), ('Cholesterol', 0.46427180720735495), ('Lactate', 0.432934002045685)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('SaO2', 0.3956365150352539), ('Cholesterol', 0.31334687951493556), ('TroponinI', 0.2811986010883921), ('Platelets', 0.21973579409347943), ('ALP', 0.14108821471054112)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.9122714170407268), ('SaO2', 0.2216147767636339), ('Bilirubin', 0.21564253602217226), ('TroponinI', 0.20102396419219326), ('ALT', 0.19595996986933695)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8860429007708258), ('TroponinI', 0.27796277400600267), ('SaO2', 0.2630706443883271), ('Albumin', 0.2194660852

In [119]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_brits_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_brits_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_brits_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_brits_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_brits_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.7929565219313819), ('Bilirubin', 0.9513305382204593), ('Cholesterol', 0.7776186867222905), ('SaO2', 0.6685476719781142), ('TroponinI', 0.5649880948388608)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 2.060865479901618), ('Cholesterol', 1.3765432242480817), ('Bilirubin', 1.2790630739840312), ('TroponinI', 1.0472106601194462), ('Glucose', 0.40732195398769067)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 2.0338878789564587), ('Bilirubin', 1.0689277903377052), ('Ph', 0.8810842114333002), ('TroponinI', 0.5682239219212503), ('Glucose', 0.5610521608500285)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 2.083344030352053), ('Bilirubin', 1.189908383818061), ('TroponinT', 0.8194450582455736), ('AST', 0.44272365749481374), ('HCO3', 0.434470181819755

In [120]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_brits_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_brits_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_brits_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_brits_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('SaO2', 0.6172512917988877), ('Cholesterol', 0.5989245375257912), ('TroponinI', 0.4822225652805854), ('Bilirubin', 0.32773253576357175), ('ALT', 0.2736328573496928)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8903667412858379), ('SaO2', 0.6587071594235809), ('Cholesterol', 0.36517235893560385), ('Albumin', 0.2900474046493634), ('ALP', 0.24093135702507662)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.220432293290763), ('Cholesterol', 0.688511448385237), ('TroponinI', 0.5649880948388608), ('SaO2', 0.4915174467035355), ('ALT', 0.3479177744530949)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('TroponinT', 0.9141402460637791), ('Cholesterol', 0.8876133876483889), ('ALT', 0.8550561867312895), ('SaO2', 0.5055719721885107), ('ALP', 0.3959647

In [121]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_brits_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_brits_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_brits_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 0.9640968964613951), ('Ph', 0.8905324606918038), ('TroponinI', 0.4789867381981959), ('ALT', 0.28228479496676956), ('Bilirubin', 0.2101352836463259)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('Cholesterol', 1.2874359859110283), ('TroponinI', 1.0472106601194462), ('TroponinT', 1.0262220986441177), ('AST', 0.21817186815478096), ('BUN', 0.19186305260550485)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.4865379251741802), ('TroponinT', 0.7199300514171338), ('TroponinI', 0.6550984461594344), ('ALT', 0.5814233293815967), ('Glucose', 0.23574108257412307)]
----------------------------


In [122]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_brits_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_brits_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.193502313404874), ('Ph', 0.8867898203626943), ('TroponinI', 0.5682239219212503), ('ALT', 0.3565697120701717), ('Cholesterol', 0.32333908944963324)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Ph', 0.8897412963473199), ('TroponinT', 0.88721026617789), ('ALT', 0.8637081243483662), ('Cholesterol', 0.5224410287127851), ('Glucose', 0.38947128943646087)]
----------------------------


In [123]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_brits_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('ALT', 0.5071384122781946), ('TroponinI', 0.39211221396001184), ('TroponinT', 0.3062920472269839), ('HCO3', 0.23358518974369696), ('Glucose', 0.20095366197425335)]
----------------------------


In [124]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[0])
print(variables[6])
print(variables[32])
print(variables[31])
print(variables[5])

Top 5 variáveis BMI
-----------------------
ALP
Cholesterol
TroponinT
TroponinI
Bilirubin


### Confidence Interval (General)(variable)

#### Distribution Bootstrap per variable

In [28]:
results_bootstrap_brits_general = []

for i in range(len(variables)):
       results_bootstrap_brits_general.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 0, i, 9000))

#### Calculating lower bound and upper bound

In [29]:
lower_bounds_brits_general, upper_bounds_brits_general = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_general)

print(lower_bounds_brits_general)
print(upper_bounds_brits_general)

[np.float64(0.32538627159459327), np.float64(0.16258679089089118), np.float64(0.15478905875444485), np.float64(0.5744084158025771), np.float64(0.3616311896320276), np.float64(0.2906151814149565), np.float64(0.6434380620957643), np.float64(0.3022046634073629), np.float64(0.20478965898756366), np.float64(0.3393442600555267), np.float64(0.20910049288260976), np.float64(0.5193110405597358), np.float64(0.4842090575488685), np.float64(0.6787845984140315), np.float64(0.24036621951397508), np.float64(0.6120427371840028), np.float64(0.41747645646717085), np.float64(0.14623547110740304), np.float64(0.001227222967261528), np.float64(0.5385050815383894), np.float64(0.1888417502786607), np.float64(0.10076061634199096), np.float64(0.27109218051427525), np.float64(0.6132163460258868), np.float64(0.46957984924039886), np.float64(0.45504042706424874), np.float64(0.640492041914929), np.float64(0.45851394435041976), np.float64(0.2856337697558636), np.float64(0.24699368119691656), np.float64(0.22657362656

#### Mean values of lower bound and upper bound


In [30]:
mean_values_ci_brits_general = toolkits.calc_mean_values_ci(lower_bounds_brits_general, upper_bounds_brits_general)

print(mean_values_ci_brits_general)

[np.float64(0.5001442903870633), np.float64(0.2679890236599881), np.float64(0.29206779881915323), np.float64(0.6503771646962193), np.float64(0.39692131748298787), np.float64(0.40690780542481764), np.float64(0.9017963009165705), np.float64(0.3481658437500393), np.float64(0.21607935126049263), np.float64(0.3621189416870011), np.float64(0.21907748433549765), np.float64(0.5674486208657743), np.float64(0.5133949330151646), np.float64(0.7130633573111789), np.float64(0.24560882497615516), np.float64(0.6544901604976849), np.float64(0.47111399718133484), np.float64(0.1659097375047333), np.float64(0.0012748854943760877), np.float64(0.5979575508691746), np.float64(0.19649628917742135), np.float64(0.10652795923613279), np.float64(0.2819266002736249), np.float64(0.6546517743936816), np.float64(0.8671831943161424), np.float64(0.4855578829184715), np.float64(0.6856451232152639), np.float64(0.47436009963806736), np.float64(0.35256130483111586), np.float64(0.2582116100318147), np.float64(0.247794193780

### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [125]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_brits_female = []

for i in indices:
       results_bootstrap_brits_female.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [126]:
means_bootstraps_brits_female,standards_deviations_brits_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_female)

print(means_bootstraps_brits_female)
print(standards_deviations_brits_female)


[np.float64(1.179863930834408), np.float64(0.5233882194493867), np.float64(0.5669394454773564), np.float64(0.4262927607316525), np.float64(0.2927745387996273), np.float64(0.30161012783732477), np.float64(0.2520432195506154), np.float64(0.6740430489599522), np.float64(0.29792444872181434), np.float64(0.2996217790430212), np.float64(0.060521396312743225), np.float64(0.5075399124539731)]
[np.float64(0.33967759869351355), np.float64(0.07230701862030067), np.float64(0.13940375834716123), np.float64(0.03847644149074913), np.float64(0.008670900439200447), np.float64(0.0848645872760513), np.float64(0.024759857939583314), np.float64(0.03441311480209853), np.float64(0.06605797747312059), np.float64(0.08435135157044552), np.float64(0.004754151191010087), np.float64(0.021740570722039883)]


#### Calculating lower bound and upper bound

In [127]:
lower_bounds_brits_female, upper_bounds_brits_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_female, means_bootstraps_brits_female,standards_deviations_brits_female)

print(lower_bounds_brits_female)
print(upper_bounds_brits_female)

[np.float64(0.5141080710400718), np.float64(0.38166906712413035), np.float64(0.29371309980739546), np.float64(0.35088032115652157), np.float64(0.2757798862252619), np.float64(0.1352785932134081), np.float64(0.20351478972670403), np.float64(0.6065945833519968), np.float64(0.1684531919829398), np.float64(0.13429616791767185), np.float64(0.05120343120130525), np.float64(0.464929176835429)]
[np.float64(1.8456197906287444), np.float64(0.6651073717746432), np.float64(0.8401657911473174), np.float64(0.5017052003067834), np.float64(0.30976919137399267), np.float64(0.46794166246124147), np.float64(0.30057164937452685), np.float64(0.7414915145679075), np.float64(0.4273957054606889), np.float64(0.4649473901683705), np.float64(0.0698393614241812), np.float64(0.5501506480725172)]


In [128]:
lower_bounds_brits_female, upper_bounds_brits_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_female)

print(lower_bounds_brits_female)
print(upper_bounds_brits_female)

[np.float64(0.4968807905077575), np.float64(0.38193937258413285), np.float64(0.32993095168375397), np.float64(0.3549349251700391), np.float64(0.27620089719597324), np.float64(0.16091099800584968), np.float64(0.20714247470515038), np.float64(0.6097020735297847), np.float64(0.18469541164541012), np.float64(0.15861808082118425), np.float64(0.05168324294505243), np.float64(0.4662975796151602)]
[np.float64(1.8393478124459337), np.float64(0.6682822710147335), np.float64(0.8757607526086687), np.float64(0.5049085262899291), np.float64(0.31004767626972984), np.float64(0.4877700875445269), np.float64(0.30484660206533915), np.float64(0.7450377725488991), np.float64(0.4418350473462839), np.float64(0.48734408360396114), np.float64(0.07025076961204074), np.float64(0.5509092356408299)]


#### Mean values of lower bound and upper bound


In [129]:
mean_values_ci_brits_female = toolkits.calc_mean_values_ci(lower_bounds_brits_female, upper_bounds_brits_female)

print(mean_values_ci_brits_female)

[np.float64(1.1681143014768456), np.float64(0.5251108217994331), np.float64(0.6028458521462113), np.float64(0.4299217257299841), np.float64(0.2931242867328515), np.float64(0.3243405427751883), np.float64(0.2559945383852448), np.float64(0.677369923039342), np.float64(0.313265229495847), np.float64(0.3229810822125727), np.float64(0.060967006278546586), np.float64(0.508603407627995)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [59]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_brits_male = []

for i in indices:
       results_bootstrap_brits_male.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

NameError: name 'testing_ae_brits_variables_standard' is not defined

#### Calculating the mean and standard deviation of the bootstrap distribution

In [131]:
means_bootstraps_brits_male,standards_deviations_brits_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_male)

print(means_bootstraps_brits_male)
print(standards_deviations_brits_male)


[np.float64(0.854312785965969), np.float64(0.69793226655648), np.float64(0.5788874609838146), np.float64(0.5218477147546687), np.float64(0.3547467656480773), np.float64(0.12182354571033481), np.float64(0.4073172652594466), np.float64(0.6193417493863396), np.float64(0.28308368574229126), np.float64(0.12197737498234233), np.float64(0.05506604203706228), np.float64(0.4785682671425299)]
[np.float64(0.17112048417178624), np.float64(0.20950734713753777), np.float64(0.2222638014715227), np.float64(0.04261663101563413), np.float64(0.011539284042436456), np.float64(0.02574568952633491), np.float64(0.03829764363415067), np.float64(0.026269546181583104), np.float64(0.04821267153960221), np.float64(0.02561385052707165), np.float64(0.0033874977733877112), np.float64(0.018186917981419373)]


#### Calculating lower bound and upper bound

In [132]:
lower_bounds_brits_male, upper_bounds_brits_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_male, means_bootstraps_brits_male,standards_deviations_brits_male)

print(lower_bounds_brits_male)
print(upper_bounds_brits_male)

[np.float64(0.5189227999722117), np.float64(0.2873054116703752), np.float64(0.14325841503266945), np.float64(0.4383206528215932), np.float64(0.3321301845175241), np.float64(0.07136292148156831), np.float64(0.33225526304376163), np.float64(0.567854384980225), np.float64(0.18858858592621164), np.float64(0.07177515044388962), np.float64(0.04842666840351274), np.float64(0.44292256290916404)]
[np.float64(1.1897027719597264), np.float64(1.1085591214425847), np.float64(1.0145165069349598), np.float64(0.6053747766877442), np.float64(0.3773633467786305), np.float64(0.1722841699391013), np.float64(0.4823792674751316), np.float64(0.6708291137924541), np.float64(0.3775787855583709), np.float64(0.17217959952079503), np.float64(0.061705415670611816), np.float64(0.5142139713758958)]


In [133]:
lower_bounds_brits_male, upper_bounds_brits_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_male)

print(lower_bounds_brits_male)
print(upper_bounds_brits_male)

[np.float64(0.5301380329394242), np.float64(0.32927026897220596), np.float64(0.3079566108428539), np.float64(0.44188543304379085), np.float64(0.3333570075193866), np.float64(0.07656795703837219), np.float64(0.3343701939070786), np.float64(0.568717674572388), np.float64(0.1973842219941603), np.float64(0.07716981633329767), np.float64(0.04862952157381154), np.float64(0.44360949397630695)]
[np.float64(1.199117427080588), np.float64(1.155415750712769), np.float64(1.0880867890273216), np.float64(0.6065284197132054), np.float64(0.37796334312292557), np.float64(0.1770223180266911), np.float64(0.48572637424963916), np.float64(0.6717754724053021), np.float64(0.38535063725911683), np.float64(0.1773071477410518), np.float64(0.06191062058012285), np.float64(0.5153353845843236)]


#### Mean values of lower bound and upper bound


In [134]:
mean_values_ci_brits_male = toolkits.calc_mean_values_ci(lower_bounds_brits_male, upper_bounds_brits_male)

print(mean_values_ci_brits_male)

[np.float64(0.864627730010006), np.float64(0.7423430098424875), np.float64(0.6980216999350878), np.float64(0.5242069263784981), np.float64(0.35566017532115607), np.float64(0.12679513753253163), np.float64(0.4100482840783589), np.float64(0.620246573488845), np.float64(0.2913674296266386), np.float64(0.12723848203717475), np.float64(0.0552700710769672), np.float64(0.47947243928031524)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [135]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_brits_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_brits_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [136]:
means_bootstraps_brits_more_than_or_equal_to_65_subgroup,standards_deviations_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_brits_more_than_or_equal_to_65_subgroup)
print(standards_deviations_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.662244906270694), np.float64(0.33551822688304433), np.float64(0.6362342133086535), np.float64(0.6383574162849505), np.float64(0.23878634109180352), np.float64(0.924878783743687), np.float64(0.5731951031171467)]
[np.float64(0.08841667646254166), np.float64(0.0631813692473505), np.float64(0.14098598742203416), np.float64(0.24085299923210227), np.float64(0.025378362701790873), np.float64(0.2879896525743772), np.float64(0.0619036891476668)]


#### Calculating lower bound and upper bound


In [137]:
lower_bounds_brits_more_than_or_equal_to_65_subgroup, upper_bounds_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_more_than_or_equal_to_65_subgroup, means_bootstraps_brits_more_than_or_equal_to_65_subgroup,standards_deviations_brits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_brits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_brits_more_than_or_equal_to_65_subgroup)


[np.float64(0.488951404771382), np.float64(0.2116850186643108), np.float64(0.3599067556366495), np.float64(0.16629421222157675), np.float64(0.1890456642096988), np.float64(0.3604294367777049), np.float64(0.4518661018775568)]
[np.float64(0.835538407770006), np.float64(0.4593514351017779), np.float64(0.9125616709806574), np.float64(1.1104206203483242), np.float64(0.28852701797390823), np.float64(1.4893281307096693), np.float64(0.6945241043567366)]


In [138]:
lower_bounds_brits_more_than_or_equal_to_65_subgroup, upper_bounds_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_brits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.49013614681391765), np.float64(0.22848200592268603), np.float64(0.39417516510179157), np.float64(0.3436040659088286), np.float64(0.1922309176367736), np.float64(0.461134543912614), np.float64(0.4554978714069311)]
[np.float64(0.8366997180001123), np.float64(0.47458613177319675), np.float64(0.9440111445954974), np.float64(1.1796209064031462), np.float64(0.29240414008122545), np.float64(1.5779181655450232), np.float64(0.6986735655061573)]


#### Mean values of lower bound and upper bound

In [139]:
mean_values_ci_brits_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_more_than_or_equal_to_65_subgroup , upper_bounds_brits_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.663417932407015), np.float64(0.35153406884794136), np.float64(0.6690931548486445), np.float64(0.7616124861559874), np.float64(0.24231752885899954), np.float64(1.0195263547288187), np.float64(0.5770857184565442)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [140]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_brits_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_brits_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [141]:
means_bootstraps_brits_less_than_65_subgroup,standards_deviations_brits_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_less_than_65_subgroup)

print(means_bootstraps_brits_less_than_65_subgroup)
print(standards_deviations_brits_less_than_65_subgroup)


[np.float64(0.7722524194006836), np.float64(0.36263183850017955), np.float64(1.084029737789501), np.float64(0.571527362778218), np.float64(0.4751239089993839), np.float64(0.5373403409647254), np.float64(0.7127001763464236)]
[np.float64(0.6086238944869353), np.float64(0.09109588956527907), np.float64(0.43935299070541206), np.float64(0.13437165851905025), np.float64(0.08710455881773665), np.float64(0.07056198341407932), np.float64(0.06180206326074808)]


#### Calculating lower bound and upper bound

In [142]:
lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_less_than_65_subgroup, means_bootstraps_brits_less_than_65_subgroup, standards_deviations_brits_less_than_65_subgroup)

print(lower_bounds_brits_less_than_65_subgroup)
print(upper_bounds_brits_less_than_65_subgroup)

[np.float64(-0.42062849392421564), np.float64(0.18408717581259446), np.float64(0.22291369950693218), np.float64(0.3081637515379648), np.float64(0.30440211082736923), np.float64(0.3990413947954173), np.float64(0.5915703581850913)]
[np.float64(1.9651333327255829), np.float64(0.5411765011877646), np.float64(1.9451457760720698), np.float64(0.8348909740184712), np.float64(0.6458457071713986), np.float64(0.6756392871340335), np.float64(0.8338299945077559)]


In [143]:
lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_less_than_65_subgroup)

print(lower_bounds_brits_less_than_65_subgroup)
print(upper_bounds_brits_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.2074044801250295), np.float64(0.41237684781003814), np.float64(0.3415031837447295), np.float64(0.3212877496534465), np.float64(0.38080614880421926), np.float64(0.5948515926626379)]
[np.float64(1.6400321328986665), np.float64(0.5615428890114939), np.float64(2.08700193546815), np.float64(0.8688863074412774), np.float64(0.6621744792199287), np.float64(0.6432682344289811), np.float64(0.836412282791994)]


#### Mean values of lower bound and upper bound

In [144]:
mean_values_ci_brits_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup)

print(mean_values_ci_brits_less_than_65_subgroup)

[np.float64(0.8200160664493332), np.float64(0.3844736845682617), np.float64(1.249689391639094), np.float64(0.6051947455930035), np.float64(0.4917311144366876), np.float64(0.5120371916166002), np.float64(0.7156319377273159)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [145]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [146]:
means_bootstraps_brits_ICUType_1_subgroup, standards_deviations_brits_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_1_subgroup)

print(means_bootstraps_brits_ICUType_1_subgroup)
print(standards_deviations_brits_ICUType_1_subgroup)


[np.float64(0.5242391082650917), np.float64(1.3415657504066385), np.float64(0.9641261893849576), np.float64(0.22523224662477), np.float64(0.7214285617066842), np.float64(0.45725893867605666)]
[np.float64(0.179049986745374), np.float64(0.3732886535480633), np.float64(0.26648121535108293), np.float64(0.03475251277970779), np.float64(0.15381048461581723), np.float64(0.04360406748169409)]


#### Calculating lower bound and upper bound

In [147]:
lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_1_subgroup, means_bootstraps_brits_ICUType_1_subgroup, standards_deviations_brits_ICUType_1_subgroup)

print(lower_bounds_brits_ICUType_1_subgroup)
print(upper_bounds_brits_ICUType_1_subgroup)

[np.float64(0.1733075828117846), np.float64(0.6099334336149846), np.float64(0.4418326047403729), np.float64(0.15711857320427478), np.float64(0.4199655514150304), np.float64(0.3717965368324821)]
[np.float64(0.8751706337183988), np.float64(2.0731980671982924), np.float64(1.4864197740295424), np.float64(0.2933459200452653), np.float64(1.022891571998338), np.float64(0.5427213405196312)]


In [148]:
lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_1_subgroup)

print(lower_bounds_brits_ICUType_1_subgroup)
print(upper_bounds_brits_ICUType_1_subgroup)

[np.float64(0.2818701088590386), np.float64(0.6832919189918449), np.float64(0.5044988888435504), np.float64(0.15881062421212594), np.float64(0.4440415795895485), np.float64(0.375999018085907)]
[np.float64(0.9347071562797221), np.float64(2.1424587501889225), np.float64(1.545974285796128), np.float64(0.29692894998043906), np.float64(1.0435013017386077), np.float64(0.5459618084824918)]


#### Mean values of lower bound and upper bound

In [149]:
mean_values_ci_brits_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup)

print(mean_values_ci_brits_ICUType_1_subgroup)

[np.float64(0.6082886325693804), np.float64(1.4128753345903837), np.float64(1.0252365873198392), np.float64(0.2278697870962825), np.float64(0.7437714406640781), np.float64(0.46098041328419936)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [37]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [38]:
means_bootstraps_brits_ICUType_2_subgroup, standards_deviations_brits_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_2_subgroup)

print(means_bootstraps_brits_ICUType_2_subgroup)
print(standards_deviations_brits_ICUType_2_subgroup)


[np.float64(0.5621925335565238), np.float64(0.37599610018440305), np.float64(0.0), np.float64(0.25361202408619077), np.float64(0.17263922025885275), np.float64(0.43960499932788366)]
[np.float64(0.24695541531596954), np.float64(0.12144212659362798), np.float64(0.0), np.float64(0.027974171942437683), np.float64(0.04832351472767887), np.float64(0.017892639144452414)]


#### Calculating lower bound and upper bound

In [39]:
lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_2_subgroup, means_bootstraps_brits_ICUType_2_subgroup, standards_deviations_brits_ICUType_2_subgroup)

print(lower_bounds_brits_ICUType_2_subgroup)
print(upper_bounds_brits_ICUType_2_subgroup)

[np.float64(0.07816881375009221), np.float64(0.1379739058549383), np.float64(0.0), np.float64(0.198783654581682), np.float64(0.07792687178621129), np.float64(0.40453607101638533)]
[np.float64(1.0462162533629553), np.float64(0.6140182945138678), np.float64(0.0), np.float64(0.3084403935906995), np.float64(0.26735156873149424), np.float64(0.474673927639382)]


In [42]:
lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_2_subgroup)
print(lower_bounds_brits_ICUType_2_subgroup)
print(upper_bounds_brits_ICUType_2_subgroup)

[np.float64(0.28535170875429494), np.float64(0.1355656241631972), np.float64(0.0), np.float64(0.20046852841020707), np.float64(0.08863894347413558), np.float64(0.4050478860711588)]
[np.float64(1.116012281294197), np.float64(0.6352708608548769), np.float64(0.0), np.float64(0.3099178354532329), np.float64(0.2762919387240098), np.float64(0.47489211583914764)]


#### Mean values of lower bound and upper bound

In [43]:
mean_values_ci_brits_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup)

print(mean_values_ci_brits_ICUType_2_subgroup)

[np.float64(0.700681995024246), np.float64(0.38541824250903706), np.float64(0.0), np.float64(0.25519318193172), np.float64(0.1824654410990727), np.float64(0.43997000095515326)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [155]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [156]:
means_bootstraps_brits_ICUType_3_subgroup, standards_deviations_brits_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_3_subgroup)

print(means_bootstraps_brits_ICUType_3_subgroup)
print(standards_deviations_brits_ICUType_3_subgroup)


[np.float64(1.5950335104696207), np.float64(0.5406018573257461), np.float64(0.6878097755242151), np.float64(1.1319651942674953), np.float64(0.23651905141316573), np.float64(0.5806266335503317)]
[np.float64(0.45243214716175434), np.float64(0.0630307978175607), np.float64(0.18994101591217263), np.float64(0.4467237279126342), np.float64(0.03447171296734733), np.float64(0.036862477331274575)]


#### Calculating lower bound and upper bound

In [157]:
lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_3_subgroup, means_bootstraps_brits_ICUType_3_subgroup, standards_deviations_brits_ICUType_3_subgroup)

print(lower_bounds_brits_ICUType_3_subgroup)
print(upper_bounds_brits_ICUType_3_subgroup)

[np.float64(0.7082827965844566), np.float64(0.4170637636865012), np.float64(0.3155322251494074), np.float64(0.2564027765192618), np.float64(0.16895573551176263), np.float64(0.5083775056001094)]
[np.float64(2.4817842243547847), np.float64(0.6641399509649909), np.float64(1.0600873258990227), np.float64(2.007527612015729), np.float64(0.30408236731456884), np.float64(0.652875761500554)]


In [158]:
lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_3_subgroup)

print(lower_bounds_brits_ICUType_3_subgroup)
print(upper_bounds_brits_ICUType_3_subgroup)

[np.float64(0.7939550857221166), np.float64(0.4205872182263832), np.float64(0.3370788046599422), np.float64(0.42849663050814585), np.float64(0.17832924327612418), np.float64(0.5103321820529819)]
[np.float64(2.5493768779559924), np.float64(0.6674941648076488), np.float64(1.1317698409105241), np.float64(2.132643843593351), np.float64(0.3119720884871641), np.float64(0.6555130248767465)]


#### Mean values of lower bound and upper bound

In [159]:
mean_values_ci_brits_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup)

print(mean_values_ci_brits_ICUType_3_subgroup)

[np.float64(1.6716659818390545), np.float64(0.544040691517016), np.float64(0.7344243227852332), np.float64(1.2805702370507484), np.float64(0.24515066588164414), np.float64(0.5829226034648642)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [160]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_brits_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [161]:
means_bootstraps_brits_ICUType_4_subgroup, standards_deviations_brits_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_4_subgroup)

print(means_bootstraps_brits_ICUType_4_subgroup)
print(standards_deviations_brits_ICUType_4_subgroup)


[np.float64(0.44498405390307794), np.float64(0.6983041667063125), np.float64(0.7306252220827371), np.float64(0.49060926376446706), np.float64(0.18038459066093948), np.float64(0.43616529618609085)]
[np.float64(0.0941342290916942), np.float64(0.26590252692965594), np.float64(0.26936222261266096), np.float64(0.0802554943001893), np.float64(0.021864311504363326), np.float64(0.0202726158372185)]


#### Calculating lower bound and upper bound

In [162]:
lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_4_subgroup, means_bootstraps_brits_ICUType_4_subgroup, standards_deviations_brits_ICUType_4_subgroup)

print(lower_bounds_brits_ICUType_4_subgroup)
print(upper_bounds_brits_ICUType_4_subgroup)

[np.float64(0.2604843551709147), np.float64(0.17714479052599497), np.float64(0.20268496696626104), np.float64(0.33331138537463645), np.float64(0.13753132756562259), np.float64(0.39643169927272626)]
[np.float64(0.6294837526352411), np.float64(1.21946354288663), np.float64(1.258565477199213), np.float64(0.6479071421542977), np.float64(0.22323785375625638), np.float64(0.47589889309945543)]


In [163]:
lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_4_subgroup)

print(lower_bounds_brits_ICUType_4_subgroup)
print(upper_bounds_brits_ICUType_4_subgroup)

[np.float64(0.28602392839788426), np.float64(0.031016012049908958), np.float64(0.23850040491847574), np.float64(0.346799139468531), np.float64(0.1381858946213613), np.float64(0.3970780677494158)]
[np.float64(0.6550222762143815), np.float64(1.1697815367073499), np.float64(1.2839793652411828), np.float64(0.660672778318496), np.float64(0.22305811075201631), np.float64(0.47687461780075635)]


#### Mean values of lower bound and upper bound

In [164]:
mean_values_ci_brits_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup)

print(mean_values_ci_brits_ICUType_4_subgroup)

[np.float64(0.47052310230613287), np.float64(0.6003987743786294), np.float64(0.7612398850798293), np.float64(0.5037359588935135), np.float64(0.18062200268668882), np.float64(0.43697634277508607)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [165]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [166]:
means_bootstraps_brits_undefined_classification_subgroup, standards_deviations_brits_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_undefined_classification_subgroup)

print(means_bootstraps_brits_undefined_classification_subgroup)
print(standards_deviations_brits_undefined_classification_subgroup)


[np.float64(0.27580452340722195), np.float64(0.4069967174467866), np.float64(0.5761801901712693), np.float64(0.8422198209476998), np.float64(0.4122577787901838), np.float64(0.2332685567510703), np.float64(0.06740695618510778)]
[np.float64(0.04486338677452591), np.float64(0.05954413271613699), np.float64(0.18081126434508163), np.float64(0.2855688569549286), np.float64(0.12515486871795073), np.float64(0.08494212686592569), np.float64(0.005037956728670317)]


#### Calculating lower bound and upper bound

In [167]:
lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_undefined_classification_subgroup, means_bootstraps_brits_undefined_classification_subgroup, standards_deviations_brits_undefined_classification_subgroup)

print(lower_bounds_brits_undefined_classification_subgroup)
print(upper_bounds_brits_undefined_classification_subgroup)

[np.float64(0.18787390110466057), np.float64(0.29029236183248497), np.float64(0.22179662405575812), np.float64(0.28251514620976914), np.float64(0.16695874361316174), np.float64(0.06678504732362384), np.float64(0.05753274244124273)]
[np.float64(0.3637351457097833), np.float64(0.5237010730610883), np.float64(0.9305637562867806), np.float64(1.4019244956856305), np.float64(0.6575568139672059), np.float64(0.3997520661785168), np.float64(0.07728116992897283)]


In [168]:
lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_undefined_classification_subgroup)

print(lower_bounds_brits_undefined_classification_subgroup)
print(upper_bounds_brits_undefined_classification_subgroup)

[np.float64(0.20263481715092455), np.float64(0.297765632174484), np.float64(0.2464090364371217), np.float64(0.40487025525627784), np.float64(0.215910276833765), np.float64(0.11516010285916287), np.float64(0.058037491556782485)]
[np.float64(0.3759709288423774), np.float64(0.5321918576413711), np.float64(0.9576805451793318), np.float64(1.4766778926129156), np.float64(0.6931543221897899), np.float64(0.42856972926983783), np.float64(0.07776482011735085)]


#### Mean values of lower bound and upper bound

In [169]:
mean_values_ci_brits_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup)

print(mean_values_ci_brits_undefined_classification_subgroup)

[np.float64(0.289302872996651), np.float64(0.4149787449079275), np.float64(0.6020447908082267), np.float64(0.9407740739345967), np.float64(0.4545322995117774), np.float64(0.27186491606450036), np.float64(0.06790115583706667)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [170]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [171]:
means_bootstraps_brits_low_weight_classification_subgroup, standards_deviations_brits_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_low_weight_classification_subgroup)

print(means_bootstraps_brits_low_weight_classification_subgroup)
print(standards_deviations_brits_low_weight_classification_subgroup)


[np.float64(0.4117071767409993), np.float64(2.02462479593246), np.float64(0.0697610836540352), np.float64(0.0), np.float64(1.452464767049052), np.float64(0.22511391992235394), np.float64(0.07876575200090079)]
[np.float64(0.313562029381117), np.float64(1.6710560842546742), np.float64(0.05297868194116039), np.float64(0.0), np.float64(1.2840757488636576), np.float64(0.11005089543492753), np.float64(0.013787861556698637)]


#### Calculating lower bound and upper bound

In [172]:
lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_low_weight_classification_subgroup, means_bootstraps_brits_low_weight_classification_subgroup, standards_deviations_brits_low_weight_classification_subgroup)

print(lower_bounds_brits_low_weight_classification_subgroup)
print(upper_bounds_brits_low_weight_classification_subgroup)

[np.float64(-0.20286310776528027), np.float64(-1.2505849453532316), np.float64(-0.03407522489904173), np.float64(0.0), np.float64(-1.0642774541450162), np.float64(0.009418128403512527), np.float64(0.05174203992594709)]
[np.float64(1.026277461247279), np.float64(5.2998345372181515), np.float64(0.17359739220711212), np.float64(0.0), np.float64(3.96920698824312), np.float64(0.44080971144119535), np.float64(0.10578946407585449)]


In [173]:
lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_low_weight_classification_subgroup)

print(lower_bounds_brits_low_weight_classification_subgroup)
print(upper_bounds_brits_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.027231435799876812), np.float64(0.06702356114639711), np.float64(0.053893569760482124)]
[np.float64(0.6505204688677284), np.float64(4.413896485801919), np.float64(0.10999470092617175), np.float64(0.0), np.float64(4.389479579851235), np.float64(0.4730132375396637), np.float64(0.10773608295033878)]


#### Mean values of lower bound and upper bound

In [174]:
mean_values_ci_brits_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup)

print(mean_values_ci_brits_low_weight_classification_subgroup)

[np.float64(0.3252602344338642), np.float64(2.2069482429009595), np.float64(0.054997350463085876), np.float64(0.0), np.float64(2.208355507825556), np.float64(0.2700183993430304), np.float64(0.08081482635541046)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [175]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [176]:
means_bootstraps_brits_normal_weight_classification_subgroup, standards_deviations_brits_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_normal_weight_classification_subgroup)

print(means_bootstraps_brits_normal_weight_classification_subgroup)
print(standards_deviations_brits_normal_weight_classification_subgroup)


[np.float64(0.2507157349963609), np.float64(0.5457061795759722), np.float64(0.7665562197715444), np.float64(0.5652688982187426), np.float64(0.5233307205655814), np.float64(0.15534101010166437), np.float64(0.051354752349358136)]
[np.float64(0.07419681715923193), np.float64(0.12456109593044702), np.float64(0.3715302459269295), np.float64(0.08997435719450365), np.float64(0.20499054790313423), np.float64(0.040408627749961123), np.float64(0.004347503178908474)]


#### Calculating lower bound and upper bound

In [177]:
lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_normal_weight_classification_subgroup, means_bootstraps_brits_normal_weight_classification_subgroup, standards_deviations_brits_normal_weight_classification_subgroup)

print(lower_bounds_brits_normal_weight_classification_subgroup)
print(upper_bounds_brits_normal_weight_classification_subgroup)

[np.float64(0.10529264559676282), np.float64(0.30157091767745736), np.float64(0.038370318587453434), np.float64(0.38892239858537314), np.float64(0.12155662950430562), np.float64(0.07614155504705478), np.float64(0.04283380269602413)]
[np.float64(0.396138824395959), np.float64(0.7898414414744871), np.float64(1.4947421209556353), np.float64(0.7416153978521121), np.float64(0.9251048116268572), np.float64(0.23454046515627397), np.float64(0.05987570200269214)]


In [178]:
lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_normal_weight_classification_subgroup)

print(lower_bounds_brits_normal_weight_classification_subgroup)
print(upper_bounds_brits_normal_weight_classification_subgroup)

[np.float64(0.12752475937455598), np.float64(0.33907978472608374), np.float64(0.0), np.float64(0.38765003219112587), np.float64(0.19942872679027562), np.float64(0.0877463857807708), np.float64(0.04312146702574288)]
[np.float64(0.41512220820559886), np.float64(0.8247740882457966), np.float64(1.1889972456754012), np.float64(0.7450995778748506), np.float64(0.9872682339171264), np.float64(0.2440400125384495), np.float64(0.06017121826957199)]


#### Mean values of lower bound and upper bound

In [179]:
mean_values_ci_brits_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup)

print(mean_values_ci_brits_normal_weight_classification_subgroup)

[np.float64(0.2713234837900774), np.float64(0.5819269364859402), np.float64(0.5944986228377006), np.float64(0.5663748050329882), np.float64(0.593348480353701), np.float64(0.16589319915961015), np.float64(0.05164634264765743)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [180]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [181]:
means_bootstraps_brits_overweight_classification_subgroup, standards_deviations_brits_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_overweight_classification_subgroup)

print(means_bootstraps_brits_overweight_classification_subgroup)
print(standards_deviations_brits_overweight_classification_subgroup)


[np.float64(0.4443343674155459), np.float64(0.28076309393714294), np.float64(1.4893652815898761), np.float64(1.0439855928490005), np.float64(0.19602664934732794), np.float64(0.4284199627653709), np.float64(0.03366806475240419)]
[np.float64(0.18880560340842148), np.float64(0.056995221450811734), np.float64(0.2749030032241481), np.float64(0.5598008593806592), np.float64(0.06397525812267431), np.float64(0.1589312177792946), np.float64(0.002828427833127552)]


#### Calculating lower bound and upper bound

In [182]:
lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_overweight_classification_subgroup, means_bootstraps_brits_overweight_classification_subgroup, standards_deviations_brits_overweight_classification_subgroup)

print(lower_bounds_brits_overweight_classification_subgroup)
print(upper_bounds_brits_overweight_classification_subgroup)

[np.float64(0.07428218465568692), np.float64(0.1690545126026672), np.float64(0.9505652960286475), np.float64(-0.053203930051662685), np.float64(0.07063744752523274), np.float64(0.11692049989886161), np.float64(0.028124448066603525)]
[np.float64(0.8143865501754048), np.float64(0.39247167527161864), np.float64(2.028165267151105), np.float64(2.1411751157496637), np.float64(0.3214158511694232), np.float64(0.7399194256318802), np.float64(0.03921168143820486)]


In [183]:
lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_overweight_classification_subgroup)

print(lower_bounds_brits_overweight_classification_subgroup)
print(upper_bounds_brits_overweight_classification_subgroup)

[np.float64(0.14624609297999341), np.float64(0.18415528921903443), np.float64(0.909720282165972), np.float64(0.38052832269971276), np.float64(0.10551525836734299), np.float64(0.16485663295550892), np.float64(0.028397843809544517)]
[np.float64(0.8693158474038792), np.float64(0.4048962706035148), np.float64(1.9831085736449836), np.float64(2.2776141033419854), np.float64(0.34420218670371083), np.float64(0.7771147900884521), np.float64(0.03955727457150905)]


#### Mean values of lower bound and upper bound

In [184]:
mean_values_ci_brits_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup)

print(mean_values_ci_brits_overweight_classification_subgroup)

[np.float64(0.5077809701919364), np.float64(0.2945257799112746), np.float64(1.4464144279054778), np.float64(1.329071213020849), np.float64(0.2248587225355269), np.float64(0.4709857115219805), np.float64(0.033977559190526786)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [185]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [186]:
means_bootstraps_brits_obs1_classification_subgroup, standards_deviations_brits_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs1_classification_subgroup)

print(means_bootstraps_brits_obs1_classification_subgroup)
print(standards_deviations_brits_obs1_classification_subgroup)


[np.float64(0.27758254972645824), np.float64(0.30673163478710974), np.float64(0.49782627534217366), np.float64(0.49254042530897457), np.float64(0.40482788915482854), np.float64(0.14680209434138097), np.float64(0.04955810846956136)]
[np.float64(0.08663082547785), np.float64(0.05841847062881554), np.float64(0.25391575091978175), np.float64(0.19734694495289493), np.float64(0.12175443025243234), np.float64(0.044400604014272906), np.float64(0.006134371972722228)]


#### Calculating lower bound and upper bound

In [187]:
lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs1_classification_subgroup, means_bootstraps_brits_obs1_classification_subgroup, standards_deviations_brits_obs1_classification_subgroup)

print(lower_bounds_brits_obs1_classification_subgroup)
print(upper_bounds_brits_obs1_classification_subgroup)

[np.float64(0.10778925183889732), np.float64(0.19223353632272033), np.float64(0.00016054843195834057), np.float64(0.10574752074229193), np.float64(0.16619359090186717), np.float64(0.059778509581581526), np.float64(0.03753496033525387)]
[np.float64(0.44737584761401916), np.float64(0.42122973325149915), np.float64(0.995492002252389), np.float64(0.8793333298756572), np.float64(0.6434621874077899), np.float64(0.2338256791011804), np.float64(0.061581256603868845)]


In [188]:
lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs1_classification_subgroup)

print(lower_bounds_brits_obs1_classification_subgroup)
print(upper_bounds_brits_obs1_classification_subgroup)

[np.float64(0.12289231359182605), np.float64(0.2000579254761009), np.float64(0.0), np.float64(0.0), np.float64(0.21008376164389972), np.float64(0.0736643816686853), np.float64(0.03833129435496211)]
[np.float64(0.46007914986975795), np.float64(0.42917899390694847), np.float64(1.0159483154566267), np.float64(0.6210031331183582), np.float64(0.6817938338223474), np.float64(0.24464222148606513), np.float64(0.06211861668493779)]


#### Mean values of lower bound and upper bound

In [189]:
mean_values_ci_brits_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup)

print(mean_values_ci_brits_obs1_classification_subgroup)

[np.float64(0.291485731730792), np.float64(0.3146184596915247), np.float64(0.5079741577283133), np.float64(0.3105015665591791), np.float64(0.44593879773312356), np.float64(0.15915330157737523), np.float64(0.050224955519949946)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [190]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [191]:
means_bootstraps_brits_obs2_classification_subgroup, standards_deviations_brits_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs2_classification_subgroup)

print(means_bootstraps_brits_obs2_classification_subgroup)
print(standards_deviations_brits_obs2_classification_subgroup)


[np.float64(1.3987342012682182), np.float64(0.25770282954810214), np.float64(0.12437234472641032), np.float64(0.0), np.float64(0.28613978848179056), np.float64(0.5067906683265901), np.float64(0.06420550627348588)]
[np.float64(1.0273410411361406), np.float64(0.04901938288945451), np.float64(0.09640692347022636), np.float64(0.0), np.float64(0.13581125260590501), np.float64(0.2761364677567751), np.float64(0.012361877548406335)]


#### Calculating lower bound and upper bound

In [192]:
lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs2_classification_subgroup, means_bootstraps_brits_obs2_classification_subgroup, standards_deviations_brits_obs2_classification_subgroup)

print(lower_bounds_brits_obs2_classification_subgroup)
print(upper_bounds_brits_obs2_classification_subgroup)

[np.float64(-0.6148172391984992), np.float64(0.16162660454039235), np.float64(-0.0645817531355426), np.float64(0.0), np.float64(0.019954624678945154), np.float64(-0.03442686329479494), np.float64(0.03997667149731517)]
[np.float64(3.4122856417349356), np.float64(0.35377905455581193), np.float64(0.31332644258836323), np.float64(0.0), np.float64(0.552324952284636), np.float64(1.0480081999479753), np.float64(0.0884343410496566)]


In [193]:
lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs2_classification_subgroup)

print(lower_bounds_brits_obs2_classification_subgroup)
print(upper_bounds_brits_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.16085406685781214), np.float64(0.0), np.float64(0.0), np.float64(0.11027911334373292), np.float64(0.08223802713905004), np.float64(0.04146215450961552)]
[np.float64(3.581291190927886), np.float64(0.35189023952639004), np.float64(0.1991019392632846), np.float64(0.0), np.float64(0.5926599976593028), np.float64(1.0950697955323436), np.float64(0.08963354350479674)]


#### Mean values of lower bound and upper bound

In [194]:
mean_values_ci_brits_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup)

print(mean_values_ci_brits_obs2_classification_subgroup)

[np.float64(1.790645595463943), np.float64(0.2563721531921011), np.float64(0.0995509696316423), np.float64(0.0), np.float64(0.3514695555015178), np.float64(0.5886539113356968), np.float64(0.06554784900720613)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [195]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [196]:
means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs3_classification_subgroup)

print(means_bootstraps_brits_obs3_classification_subgroup)
print(standards_deviations_brits_obs3_classification_subgroup)


[np.float64(1.1558137039269567), np.float64(0.15105622662933305), np.float64(0.0), np.float64(0.24833773550806432), np.float64(0.2076193164712968), np.float64(0.9800082280153003), np.float64(0.13877564149678087)]
[np.float64(0.8099368792496682), np.float64(0.050537477693121294), np.float64(0.0), np.float64(0.18895668393214338), np.float64(0.13024546336467263), np.float64(0.8852108446590914), np.float64(0.023450873782400468)]


#### Calculating lower bound and upper bound

In [197]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs3_classification_subgroup, means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(-0.4316334091531595), np.float64(0.05200459048131893), np.float64(0.0), np.float64(-0.12201055963705504), np.float64(-0.04765710087319261), np.float64(-0.7549731462407993), np.float64(0.09281277347728137)]
[np.float64(2.743260817007073), np.float64(0.25010786277734715), np.float64(0.0), np.float64(0.6186860306531836), np.float64(0.4628957338157862), np.float64(2.7149896022714), np.float64(0.18473850951628037)]


In [198]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(0.15495382950717612), np.float64(0.06587815295679428), np.float64(0.0), np.float64(0.0), np.float64(0.030002538375966944), np.float64(0.0), np.float64(0.09526878194415268)]
[np.float64(2.9770854048865076), np.float64(0.2609092593596054), np.float64(0.0), np.float64(0.39211221396027324), np.float64(0.49802743026682), np.float64(2.877838304401334), np.float64(0.18759590004491447)]


#### Mean values of lower bound and upper bound

In [199]:
mean_values_ci_brits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup)

print(mean_values_ci_brits_obs3_classification_subgroup)

[np.float64(1.566019617196842), np.float64(0.16339370615819984), np.float64(0.0), np.float64(0.19605610698013662), np.float64(0.2640149843213935), np.float64(1.438919152200667), np.float64(0.14143234099453358)]


### Confidence Interval (Obesity subgroup)

In [61]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_brits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



In [62]:
means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs3_classification_subgroup)

print(means_bootstraps_brits_obs3_classification_subgroup)
print(standards_deviations_brits_obs3_classification_subgroup)


[np.float64(0.2806926933863719), np.float64(0.3820663150379395), np.float64(0.9057647775017064), np.float64(0.5853079679948625), np.float64(0.8725816158315272), np.float64(0.38925668747975356), np.float64(0.04761286414597215)]
[np.float64(0.06373721155843905), np.float64(0.18012747998930878), np.float64(0.4688572649094767), np.float64(0.096513834263852), np.float64(0.3077718964288261), np.float64(0.24702680671125146), np.float64(0.003977853360674446)]


In [63]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(0.168612870048241), np.float64(0.15977144108497995), np.float64(0.0), np.float64(0.3978499660236096), np.float64(0.3429081123820663), np.float64(0.10774778339258363), np.float64(0.04010443955278869)]
[np.float64(0.4172395918238282), np.float64(0.7999125086772191), np.float64(1.6622886860714472), np.float64(0.7761771096355127), np.float64(1.5331644393667998), np.float64(0.9768906000456562), np.float64(0.055769877446810424)]


In [64]:
mean_values_ci_brits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup)

print(mean_values_ci_brits_obs3_classification_subgroup)

[np.float64(0.2929262309360346), np.float64(0.47984197488109953), np.float64(0.8311443430357236), np.float64(0.5870135378295611), np.float64(0.9380362758744331), np.float64(0.5423191917191199), np.float64(0.047937158499799556)]


## USGAN

### Calculate MAE and AE

In [73]:
testing_mae_usgan_variables_standard, testing_ae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for USGAN model

#### Difference between gender group

In [201]:
diff_usgan_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 1, 2, variables)
diff_usgan_gender_subgroup

[('Cholesterol', 0.1139685240845858),
 ('K', 0.07962183065280748),
 ('Bilirubin', 0.07273528639114712),
 ('SaO2', 0.06724927135996472),
 ('AST', 0.06357152153738319)]

#### Difference between age group

In [202]:
diff_usgan_age_subgroup = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 4, 5, variables)
diff_usgan_age_subgroup

[('Cholesterol', 1.4080334556310095),
 ('TroponinI', 1.2745449266559237),
 ('ALP', 0.4648972864315326),
 ('SaO2', 0.14584230690666233),
 ('Bilirubin', 0.1443865131739146)]

#### Difference between ICUType group

In [203]:
diff_usgan_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 7, variables)
diff_usgan_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 8, variables)
diff_usgan_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 9, variables)
diff_usgan_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 7, 8, variables)
diff_usgan_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 7, 9, variables)
diff_usgan_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 8, 9, variables)

In [204]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_usgan_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_usgan_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_usgan_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_usgan_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_usgan_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_usgan_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 1.968532838973082), ('TroponinT', 0.31397445203755153), ('TroponinI', 0.24761901013296939), ('Glucose', 0.17361770179039776), ('Temp', 0.1581287282962357)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('TroponinI', 1.0540542904431034), ('ALP', 0.9588981039437612), ('SaO2', 0.9557847709091482), ('Cholesterol', 0.7208086032231116), ('TroponinT', 0.3983027555297005)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.8469546614770607), ('Cholesterol', 0.4515708998874557), ('TroponinT', 0.4392851376770836), ('ALP', 0.2595152189283853), ('Glucose', 0.1775868348873213)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('Cholesterol', 2.6893414421961936), ('TroponinI', 1.3016733005760728), ('ALP', 0.978153547526703), ('SaO2', 0.872219162894308), ('Temp', 0.2501181542790452)]
----------------------------
Diferença(M

In [205]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[28])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
Cholesterol
TroponinI
ALP
SaO2
TroponinT


#### Difference between BMI group

In [206]:
diff_usgan_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 11, variables)
diff_usgan_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 12, variables)
diff_usgan_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 13, variables)
diff_usgan_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 14, variables)
diff_usgan_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 15, variables)
diff_usgan_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 16, variables)

diff_usgan_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 12, variables)
diff_usgan_classification_low_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 13, variables)
diff_usgan_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 14, variables)
diff_usgan_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 15, variables)
diff_usgan_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 16, variables)

diff_usgan_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 13, variables)
diff_usgan_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 14, variables)
diff_usgan_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 15, variables)
diff_usgan_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 16, variables)

diff_usgan_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 14, variables)
diff_usgan_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 15, variables)
diff_usgan_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 16, variables)


diff_usgan_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 14, 15, variables)
diff_usgan_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 14, 16, variables)

diff_usgan_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 15, 16, variables)

In [207]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_usgan_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_usgan_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_usgan_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_usgan_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_usgan_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_usgan_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.9859619551977459), ('TroponinI', 1.366326168723437), ('Bilirubin', 1.040021736197295), ('Cholesterol', 0.9605295647070888), ('SaO2', 0.38269896484952315)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 1.5579458954552443), ('TroponinI', 0.8172831271866614), ('SaO2', 0.5748494144707222), ('Platelets', 0.11877231069789002), ('ALP', 0.1039191923535443)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.7813610360341876), ('TroponinI', 0.5289144739027736), ('SaO2', 0.26335363873788353), ('TroponinT', 0.15046357996993215), ('Bilirubin', 0.13730851481079887)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Cholesterol', 1.1133597984183674), ('Ph', 0.8795575943383244), ('TroponinI', 0.5672434018132706), ('Albumin', 0.2786

In [208]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_usgan_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_usgan_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_usgan_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_usgan_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_usgan_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('TroponinI', 2.1836092959100983), ('ALP', 1.8820427628442014), ('Bilirubin', 1.0046746012576198), ('SaO2', 0.9575483793202454), ('Cholesterol', 0.5974163307481555)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 2.057108071257811), ('TroponinI', 1.8952406426262105), ('Cholesterol', 1.7418906007412764), ('Bilirubin', 1.1773302510080939), ('Glucose', 0.4141106395038745)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.9622075144434934), ('TroponinI', 1.9335695705367075), ('Bilirubin', 1.2290397171661307), ('Ph', 0.8756933605302372), ('Glucose', 0.44454091315430133)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 2.209691635941559), ('Bilirubin', 1.153094644797057), ('TroponinT', 0.5311563117054711), ('HCO3', 0.38165705305271735), ('BUN', 0.342954347380909

In [209]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_usgan_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_usgan_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_usgan_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_usgan_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('Cholesterol', 2.339306931489432), ('SaO2', 0.8382030532086058), ('TroponinI', 0.28836865328388783), ('Lactate', 0.17658435533908162), ('ALP', 0.17506530841360962)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8895493103803579), ('SaO2', 0.8483769987750389), ('Cholesterol', 0.4445860970368769), ('Albumin', 0.3456592349280355), ('TroponinI', 0.2500397253733908)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinI', 2.1836092959100983), ('Cholesterol', 0.9187488642412167), ('SaO2', 0.7611965750861914), ('TroponinT', 0.6853385877772105), ('ALP', 0.32764887309735746)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('Cholesterol', 3.416495052966329), ('TroponinT', 0.7593013402882078), ('SaO2', 0.691906211296852), ('ALT', 0.6246716987082246), ('Albumin', 0

In [210]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_usgan_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_usgan_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_usgan_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 1.894720834452555), ('Ph', 0.8907999497218877), ('TroponinT', 0.3086857131955071), ('Albumin', 0.1924964141650365), ('ALP', 0.09490055681431758)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinI', 1.8952406426262105), ('Cholesterol', 1.4205580672482152), ('TroponinT', 0.5233142228450464), ('BUN', 0.2234609775735314), ('ALT', 0.16362559122010772)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.077188121476897), ('TroponinT', 0.5972769753560437), ('ALT', 0.5418724610143977), ('K', 0.15370070433593253), ('SaO2', 0.14629684191175374)]
----------------------------


In [211]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_usgan_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_usgan_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinI', 1.9335695705367075), ('Ph', 0.894254151573005), ('TroponinT', 0.8319999360405534), ('Cholesterol', 0.4741627672043398), ('ALT', 0.25324163267164324)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Cholesterol', 2.971908955929452), ('TroponinT', 0.9059626885515508), ('Ph', 0.8849431239052544), ('ALT', 0.6314885024659332), ('SaO2', 0.15647078747818682)]
----------------------------


In [212]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_usgan_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('Cholesterol', 2.4977461887251122), ('TroponinI', 2.02984317003595), ('ALT', 0.37824686979429), ('ALP', 0.18814524657663967), ('BUN', 0.15424784345271514)]
----------------------------


In [213]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[32])
print(variables[5])

Top 5 variáveis BMI
-----------------------
Cholesterol
TroponinI
ALP
TroponinT
Bilirubin


### Confidence Interval (General)(variable)

#### Distribution Bootstrap per variable

In [24]:
results_bootstrap_usgan_general = []

for i in range(len(variables)):
       results_bootstrap_usgan_general.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 0, i, 9000))

#### Calculating lower bound and upper bound

In [25]:
lower_bounds_usgan_general, upper_bounds_usgan_general = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_general)

print(lower_bounds_usgan_general)
print(upper_bounds_usgan_general)

[np.float64(0.37987774064344376), np.float64(0.2061196710476389), np.float64(0.2642059808571521), np.float64(0.6262144676087764), np.float64(0.34996752928595315), np.float64(0.4608825163427001), np.float64(1.5798287271142717), np.float64(0.26426029102168), np.float64(0.2501416215302208), np.float64(0.3672310025744434), np.float64(0.2336397496988845), np.float64(0.547317726704121), np.float64(0.40490283216049716), np.float64(0.4541917576079991), np.float64(0.2398693327438087), np.float64(0.5672127995716005), np.float64(0.3386268945761316), np.float64(0.19621905761422262), np.float64(0.09461877278888023), np.float64(0.5034148469916283), np.float64(0.2009237838801295), np.float64(0.13768680289388924), np.float64(0.2784384772778879), np.float64(0.41021893002126186), np.float64(0.44691557521753594), np.float64(0.4362492117335727), np.float64(0.35191724828948184), np.float64(0.4631821529483603), np.float64(0.3038516671866549), np.float64(0.2764093154673026), np.float64(0.23389959242605715), 

#### Mean values of lower bound and upper bound


In [26]:
mean_values_ci_usgan_general = toolkits.calc_mean_values_ci(lower_bounds_usgan_general, upper_bounds_usgan_general)

print(mean_values_ci_usgan_general)

[np.float64(0.5523861815611371), np.float64(0.27656506774147327), np.float64(0.36055370643786144), np.float64(0.7223585592129538), np.float64(0.37632116591016346), np.float64(0.5579415842955476), np.float64(2.18612841636589), np.float64(0.29391245942874644), np.float64(0.2582181907514178), np.float64(0.3868772750471532), np.float64(0.24308195045283068), np.float64(0.5881713069736254), np.float64(0.4302979298475895), np.float64(0.4796243491438271), np.float64(0.2447949691639964), np.float64(0.6054427754291538), np.float64(0.36878196717459144), np.float64(0.20630665234832965), np.float64(0.0993638258381053), np.float64(0.5588644884287883), np.float64(0.2089924274540188), np.float64(0.14455185243629404), np.float64(0.2892461978808375), np.float64(0.4365913314600694), np.float64(0.6090794076714355), np.float64(0.46219269164196647), np.float64(0.3757870744169378), np.float64(0.4788525273977311), np.float64(0.3649603632333839), np.float64(0.28479634990124036), np.float64(0.24754436603288543)

### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [214]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_usgan_female = []

for i in indices:
       results_bootstrap_usgan_female.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [215]:
means_bootstraps_usgan_female,standards_deviations_usgan_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_female)

print(means_bootstraps_usgan_female)
print(standards_deviations_usgan_female)


[np.float64(1.9426236689245937), np.float64(1.476607683872043), np.float64(0.5824835235607073), np.float64(0.36850169787683984), np.float64(0.31434980584103134), np.float64(0.2969288919294192), np.float64(0.2539369589667588), np.float64(0.6189688903613779), np.float64(0.5199231364970216), np.float64(0.29602329913626385), np.float64(0.08456221539013241), np.float64(0.48191092834966043)]
[np.float64(0.9541881868285297), np.float64(0.1877480782810291), np.float64(0.1341446209881592), np.float64(0.022208191609570507), np.float64(0.007781894873228429), np.float64(0.04921979920019426), np.float64(0.017271259436821576), np.float64(0.032413015838310834), np.float64(0.0564424455784571), np.float64(0.0494075393210884), np.float64(0.0036063142786134585), np.float64(0.019843005847499713)]


#### Calculating lower bound and upper bound

In [216]:
lower_bounds_usgan_female, upper_bounds_usgan_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_female, means_bootstraps_usgan_female,standards_deviations_usgan_female)

print(lower_bounds_usgan_female)
print(upper_bounds_usgan_female)

[np.float64(0.07244918826709923), np.float64(1.1086282122746192), np.float64(0.31956489770413943), np.float64(0.324974442160317), np.float64(0.29909757215802674), np.float64(0.20045985817074508), np.float64(0.22008591250294096), np.float64(0.5554405466879624), np.float64(0.4092979759638837), np.float64(0.19918630150218403), np.float64(0.07749396928711748), np.float64(0.4430193515435433)]
[np.float64(3.812798149582088), np.float64(1.8445871554694668), np.float64(0.8454021494172752), np.float64(0.41202895359336267), np.float64(0.32960203952403594), np.float64(0.3933979256880933), np.float64(0.2877880054305766), np.float64(0.6824972340347935), np.float64(0.6305482970301596), np.float64(0.3928602967703437), np.float64(0.09163046149314734), np.float64(0.5208025051557775)]


In [217]:
lower_bounds_usgan_female, upper_bounds_usgan_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_female)

print(lower_bounds_usgan_female)
print(upper_bounds_usgan_female)

[np.float64(0.552297478025096), np.float64(1.1017362219702032), np.float64(0.35641161373652314), np.float64(0.3260790413594915), np.float64(0.2997033002187566), np.float64(0.21475375438518265), np.float64(0.22333706135388795), np.float64(0.5583830419668504), np.float64(0.42322000620175504), np.float64(0.212761117538852), np.float64(0.07781804540130907), np.float64(0.4442725023927491)]
[np.float64(4.152962140136351), np.float64(1.849726768264328), np.float64(0.8713501895528681), np.float64(0.4134724300394901), np.float64(0.33015364458349306), np.float64(0.4053563042704095), np.float64(0.29103420259605683), np.float64(0.6859710762707061), np.float64(0.6449312130075252), np.float64(0.4042530083266986), np.float64(0.09180762671607288), np.float64(0.5217146519652324)]


#### Mean values of lower bound and upper bound


In [218]:
mean_values_ci_usgan_female = toolkits.calc_mean_values_ci(lower_bounds_usgan_female, upper_bounds_usgan_female)

print(mean_values_ci_usgan_female)

[np.float64(2.3526298090807236), np.float64(1.4757314951172655), np.float64(0.6138809016446957), np.float64(0.3697757356994908), np.float64(0.3149284724011248), np.float64(0.31005502932779605), np.float64(0.2571856319749724), np.float64(0.6221770591187783), np.float64(0.5340756096046402), np.float64(0.3085070629327753), np.float64(0.08481283605869097), np.float64(0.4829935771789907)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [219]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_usgan_male = []

for i in indices:
       results_bootstrap_usgan_male.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [220]:
means_bootstraps_usgan_male,standards_deviations_usgan_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_male)

print(means_bootstraps_usgan_male)
print(standards_deviations_usgan_male)


[np.float64(2.0726677071880224), np.float64(1.521719357985055), np.float64(0.647884100775224), np.float64(0.41339022264865743), np.float64(0.36294211357141837), np.float64(0.23326783402162193), np.float64(0.2991813494222848), np.float64(0.5396922523604093), np.float64(0.4471123537553462), np.float64(0.23310134620622072), np.float64(0.08799528626109657), np.float64(0.4453718710703944)]
[np.float64(0.44660915538820406), np.float64(0.4204146672622888), np.float64(0.2854069746839292), np.float64(0.029647951135701615), np.float64(0.009465638157692849), np.float64(0.019807005431765896), np.float64(0.019766307137517734), np.float64(0.022066882314831754), np.float64(0.03816450825263489), np.float64(0.020037261900200602), np.float64(0.0032705418736737957), np.float64(0.01665269341383374)]


#### Calculating lower bound and upper bound

In [221]:
lower_bounds_usgan_male, upper_bounds_usgan_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_male, means_bootstraps_usgan_male,standards_deviations_usgan_male)

print(lower_bounds_usgan_male)
print(upper_bounds_usgan_male)

[np.float64(1.19732984746129), np.float64(0.6977217515785784), np.float64(0.08849670945818766), np.float64(0.3552813062072789), np.float64(0.34438980369165234), np.float64(0.19444681673377157), np.float64(0.260440099325393), np.float64(0.4964419577722552), np.float64(0.37231129209250013), np.float64(0.19382903453303094), np.float64(0.08158514197876578), np.float64(0.4127331917336929)]
[np.float64(2.948005566914755), np.float64(2.3457169643915314), np.float64(1.2072714920922603), np.float64(0.471499139090036), np.float64(0.3814944234511844), np.float64(0.2720888513094723), np.float64(0.3379225995191766), np.float64(0.5829425469485634), np.float64(0.5219134154181923), np.float64(0.2723736578794105), np.float64(0.09440543054342736), np.float64(0.4780105504070959)]


In [222]:
lower_bounds_usgan_male, upper_bounds_usgan_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_male)

print(lower_bounds_usgan_male)
print(upper_bounds_usgan_male)

[np.float64(1.2292811368616663), np.float64(0.6555007597293402), np.float64(0.32313291882691214), np.float64(0.35914749095839094), np.float64(0.34511660082043605), np.float64(0.1958850809727209), np.float64(0.26313137436218476), np.float64(0.4979339758458074), np.float64(0.37591374096484853), np.float64(0.1956756826026091), np.float64(0.08190129602300607), np.float64(0.4132579091871235)]
[np.float64(2.9854508929724686), np.float64(2.3071585381188053), np.float64(1.2807265847175557), np.float64(0.4746176670174084), np.float64(0.3820471089537273), np.float64(0.27375953694426686), np.float64(0.3399750978704415), np.float64(0.5842094101381341), np.float64(0.5267720993598021), np.float64(0.27468027861458016), np.float64(0.0947655793872466), np.float64(0.4785389832519423)]


#### Mean values of lower bound and upper bound


In [223]:
mean_values_ci_usgan_male = toolkits.calc_mean_values_ci(lower_bounds_usgan_male, upper_bounds_usgan_male)

print(mean_values_ci_usgan_male)

[np.float64(2.1073660149170674), np.float64(1.4813296489240728), np.float64(0.8019297517722339), np.float64(0.41688257898789965), np.float64(0.36358185488708167), np.float64(0.23482230895849387), np.float64(0.30155323611631313), np.float64(0.5410716929919708), np.float64(0.4513429201623253), np.float64(0.23517798060859463), np.float64(0.08833343770512633), np.float64(0.4458984462195329)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [224]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_usgan_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_usgan_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [225]:
means_bootstraps_usgan_more_than_or_equal_to_65_subgroup,standards_deviations_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_usgan_more_than_or_equal_to_65_subgroup)
print(standards_deviations_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(2.3277195513431117), np.float64(0.44589850849065876), np.float64(0.6949215827305563), np.float64(0.7147283454211643), np.float64(0.45957213952273107), np.float64(2.4792889398943343), np.float64(0.6058696934449835)]
[np.float64(0.34526111955149585), np.float64(0.053276012188330245), np.float64(0.14408238878591878), np.float64(0.31290303869328545), np.float64(0.030293228981973206), np.float64(0.4337610458951387), np.float64(0.06289506669009498)]


#### Calculating lower bound and upper bound


In [226]:
lower_bounds_usgan_more_than_or_equal_to_65_subgroup, upper_bounds_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup, means_bootstraps_usgan_more_than_or_equal_to_65_subgroup,standards_deviations_usgan_more_than_or_equal_to_65_subgroup)

print(lower_bounds_usgan_more_than_or_equal_to_65_subgroup)
print(upper_bounds_usgan_more_than_or_equal_to_65_subgroup)


[np.float64(1.6510201917602019), np.float64(0.3414794433616145), np.float64(0.41252528990365767), np.float64(0.10144965892918179), np.float64(0.4001985017426386), np.float64(1.629132912043437), np.float64(0.48259762792715255)]
[np.float64(3.0044189109260215), np.float64(0.550317573619703), np.float64(0.9773178755574549), np.float64(1.3280070319131467), np.float64(0.5189457773028235), np.float64(3.3294449677452316), np.float64(0.7291417589628145)]


In [227]:
lower_bounds_usgan_more_than_or_equal_to_65_subgroup, upper_bounds_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup)

print(lower_bounds_usgan_more_than_or_equal_to_65_subgroup)
print(upper_bounds_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(1.6281411700965303), np.float64(0.3538409751089212), np.float64(0.43798509911056305), np.float64(0.3426913152057593), np.float64(0.4013893680394128), np.float64(1.6962287715313156), np.float64(0.4890799925089994)]
[np.float64(2.986609104407262), np.float64(0.5618845069425482), np.float64(1.0009809872077418), np.float64(1.4232026323484746), np.float64(0.5191876623736257), np.float64(3.3925309502286316), np.float64(0.7333418465627987)]


#### Mean values of lower bound and upper bound

In [228]:
mean_values_ci_usgan_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_more_than_or_equal_to_65_subgroup , upper_bounds_usgan_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(2.307375137251896), np.float64(0.4578627410257347), np.float64(0.7194830431591523), np.float64(0.882946973777117), np.float64(0.46028851520651926), np.float64(2.5443798608799737), np.float64(0.611210919535899)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [229]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_usgan_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_usgan_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [230]:
means_bootstraps_usgan_less_than_65_subgroup,standards_deviations_usgan_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_less_than_65_subgroup)

print(means_bootstraps_usgan_less_than_65_subgroup)
print(standards_deviations_usgan_less_than_65_subgroup)


[np.float64(3.1915916167949896), np.float64(0.49700833187780014), np.float64(1.1540519599147494), np.float64(0.5684573133058964), np.float64(0.6047754764421036), np.float64(1.2037718112165114), np.float64(0.6563198514072964)]
[np.float64(1.8186843986544805), np.float64(0.06888260767323874), np.float64(0.4397032372115261), np.float64(0.13007067512146603), np.float64(0.07972056605731984), np.float64(0.11822595277499039), np.float64(0.05582518999743342)]


#### Calculating lower bound and upper bound

In [231]:
lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_less_than_65_subgroup, means_bootstraps_usgan_less_than_65_subgroup, standards_deviations_usgan_less_than_65_subgroup)

print(lower_bounds_usgan_less_than_65_subgroup)
print(upper_bounds_usgan_less_than_65_subgroup)

[np.float64(-0.3729643038126782), np.float64(0.3620009016770498), np.float64(0.2922494510944862), np.float64(0.313523474623013), np.float64(0.44852603814261044), np.float64(0.9720532017395971), np.float64(0.5469044895822213)]
[np.float64(6.756147537402658), np.float64(0.6320157620785505), np.float64(2.0158544687350126), np.float64(0.8233911519887799), np.float64(0.7610249147415968), np.float64(1.4354904206934258), np.float64(0.7657352132323716)]


In [232]:
lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_less_than_65_subgroup)

print(lower_bounds_usgan_less_than_65_subgroup)
print(upper_bounds_usgan_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.3702739951653813), np.float64(0.4778360893803177), np.float64(0.35044900408143564), np.float64(0.46341740316866975), np.float64(0.9540208063055534), np.float64(0.548691295124491)]
[np.float64(5.588982304367992), np.float64(0.6415627036521072), np.float64(2.1304576536745676), np.float64(0.8534265972932001), np.float64(0.7798798036610687), np.float64(1.4144873643084972), np.float64(0.7678905150942262)]


#### Mean values of lower bound and upper bound

In [233]:
mean_values_ci_usgan_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup)

print(mean_values_ci_usgan_less_than_65_subgroup)

[np.float64(2.794491152183996), np.float64(0.5059183494087443), np.float64(1.3041468715274427), np.float64(0.601937800687318), np.float64(0.6216486034148692), np.float64(1.1842540853070254), np.float64(0.6582909051093586)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [234]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [235]:
means_bootstraps_usgan_ICUType_1_subgroup, standards_deviations_usgan_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_1_subgroup)

print(means_bootstraps_usgan_ICUType_1_subgroup)
print(standards_deviations_usgan_ICUType_1_subgroup)


[np.float64(0.563887600037251), np.float64(2.3529393563125782), np.float64(1.9728257383543046), np.float64(0.252140969171943), np.float64(0.747602868180934), np.float64(0.46306218526438647)]
[np.float64(0.17889963377447138), np.float64(0.510335735595341), np.float64(0.3422505640529371), np.float64(0.04336115844262078), np.float64(0.11535988455863048), np.float64(0.04316677190793421)]


#### Calculating lower bound and upper bound

In [236]:
lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_1_subgroup, means_bootstraps_usgan_ICUType_1_subgroup, standards_deviations_usgan_ICUType_1_subgroup)

print(lower_bounds_usgan_ICUType_1_subgroup)
print(upper_bounds_usgan_ICUType_1_subgroup)

[np.float64(0.21325076099188167), np.float64(1.3526996945219543), np.float64(1.302026959122029), np.float64(0.16715466029647139), np.float64(0.5215016491853199), np.float64(0.3784568669959801)]
[np.float64(0.9145244390826204), np.float64(3.3531790181032024), np.float64(2.6436245175865802), np.float64(0.3371272780474146), np.float64(0.9737040871765481), np.float64(0.5476675035327929)]


In [237]:
lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_1_subgroup)

print(lower_bounds_usgan_ICUType_1_subgroup)
print(upper_bounds_usgan_ICUType_1_subgroup)

[np.float64(0.32064584728118506), np.float64(1.47123806257609), np.float64(1.2904636582424243), np.float64(0.17406586455570075), np.float64(0.5317508076580856), np.float64(0.3823837590088833)]
[np.float64(0.9910105632281638), np.float64(3.452901506618758), np.float64(2.6480144431956956), np.float64(0.3438911621630157), np.float64(0.9846942599779819), np.float64(0.5518614447096367)]


#### Mean values of lower bound and upper bound

In [238]:
mean_values_ci_usgan_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup)

print(mean_values_ci_usgan_ICUType_1_subgroup)

[np.float64(0.6558282052546744), np.float64(2.462069784597424), np.float64(1.96923905071906), np.float64(0.25897851335935823), np.float64(0.7582225338180337), np.float64(0.46712260185926)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [239]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [240]:
means_bootstraps_usgan_ICUType_2_subgroup, standards_deviations_usgan_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_2_subgroup)

print(means_bootstraps_usgan_ICUType_2_subgroup)
print(standards_deviations_usgan_ICUType_2_subgroup)


[np.float64(0.6494848142269435), np.float64(2.588516407140481), np.float64(0.0), np.float64(0.23292001088560635), np.float64(0.43183600103461445), np.float64(0.42268415077612986)]
[np.float64(0.32627852399781204), np.float64(0.28458306540297795), np.float64(0.0), np.float64(0.045026531085604844), np.float64(0.09272501096614483), np.float64(0.01776057524019988)]


#### Calculating lower bound and upper bound

In [241]:
lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_2_subgroup, means_bootstraps_usgan_ICUType_2_subgroup, standards_deviations_usgan_ICUType_2_subgroup)

print(lower_bounds_usgan_ICUType_2_subgroup)
print(upper_bounds_usgan_ICUType_2_subgroup)

[np.float64(0.009990658262344132), np.float64(2.030743848340638), np.float64(0.0), np.float64(0.14466963160904767), np.float64(0.250098319074889), np.float64(0.38787406296062427)]
[np.float64(1.2889789701915428), np.float64(3.1462889659403244), np.float64(0.0), np.float64(0.32117039016216503), np.float64(0.6135736829943399), np.float64(0.45749423859163546)]


In [242]:
lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_2_subgroup)

print(lower_bounds_usgan_ICUType_2_subgroup)
print(upper_bounds_usgan_ICUType_2_subgroup)

[np.float64(0.293777431241365), np.float64(2.1773231830929753), np.float64(0.0), np.float64(0.1480191351787115), np.float64(0.2514350706347436), np.float64(0.3892437119907599)]
[np.float64(1.39187261008907), np.float64(2.9719600641874147), np.float64(0.0), np.float64(0.32788912779348267), np.float64(0.6167728348695134), np.float64(0.4585444457841812)]


#### Mean values of lower bound and upper bound

In [243]:
mean_values_ci_usgan_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup)

print(mean_values_ci_usgan_ICUType_2_subgroup)

[np.float64(0.8428250206652175), np.float64(2.574641623640195), np.float64(0.0), np.float64(0.23795413148609706), np.float64(0.4341039527521285), np.float64(0.42389407888747055)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [244]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [245]:
means_bootstraps_usgan_ICUType_3_subgroup, standards_deviations_usgan_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_3_subgroup)

print(means_bootstraps_usgan_ICUType_3_subgroup)
print(standards_deviations_usgan_ICUType_3_subgroup)


[np.float64(1.5122824802300234), np.float64(1.303657289228127), np.float64(2.639665589919476), np.float64(1.216254237639582), np.float64(0.3489812828206453), np.float64(0.5363634606906125)]
[np.float64(0.41975672186442503), np.float64(0.189360158634557), np.float64(0.8769678727429988), np.float64(0.4512369988168381), np.float64(0.0360681959837002), np.float64(0.03214385306870897)]


#### Calculating lower bound and upper bound

In [246]:
lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_3_subgroup, means_bootstraps_usgan_ICUType_3_subgroup, standards_deviations_usgan_ICUType_3_subgroup)

print(lower_bounds_usgan_ICUType_3_subgroup)
print(upper_bounds_usgan_ICUType_3_subgroup)

[np.float64(0.6895744231071537), np.float64(0.9325181981976041), np.float64(0.9208401437444929), np.float64(0.3318459714666363), np.float64(0.27828891770526065), np.float64(0.47336266635159563)]
[np.float64(2.334990537352893), np.float64(1.67479638025865), np.float64(4.358491036094459), np.float64(2.1006625038125275), np.float64(0.4196736479360299), np.float64(0.5993642550296293)]


In [247]:
lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_3_subgroup)

print(lower_bounds_usgan_ICUType_3_subgroup)
print(upper_bounds_usgan_ICUType_3_subgroup)

[np.float64(0.7704951751701834), np.float64(0.947548109130724), np.float64(0.34585257856854856), np.float64(0.5033567799778038), np.float64(0.2822534210065467), np.float64(0.47515410038056904)]
[np.float64(2.4097671332704476), np.float64(1.6922044310345425), np.float64(4.10455563415239), np.float64(2.2344206574851024), np.float64(0.42398422507604705), np.float64(0.6012125377583866)]


#### Mean values of lower bound and upper bound

In [248]:
mean_values_ci_usgan_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup)

print(mean_values_ci_usgan_ICUType_3_subgroup)

[np.float64(1.5901311542203156), np.float64(1.3198762700826332), np.float64(2.225204106360469), np.float64(1.3688887187314531), np.float64(0.3531188230412969), np.float64(0.5381833190694778)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [249]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_usgan_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [250]:
means_bootstraps_usgan_ICUType_4_subgroup, standards_deviations_usgan_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_4_subgroup)

print(means_bootstraps_usgan_ICUType_4_subgroup)
print(standards_deviations_usgan_ICUType_4_subgroup)


[np.float64(0.4792969545506289), np.float64(1.492520155050831), np.float64(2.4166136328207393), np.float64(0.5112285898221713), np.float64(0.3075488793627516), np.float64(0.4407496110203167)]
[np.float64(0.07264251678818208), np.float64(0.6271640410559025), np.float64(0.4315835238720545), np.float64(0.083912422121692), np.float64(0.04071595730903325), np.float64(0.021230632122603153)]


#### Calculating lower bound and upper bound

In [251]:
lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_4_subgroup, means_bootstraps_usgan_ICUType_4_subgroup, standards_deviations_usgan_ICUType_4_subgroup)

print(lower_bounds_usgan_ICUType_4_subgroup)
print(upper_bounds_usgan_ICUType_4_subgroup)

[np.float64(0.33692023789944575), np.float64(0.2633012221826623), np.float64(1.5707254697106299), np.float64(0.3467632646081329), np.float64(0.22774706944097606), np.float64(0.39913833669099535)]
[np.float64(0.621673671201812), np.float64(2.7217390879189995), np.float64(3.2625017959308487), np.float64(0.6756939150362098), np.float64(0.38735068928452715), np.float64(0.482360885349638)]


In [252]:
lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_4_subgroup)

print(lower_bounds_usgan_ICUType_4_subgroup)
print(upper_bounds_usgan_ICUType_4_subgroup)

[np.float64(0.35499588348790595), np.float64(0.2032850750806556), np.float64(1.6267678244593), np.float64(0.35750635756957544), np.float64(0.22766040627861825), np.float64(0.39983466694780556)]
[np.float64(0.6405359279930083), np.float64(3.0892252334427197), np.float64(3.2952409306100465), np.float64(0.6848105418525466), np.float64(0.3877896147030844), np.float64(0.48317143529693185)]


#### Mean values of lower bound and upper bound

In [253]:
mean_values_ci_usgan_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup)

print(mean_values_ci_usgan_ICUType_4_subgroup)

[np.float64(0.4977659057404571), np.float64(1.6462551542616877), np.float64(2.4610043775346733), np.float64(0.5211584497110611), np.float64(0.3077250104908513), np.float64(0.4415030511223687)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [254]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [255]:
means_bootstraps_usgan_undefined_classification_subgroup, standards_deviations_usgan_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_undefined_classification_subgroup)

print(means_bootstraps_usgan_undefined_classification_subgroup)
print(standards_deviations_usgan_undefined_classification_subgroup)


[np.float64(0.37927547550229274), np.float64(0.43112199091233366), np.float64(1.8593231564276025), np.float64(1.3621282758824853), np.float64(0.588938310460686), np.float64(0.1948814347428873), np.float64(0.09467248531019279)]
[np.float64(0.03993382630084488), np.float64(0.05935639678613229), np.float64(0.2569802403478074), np.float64(0.18858621368035416), np.float64(0.11301996643858488), np.float64(0.03660713963979667), np.float64(0.0037987791388823878)]


#### Calculating lower bound and upper bound

In [256]:
lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_undefined_classification_subgroup, means_bootstraps_usgan_undefined_classification_subgroup, standards_deviations_usgan_undefined_classification_subgroup)

print(lower_bounds_usgan_undefined_classification_subgroup)
print(upper_bounds_usgan_undefined_classification_subgroup)

[np.float64(0.3010066141877584), np.float64(0.31478559095944536), np.float64(1.3556511406074532), np.float64(0.9925060890882162), np.float64(0.367423246707134), np.float64(0.12313275947185726), np.float64(0.08722701501276123)]
[np.float64(0.45754433681682705), np.float64(0.547458390865222), np.float64(2.362995172247752), np.float64(1.7317504626767544), np.float64(0.810453374214238), np.float64(0.26663011001391734), np.float64(0.10211795560762435)]


In [257]:
lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_undefined_classification_subgroup)

print(lower_bounds_usgan_undefined_classification_subgroup)
print(upper_bounds_usgan_undefined_classification_subgroup)

[np.float64(0.30869098078502927), np.float64(0.32181971036208), np.float64(1.358646658135121), np.float64(0.9990198651778386), np.float64(0.4130162051800631), np.float64(0.13977708052703836), np.float64(0.08743907167123824)]
[np.float64(0.46603658734654074), np.float64(0.5549029132420611), np.float64(2.3599698972892544), np.float64(1.7117143053755217), np.float64(0.8438078806326391), np.float64(0.2774775590649913), np.float64(0.10236536428882531)]


#### Mean values of lower bound and upper bound

In [258]:
mean_values_ci_usgan_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup)

print(mean_values_ci_usgan_undefined_classification_subgroup)

[np.float64(0.387363784065785), np.float64(0.43836131180207055), np.float64(1.8593082777121879), np.float64(1.3553670852766802), np.float64(0.6284120429063511), np.float64(0.20862731979601482), np.float64(0.09490221798003178)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [259]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [260]:
means_bootstraps_usgan_low_weight_classification_subgroup, standards_deviations_usgan_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_low_weight_classification_subgroup)

print(means_bootstraps_usgan_low_weight_classification_subgroup)
print(standards_deviations_usgan_low_weight_classification_subgroup)


[np.float64(0.32919883610811934), np.float64(2.081131605946338), np.float64(1.804523613189184), np.float64(0.0), np.float64(1.607679945405586), np.float64(0.2666678727844263), np.float64(0.10243322988477836)]
[np.float64(0.25162252877837404), np.float64(1.3193634963896108), np.float64(1.3530663882920586), np.float64(0.0), np.float64(1.0737633215875115), np.float64(0.10651441079471385), np.float64(0.011198030685643613)]


#### Calculating lower bound and upper bound

In [261]:
lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_low_weight_classification_subgroup, means_bootstraps_usgan_low_weight_classification_subgroup, standards_deviations_usgan_low_weight_classification_subgroup)

print(lower_bounds_usgan_low_weight_classification_subgroup)
print(upper_bounds_usgan_low_weight_classification_subgroup)

[np.float64(-0.1639722579963871), np.float64(-0.5047733294941406), np.float64(-0.8474377765549392), np.float64(0.0), np.float64(-0.49685749282603675), np.float64(0.057903463792282756), np.float64(0.08048549304314251)]
[np.float64(0.8223699302126257), np.float64(4.667036541386817), np.float64(4.456485002933308), np.float64(0.0), np.float64(3.7122173836372085), np.float64(0.4754322817765698), np.float64(0.12438096672641422)]


In [262]:
lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_low_weight_classification_subgroup)

print(lower_bounds_usgan_low_weight_classification_subgroup)
print(upper_bounds_usgan_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.2831756123511703), np.float64(0.07568562079113288), np.float64(0.08176865642044362)]
[np.float64(0.521526056147568), np.float64(3.8852329995591535), np.float64(2.819078722220053), np.float64(0.0), np.float64(4.084151125946311), np.float64(0.489791801148735), np.float64(0.12564452131567572)]


#### Mean values of lower bound and upper bound

In [263]:
mean_values_ci_usgan_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup)

print(mean_values_ci_usgan_low_weight_classification_subgroup)

[np.float64(0.260763028073784), np.float64(1.9426164997795767), np.float64(1.4095393611100264), np.float64(0.0), np.float64(2.1836633691487406), np.float64(0.2827387109699339), np.float64(0.10370658886805967)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [264]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [265]:
means_bootstraps_usgan_normal_weight_classification_subgroup, standards_deviations_usgan_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_normal_weight_classification_subgroup)

print(means_bootstraps_usgan_normal_weight_classification_subgroup)
print(standards_deviations_usgan_normal_weight_classification_subgroup)


[np.float64(0.36716622423068535), np.float64(0.5331987130566835), np.float64(2.9647027948952647), np.float64(2.1839034547495797), np.float64(0.6236126242642359), np.float64(0.2237520333905473), np.float64(0.069408565511564)]
[np.float64(0.0380843920768835), np.float64(0.13466519171113311), np.float64(1.1773137898993913), np.float64(0.2392449700140041), np.float64(0.19038236671669487), np.float64(0.04239324169445181), np.float64(0.003553218348660062)]


#### Calculating lower bound and upper bound

In [266]:
lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_normal_weight_classification_subgroup, means_bootstraps_usgan_normal_weight_classification_subgroup, standards_deviations_usgan_normal_weight_classification_subgroup)

print(lower_bounds_usgan_normal_weight_classification_subgroup)
print(upper_bounds_usgan_normal_weight_classification_subgroup)

[np.float64(0.29252218738689106), np.float64(0.2692597873316808), np.float64(0.6572101681901015), np.float64(1.7149919300397665), np.float64(0.2504700422080169), np.float64(0.14066280648152), np.float64(0.062444385518983396)]
[np.float64(0.4418102610744796), np.float64(0.7971376387816862), np.float64(5.272195421600427), np.float64(2.652814979459393), np.float64(0.996755206320455), np.float64(0.3068412602995746), np.float64(0.0763727455041446)]


In [267]:
lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_normal_weight_classification_subgroup)

print(lower_bounds_usgan_normal_weight_classification_subgroup)
print(upper_bounds_usgan_normal_weight_classification_subgroup)

[np.float64(0.29379326154339624), np.float64(0.30066489047439204), np.float64(0.0), np.float64(1.7418619589257787), np.float64(0.33596990276394384), np.float64(0.14640515047483887), np.float64(0.06259001695077994)]
[np.float64(0.44303539902603495), np.float64(0.8255746590884991), np.float64(3.7219816477052636), np.float64(2.679536293887521), np.float64(1.0559311279379122), np.float64(0.31170609148198064), np.float64(0.07666240979539621)]


#### Mean values of lower bound and upper bound

In [268]:
mean_values_ci_usgan_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup)

print(mean_values_ci_usgan_normal_weight_classification_subgroup)

[np.float64(0.3684143302847156), np.float64(0.5631197747814456), np.float64(1.8609908238526318), np.float64(2.21069912640665), np.float64(0.695950515350928), np.float64(0.22905562097840976), np.float64(0.06962621337308808)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [269]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [270]:
means_bootstraps_usgan_overweight_classification_subgroup, standards_deviations_usgan_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_overweight_classification_subgroup)

print(means_bootstraps_usgan_overweight_classification_subgroup)
print(standards_deviations_usgan_overweight_classification_subgroup)


[np.float64(0.5313377674146835), np.float64(0.35969716090379755), np.float64(1.077706156210986), np.float64(1.8801484874938146), np.float64(0.4511712299115422), np.float64(0.3045232905897354), np.float64(0.06277935761546015)]
[np.float64(0.14852429760642916), np.float64(0.07898953496277909), np.float64(0.500833087018858), np.float64(0.8193849050247347), np.float64(0.05757374812303061), np.float64(0.07290910367719514), np.float64(0.0034726220119524784)]


#### Calculating lower bound and upper bound

In [271]:
lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_overweight_classification_subgroup, means_bootstraps_usgan_overweight_classification_subgroup, standards_deviations_usgan_overweight_classification_subgroup)

print(lower_bounds_usgan_overweight_classification_subgroup)
print(upper_bounds_usgan_overweight_classification_subgroup)

[np.float64(0.2402354932769738), np.float64(0.20488051722118314), np.float64(0.09609134338800951), np.float64(0.2741835841695617), np.float64(0.33832875713542165), np.float64(0.16162407323733607), np.float64(0.05597314354011227)]
[np.float64(0.8224400415523933), np.float64(0.5145138045864119), np.float64(2.0593209690339624), np.float64(3.4861133908180673), np.float64(0.5640137026876627), np.float64(0.4474225079421347), np.float64(0.06958557169080803)]


In [272]:
lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_overweight_classification_subgroup)

print(lower_bounds_usgan_overweight_classification_subgroup)
print(upper_bounds_usgan_overweight_classification_subgroup)

[np.float64(0.2972291056014258), np.float64(0.22280497000865498), np.float64(0.07192878140181272), np.float64(0.4092776996333325), np.float64(0.34308219439832677), np.float64(0.17869988490283376), np.float64(0.05637324028494324)]
[np.float64(0.8693070664429965), np.float64(0.5308598495935907), np.float64(2.104946220765399), np.float64(3.625243160669168), np.float64(0.5693773749776534), np.float64(0.4592751836990056), np.float64(0.07002338308825001)]


#### Mean values of lower bound and upper bound

In [273]:
mean_values_ci_usgan_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup)

print(mean_values_ci_usgan_overweight_classification_subgroup)

[np.float64(0.5832680860222111), np.float64(0.3768324098011228), np.float64(1.0884375010836058), np.float64(2.01726043015125), np.float64(0.4562297846879901), np.float64(0.3189875343009197), np.float64(0.06319831168659662)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [274]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [275]:
means_bootstraps_usgan_obs1_classification_subgroup, standards_deviations_usgan_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs1_classification_subgroup)

print(means_bootstraps_usgan_obs1_classification_subgroup)
print(standards_deviations_usgan_obs1_classification_subgroup)


[np.float64(0.22072518602261637), np.float64(0.4547208642874865), np.float64(2.813640052555757), np.float64(1.6780089082143648), np.float64(0.4013555347642542), np.float64(0.21585892519453578), np.float64(0.0933896400138659)]
[np.float64(0.06261225716930365), np.float64(0.07793442849794503), np.float64(0.8053105933410025), np.float64(0.8907219767639729), np.float64(0.08849928362062656), np.float64(0.05077296418644187), np.float64(0.00746406027845502)]


#### Calculating lower bound and upper bound

In [276]:
lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs1_classification_subgroup, means_bootstraps_usgan_obs1_classification_subgroup, standards_deviations_usgan_obs1_classification_subgroup)

print(lower_bounds_usgan_obs1_classification_subgroup)
print(upper_bounds_usgan_obs1_classification_subgroup)

[np.float64(0.09800741698002144), np.float64(0.3019721912758022), np.float64(1.2352602932388106), np.float64(-0.06777408648134498), np.float64(0.2279001262102306), np.float64(0.1163457440007677), np.float64(0.07876035068965806)]
[np.float64(0.3434429550652113), np.float64(0.6074695372991707), np.float64(4.392019811872704), np.float64(3.4237919029100743), np.float64(0.5748109433182778), np.float64(0.3153721063883039), np.float64(0.10801892933807375)]


In [277]:
lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs1_classification_subgroup)

print(lower_bounds_usgan_obs1_classification_subgroup)
print(upper_bounds_usgan_obs1_classification_subgroup)

[np.float64(0.10341447344675987), np.float64(0.31431446506705324), np.float64(0.0), np.float64(0.0), np.float64(0.24338271493885694), np.float64(0.12557371883461996), np.float64(0.07967143098146552)]
[np.float64(0.3515898574325419), np.float64(0.6213040379991732), np.float64(3.550606754912776), np.float64(2.776161767415225), np.float64(0.5904861410917989), np.float64(0.32151307089064934), np.float64(0.10900543397402626)]


#### Mean values of lower bound and upper bound

In [278]:
mean_values_ci_usgan_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup)

print(mean_values_ci_usgan_obs1_classification_subgroup)

[np.float64(0.22750216543965088), np.float64(0.46780925153311326), np.float64(1.775303377456388), np.float64(1.3880808837076124), np.float64(0.4169344280153279), np.float64(0.22354339486263464), np.float64(0.0943384324777459)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [279]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [280]:
means_bootstraps_usgan_obs2_classification_subgroup, standards_deviations_usgan_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs2_classification_subgroup)

print(means_bootstraps_usgan_obs2_classification_subgroup)
print(standards_deviations_usgan_obs2_classification_subgroup)


[np.float64(1.0076600652125578), np.float64(0.2057323379031958), np.float64(1.5921744316354691), np.float64(0.0), np.float64(0.4757423755461549), np.float64(0.46722605932707173), np.float64(0.09404879461936072)]
[np.float64(0.8240521485012194), np.float64(0.05033909687690472), np.float64(1.2007615073986333), np.float64(0.0), np.float64(0.07510613780784255), np.float64(0.242814475426298), np.float64(0.01111102819636438)]


#### Calculating lower bound and upper bound

In [281]:
lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs2_classification_subgroup, means_bootstraps_usgan_obs2_classification_subgroup, standards_deviations_usgan_obs2_classification_subgroup)

print(lower_bounds_usgan_obs2_classification_subgroup)
print(upper_bounds_usgan_obs2_classification_subgroup)

[np.float64(-0.6074524672326846), np.float64(0.10706952101018984), np.float64(-0.7612748768878776), np.float64(0.0), np.float64(0.3285370504248814), np.float64(-0.008681567433458326), np.float64(0.07227157952327751)]
[np.float64(2.6227725976578), np.float64(0.3043951547962018), np.float64(3.945623740158816), np.float64(0.0), np.float64(0.6229477006674284), np.float64(0.9431336860876018), np.float64(0.11582600971544393)]


In [282]:
lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs2_classification_subgroup)

print(lower_bounds_usgan_obs2_classification_subgroup)
print(upper_bounds_usgan_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.10602982518456486), np.float64(0.0), np.float64(0.0), np.float64(0.33026697848310405), np.float64(0.0818633952588649), np.float64(0.07379316930962601)]
[np.float64(2.7938387017906305), np.float64(0.30693167812892086), np.float64(2.4977461887267776), np.float64(0.0), np.float64(0.6239750749760374), np.float64(0.985034801599763), np.float64(0.11735426580311797)]


#### Mean values of lower bound and upper bound

In [283]:
mean_values_ci_usgan_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup)

print(mean_values_ci_usgan_obs2_classification_subgroup)

[np.float64(1.3969193508953153), np.float64(0.20648075165674284), np.float64(1.2488730943633888), np.float64(0.0), np.float64(0.47712102672957074), np.float64(0.533449098429314), np.float64(0.09557371755637199)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [284]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [285]:
means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs3_classification_subgroup)

print(means_bootstraps_usgan_obs3_classification_subgroup)
print(standards_deviations_usgan_obs3_classification_subgroup)


[np.float64(1.09687350132991), np.float64(0.3968254043557604), np.float64(0.0), np.float64(1.2754181251728833), np.float64(0.5336452979888663), np.float64(0.8021763745635597), np.float64(0.1906829133923447)]
[np.float64(0.7790474208032958), np.float64(0.11673235669558012), np.float64(0.0), np.float64(0.9809216973353783), np.float64(0.15523517934126804), np.float64(0.5561448106529544), np.float64(0.023844462803084568)]


#### Calculating lower bound and upper bound

In [286]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs3_classification_subgroup, means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

[np.float64(-0.43003138569336974), np.float64(0.16803418940194034), np.float64(0.0), np.float64(-0.6471530732583575), np.float64(0.2293899373463647), np.float64(-0.2878474245050786), np.float64(0.14394862506759395)]
[np.float64(2.6237783883531898), np.float64(0.6256166193095805), np.float64(0.0), np.float64(3.197989323604124), np.float64(0.8379006586313679), np.float64(1.892200173632198), np.float64(0.23741720171709543)]


In [287]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

[np.float64(0.21440702645782947), np.float64(0.19886143006735535), np.float64(0.0), np.float64(0.0), np.float64(0.21506725896909207), np.float64(0.0), np.float64(0.14713183932990195)]
[np.float64(2.919325100838927), np.float64(0.650154610833248), np.float64(0.0), np.float64(2.029843170037303), np.float64(0.819490112208067), np.float64(2.000817349316018), np.float64(0.240083590025278)]


#### Mean values of lower bound and upper bound

In [288]:
mean_values_ci_usgan_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup)

print(mean_values_ci_usgan_obs3_classification_subgroup)

[np.float64(1.5668660636483782), np.float64(0.42450802045030167), np.float64(0.0), np.float64(1.0149215850186515), np.float64(0.5172786855885796), np.float64(1.000408674658009), np.float64(0.19360771467758997)]


### Confidence Interval (Obesity subgroup)

In [67]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_usgan_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



In [68]:
means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs3_classification_subgroup)

print(means_bootstraps_usgan_obs3_classification_subgroup)
print(standards_deviations_usgan_obs3_classification_subgroup)


[np.float64(0.3332880303893282), np.float64(0.5032999807986784), np.float64(3.7578895700970163), np.float64(1.743707224822977), np.float64(1.0268204524806999), np.float64(0.2959756913184028), np.float64(0.06905533274411292)]
[np.float64(0.04950821031921818), np.float64(0.20180963625591736), np.float64(1.1585578928621527), np.float64(0.4131067210904921), np.float64(0.2867446399243165), np.float64(0.127801480480534), np.float64(0.003466926499685081)]


In [69]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

[np.float64(0.23393090466994923), np.float64(0.23801724972510263), np.float64(0.0), np.float64(1.0070454348515294), np.float64(0.5259391841703286), np.float64(0.1388890545964323), np.float64(0.06262584784829395)]
[np.float64(0.43002298410943507), np.float64(0.9704570818796826), np.float64(5.477538784774831), np.float64(2.597205414064576), np.float64(1.6456418467460987), np.float64(0.5960350835248346), np.float64(0.0762240908944591)]


In [70]:
mean_values_ci_usgan_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup)

print(mean_values_ci_usgan_obs3_classification_subgroup)

[np.float64(0.33197694438969216), np.float64(0.6042371658023926), np.float64(2.7387693923874155), np.float64(1.802125424458053), np.float64(1.0857905154582137), np.float64(0.36746206906063344), np.float64(0.06942496937137652)]


## GPVAE

### Calculate MAE and AE

In [71]:
testing_mae_gpvae_variables_standard, testing_ae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for GPVAE model

#### Difference between gender group

In [290]:
diff_gpvae_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 1, 2, variables)
diff_gpvae_gender_subgroup

[('Cholesterol', 0.7169802683144736),
 ('SaO2', 0.208904652557992),
 ('AST', 0.2024031290671195),
 ('Creatinine', 0.1454782316251666),
 ('Lactate', 0.10004746063821912)]

#### Difference between age group

In [291]:
diff_gpvae_age_subgroup = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 4, 5, variables)
diff_gpvae_age_subgroup

[('Cholesterol', 0.49952280234507207),
 ('ALP', 0.4445243396078029),
 ('TroponinI', 0.2618853617171846),
 ('SaO2', 0.21621031134391283),
 ('Albumin', 0.1911062176221232)]

#### Difference between ICUType group

In [292]:
diff_gpvae_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 7, variables)
diff_gpvae_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 8, variables)
diff_gpvae_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 9, variables)
diff_gpvae_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 7, 8, variables)
diff_gpvae_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 7, 9, variables)
diff_gpvae_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 8, 9, variables)

In [293]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_gpvae_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_gpvae_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_gpvae_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.950238764897089), ('TroponinI', 0.8918129993253048), ('TroponinT', 0.31149184067106683), ('HCO3', 0.23628764938024638), ('SaO2', 0.23145876351413197)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 0.9937365425854698), ('ALP', 0.8302466803907333), ('TroponinI', 0.7412846032389209), ('Cholesterol', 0.40665526960681586), ('TroponinT', 0.37480989564613776)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinT', 0.43471785879530445), ('TroponinI', 0.43173406997364017), ('Cholesterol', 0.31119054427499326), ('ALP', 0.24020579848907225), ('Bilirubin', 0.22277039350931876)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.225195306099602), ('ALP', 0.7677479140911398), ('Cholesterol', 0.5435834952902732), ('PaCO2', 0.4072917838789667), ('HCO3', 0.3961429003610216)]
----------------------------
Diferenç

In [294]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[31])
print(variables[0])
print(variables[6])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
TroponinI
ALP
Cholesterol
TroponinT


#### Difference between BMI group

In [295]:
diff_gpvae_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 11, variables)
diff_gpvae_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 12, variables)
diff_gpvae_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 13, variables)
diff_gpvae_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 14, variables)
diff_gpvae_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 15, variables)
diff_gpvae_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 16, variables)

diff_gpvae_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 12, variables)
diff_gpvae_classification_low_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 13, variables)
diff_gpvae_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 14, variables)
diff_gpvae_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 15, variables)
diff_gpvae_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 16, variables)

diff_gpvae_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 13, variables)
diff_gpvae_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 14, variables)
diff_gpvae_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 15, variables)
diff_gpvae_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 16, variables)

diff_gpvae_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 14, variables)
diff_gpvae_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 15, variables)
diff_gpvae_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 16, variables)


diff_gpvae_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 14, 15, variables)
diff_gpvae_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 14, 16, variables)

diff_gpvae_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 15, 16, variables)

In [296]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_gpvae_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_gpvae_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_gpvae_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_gpvae_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_gpvae_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_gpvae_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.4984266717947166), ('Bilirubin', 0.9811022064443601), ('TroponinI', 0.8126945346557858), ('Cholesterol', 0.5201112246042136), ('Lactate', 0.45538779618545666)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('PaCO2', 0.2641364807062686), ('SaO2', 0.2556225387559883), ('ALT', 0.20532115838786913), ('Platelets', 0.17056081465907347), ('HCO3', 0.16334610029552787)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.8097416193905995), ('TroponinI', 0.33051277651707), ('TroponinT', 0.28973030251183335), ('PaCO2', 0.25794224224957285), ('SaO2', 0.24006655113431974)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8803622264377614), ('SaO2', 0.25914302071928913), ('PaCO2', 0.25683713011890197), ('Cholesterol', 0.23642198619398747),

In [297]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_gpvae_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_gpvae_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_gpvae_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_gpvae_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_gpvae_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.403020795206654), ('Bilirubin', 0.9461131764204705), ('TroponinI', 0.6834864613719018), ('Cholesterol', 0.5801196462081181), ('HCO3', 0.5142684723885946)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.583589914917147), ('Cholesterol', 1.329852843994813), ('TroponinI', 1.1432073111728558), ('Bilirubin', 1.1287287567764515), ('HCO3', 0.5693485437198301)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.627193842369981), ('Bilirubin', 1.085231063664624), ('Ph', 0.8728845981266398), ('TroponinI', 0.5899976625704627), ('HCO3', 0.5813665028848606)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.6404697550932625), ('Bilirubin', 1.0958804314411412), ('TroponinT', 0.7260296004902231), ('HCO3', 0.5949355182340001), ('ALT', 0.49225499419649044)]
-----

In [298]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_gpvae_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_gpvae_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_gpvae_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_gpvae_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('Cholesterol', 0.749733197786695), ('TroponinI', 0.45972084980095396), ('TroponinT', 0.3535738832041319), ('ALT', 0.3216961722524321), ('Bilirubin', 0.1826155803559808)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8880095885483712), ('Cholesterol', 0.296430407797892), ('ALP', 0.22417304716332703), ('Mg', 0.17276665999162977), ('Creatinine', 0.16845675537186644)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.017187057977287), ('TroponinI', 0.6834864613719018), ('ALT', 0.5398162885004267), ('Cholesterol', 0.33531880605432773), ('BUN', 0.2971607951252878)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('ALT', 1.4356376494699443), ('TroponinT', 0.8074674122243994), ('Cholesterol', 0.6091613153559619), ('TroponinI', 0.41581575902512413), ('Wei

In [299]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_gpvae_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_gpvae_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_gpvae_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 1.0461636055845869), ('Ph', 0.8934709277431419), ('TroponinI', 0.5532096486023931), ('TroponinT', 0.3830508683751487), ('ALT', 0.26753240411641177)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinI', 1.1432073111728558), ('Cholesterol', 1.0850520038410227), ('TroponinT', 0.6636131747731551), ('BUN', 0.31705896901327235), ('AST', 0.21918738293681142)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.3588945131426569), ('ALT', 1.1139414772175122), ('TroponinI', 0.8755366088260781), ('TroponinT', 0.45389352902026747), ('Weight', 0.36850455924257874)]
----------------------------


In [300]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_gpvae_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_gpvae_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.0466640431483036), ('Ph', 0.8973569851782044), ('TroponinI', 0.5899976625704627), ('ALT', 0.4856525203644064), ('BUN', 0.25574853492131433)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('ALT', 1.381473881333924), ('Ph', 0.8941614316903403), ('TroponinT', 0.8369443973954162), ('Glucose', 0.3284075159618258), ('TroponinI', 0.322326960223685)]
----------------------------


In [301]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_gpvae_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('ALT', 0.8958213609695176), ('HCO3', 0.31834748702089233), ('Cholesterol', 0.2738425093016341), ('TroponinI', 0.2676707023467777), ('Weight', 0.26352289014969055)]
----------------------------


In [302]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[1])
print(variables[31])
print(variables[6])
print(variables[0])
print(variables[32])

Top 5 variáveis BMI
-----------------------
ALT
TroponinI
Cholesterol
ALP
TroponinT


### Confidence Interval (General)(variable)

#### Distribution Bootstrap per variable

In [28]:
results_bootstrap_gpvae_general = []

for i in range(len(variables)):
       results_bootstrap_gpvae_general.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 0, i, 9000))

#### Calculating lower bound and upper bound

In [29]:
lower_bounds_gpvae_general, upper_bounds_gpvae_general = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_general)

print(lower_bounds_gpvae_general)
print(upper_bounds_gpvae_general)

[np.float64(0.3944211243470703), np.float64(0.28513692965250315), np.float64(0.3077386949598021), np.float64(0.639580742723036), np.float64(0.5294750516526141), np.float64(0.3812194859590158), np.float64(0.6838443274584377), np.float64(0.4469405681641526), np.float64(0.44571205566183697), np.float64(0.6358827904973715), np.float64(0.500273797888857), np.float64(0.5583322192095744), np.float64(0.670881309206223), np.float64(0.7121197788381078), np.float64(0.3510381877473058), np.float64(0.6526223996995617), np.float64(0.4990587521779125), np.float64(0.4059745068304414), np.float64(0.0017638097511847785), np.float64(0.5590069119565632), np.float64(0.4987885593419733), np.float64(0.4366745925492546), np.float64(0.4714718289611185), np.float64(0.6355729971514541), np.float64(0.6555629327986514), np.float64(0.5812078084481357), np.float64(0.6962292637128991), np.float64(0.5614197308225645), np.float64(0.3785402102433225), np.float64(0.4796449587377427), np.float64(0.43888854499401964), np.f

#### Mean values of lower bound and upper bound


In [30]:
mean_values_ci_gpvae_general = toolkits.calc_mean_values_ci(lower_bounds_gpvae_general, upper_bounds_gpvae_general)

print(mean_values_ci_gpvae_general)

[np.float64(0.5610388446003297), np.float64(0.3945044151186513), np.float64(0.45820255984774033), np.float64(0.7174290256844411), np.float64(0.568159534014814), np.float64(0.4881734194490813), np.float64(0.9469926169144103), np.float64(0.49615611363877793), np.float64(0.4575436259100705), np.float64(0.6632219918178145), np.float64(0.5153427344201409), np.float64(0.6044277265692288), np.float64(0.7142223682127626), np.float64(0.7475181416925641), np.float64(0.3576014856514783), np.float64(0.6960021481412633), np.float64(0.5509764588738819), np.float64(0.4180254044044515), np.float64(0.0020797651777564583), np.float64(0.617935641691271), np.float64(0.5116161806041162), np.float64(0.4486108496275437), np.float64(0.4841896794428018), np.float64(0.6772019067365282), np.float64(0.6940396078793953), np.float64(0.6129738200422303), np.float64(0.73992276946427), np.float64(0.5805649526422816), np.float64(0.44274919409329067), np.float64(0.4915341275154468), np.float64(0.45500058140122357), np.f

### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [303]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_gpvae_female = []

for i in indices:
       results_bootstrap_gpvae_female.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [304]:
means_bootstraps_gpvae_female,standards_deviations_gpvae_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_female)

print(means_bootstraps_gpvae_female)
print(standards_deviations_gpvae_female)


[np.float64(1.4784296439985511), np.float64(0.6346125578049744), np.float64(0.6658556689268423), np.float64(0.5047877952491602), np.float64(0.4133804338902489), np.float64(0.4778377264565393), np.float64(0.3996265558889345), np.float64(0.715877756049717), np.float64(0.46254590214727825), np.float64(0.4781321012693745), np.float64(0.25177750744440364), np.float64(0.7732471096940656)]
[np.float64(0.2727897334731474), np.float64(0.06570850574192079), np.float64(0.12865133750274774), np.float64(0.03933130639515322), np.float64(0.009023099824856844), np.float64(0.09414525152874836), np.float64(0.025991005039167455), np.float64(0.03491525957464862), np.float64(0.06327222055532057), np.float64(0.09414736068511752), np.float64(0.0059375992774970115), np.float64(0.0318658287489584)]


#### Calculating lower bound and upper bound

In [305]:
lower_bounds_gpvae_female, upper_bounds_gpvae_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_female, means_bootstraps_gpvae_female,standards_deviations_gpvae_female)

print(lower_bounds_gpvae_female)
print(upper_bounds_gpvae_female)

[np.float64(0.9437715910389018), np.float64(0.5058262530728663), np.float64(0.4137036808585496), np.float64(0.42769985124974996), np.float64(0.39569548320461984), np.float64(0.293316424144728), np.float64(0.3486851220901672), np.float64(0.6474451047725385), np.float64(0.33853462863697503), np.float64(0.2936066650870419), np.float64(0.24014002670587845), np.float64(0.7107912330085862)]
[np.float64(2.0130876969582006), np.float64(0.7633988625370824), np.float64(0.9180076569951351), np.float64(0.5818757392485704), np.float64(0.431065384575878), np.float64(0.6623590287683506), np.float64(0.45056798968770173), np.float64(0.7843104073268956), np.float64(0.5865571756575815), np.float64(0.6626575374517071), np.float64(0.2634149881829288), np.float64(0.8357029863795451)]


In [306]:
lower_bounds_gpvae_female, upper_bounds_gpvae_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_female)

print(lower_bounds_gpvae_female)
print(upper_bounds_gpvae_female)

[np.float64(0.9760930036143637), np.float64(0.5108515317146598), np.float64(0.4460011935656741), np.float64(0.4324917449781014), np.float64(0.3957924074058541), np.float64(0.3191506035579622), np.float64(0.351312503696497), np.float64(0.6503337294875813), np.float64(0.35251040989403576), np.float64(0.31822319826434886), np.float64(0.24069919742964116), np.float64(0.7121808780413587)]
[np.float64(1.9689622140552305), np.float64(0.7704282439596006), np.float64(0.9501696090406971), np.float64(0.5844560030948954), np.float64(0.4308725273776041), np.float64(0.6827334450717364), np.float64(0.4543315081661061), np.float64(0.7874564596970779), np.float64(0.6003170299803641), np.float64(0.6812203116195928), np.float64(0.263735237497243), np.float64(0.8363899964996575)]


#### Mean values of lower bound and upper bound


In [307]:
mean_values_ci_gpvae_female = toolkits.calc_mean_values_ci(lower_bounds_gpvae_female, upper_bounds_gpvae_female)

print(mean_values_ci_gpvae_female)

[np.float64(1.4725276088347972), np.float64(0.6406398878371302), np.float64(0.6980854013031856), np.float64(0.5084738740364985), np.float64(0.41333246739172913), np.float64(0.5009420243148492), np.float64(0.4028220059313016), np.float64(0.7188950945923296), np.float64(0.47641371993719994), np.float64(0.49972175494197085), np.float64(0.25221721746344206), np.float64(0.7742854372705081)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [308]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_gpvae_male = []

for i in indices:
       results_bootstrap_gpvae_male.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [309]:
means_bootstraps_gpvae_male,standards_deviations_gpvae_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_male)

print(means_bootstraps_gpvae_male)
print(standards_deviations_gpvae_male)


[np.float64(0.7751840061116742), np.float64(0.6395490783745216), np.float64(0.45733233919311594), np.float64(0.6046584351301509), np.float64(0.47105975206943085), np.float64(0.275574120223454), np.float64(0.5442633655358873), np.float64(0.6558303347768325), np.float64(0.39920741379159624), np.float64(0.27605558014098464), np.float64(0.2712060974722121), np.float64(0.6857898077841537)]
[np.float64(0.18336483117397556), np.float64(0.07735515366153932), np.float64(0.032013021422004205), np.float64(0.038185066708398045), np.float64(0.011218992426769617), np.float64(0.030209078163002407), np.float64(0.039467792524263245), np.float64(0.027166568314603113), np.float64(0.0403589140361658), np.float64(0.030687420902005387), np.float64(0.005534394380315114), np.float64(0.02526005700029636)]


#### Calculating lower bound and upper bound

In [310]:
lower_bounds_gpvae_male, upper_bounds_gpvae_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_male, means_bootstraps_gpvae_male,standards_deviations_gpvae_male)

print(lower_bounds_gpvae_male)
print(upper_bounds_gpvae_male)

[np.float64(0.4157955409794147), np.float64(0.48793576317934284), np.float64(0.3945879701696785), np.float64(0.5298170796344314), np.float64(0.4490709309701348), np.float64(0.21636541501781387), np.float64(0.4669079136390321), np.float64(0.6025848392966634), np.float64(0.3201053958255632), np.float64(0.21590934039463242), np.float64(0.2603588838105536), np.float64(0.6362810058161439)]
[np.float64(1.1345724712439336), np.float64(0.7911623935697003), np.float64(0.5200767082165534), np.float64(0.6794997906258705), np.float64(0.4930485731687269), np.float64(0.33478282542909416), np.float64(0.6216188174327424), np.float64(0.7090758302570016), np.float64(0.4783094317576293), np.float64(0.3362018198873369), np.float64(0.28205331113387055), np.float64(0.7352986097521634)]


In [311]:
lower_bounds_gpvae_male, upper_bounds_gpvae_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_male)

print(lower_bounds_gpvae_male)
print(upper_bounds_gpvae_male)

[np.float64(0.42948209662696085), np.float64(0.4871575172104016), np.float64(0.3996647820671073), np.float64(0.531885147815444), np.float64(0.4498717192917469), np.float64(0.21988363189442964), np.float64(0.46959502417838817), np.float64(0.6038156038041039), np.float64(0.3256889085718612), np.float64(0.2194620478023794), np.float64(0.2606997629201076), np.float64(0.6379165192301737)]
[np.float64(1.151746234023552), np.float64(0.7620334359351716), np.float64(0.5250108928514159), np.float64(0.6820707750288179), np.float64(0.4939818295723298), np.float64(0.3376357969068453), np.float64(0.6228840225930147), np.float64(0.7096529362158059), np.float64(0.48418192961770296), np.float64(0.3392690535414582), np.float64(0.2821234816259759), np.float64(0.7369464781107631)]


#### Mean values of lower bound and upper bound


In [312]:
mean_values_ci_gpvae_male = toolkits.calc_mean_values_ci(lower_bounds_gpvae_male, upper_bounds_gpvae_male)

print(mean_values_ci_gpvae_male)

[np.float64(0.7906141653252564), np.float64(0.6245954765727866), np.float64(0.4623378374592616), np.float64(0.6069779614221309), np.float64(0.47192677443203834), np.float64(0.27875971440063746), np.float64(0.5462395233857014), np.float64(0.656734270009955), np.float64(0.4049354190947821), np.float64(0.2793655506719188), np.float64(0.2714116222730417), np.float64(0.6874314986704684)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [313]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [314]:
means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup,standards_deviations_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup)
print(standards_deviations_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.6075113310318807), np.float64(0.40169414017253074), np.float64(0.6985872412988392), np.float64(0.46720971449613724), np.float64(0.4200324600474383), np.float64(1.0273728784797138), np.float64(0.5625953730311186)]
[np.float64(0.11222612921081021), np.float64(0.05470352009831539), np.float64(0.13622945539185924), np.float64(0.034312916864910226), np.float64(0.03068957161523071), np.float64(0.28363850500375226), np.float64(0.06246180573421691)]


#### Calculating lower bound and upper bound


In [315]:
lower_bounds_gpvae_more_than_or_equal_to_65_subgroup, upper_bounds_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup, means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup,standards_deviations_gpvae_more_than_or_equal_to_65_subgroup)

print(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup)
print(upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)


[np.float64(0.3875521596543542), np.float64(0.2944772109522696), np.float64(0.43158241509728923), np.float64(0.39995763323639616), np.float64(0.3598820049806234), np.float64(0.47145162404357543), np.float64(0.44017248338271603)]
[np.float64(0.8274705024094072), np.float64(0.5089110693927918), np.float64(0.9655920675003892), np.float64(0.5344617957558783), np.float64(0.48018291511425326), np.float64(1.583294132915852), np.float64(0.6850182626795212)]


In [316]:
lower_bounds_gpvae_more_than_or_equal_to_65_subgroup, upper_bounds_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup)

print(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup)
print(upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.38894509892644263), np.float64(0.31053431019230265), np.float64(0.4629076592460675), np.float64(0.4050747961430211), np.float64(0.3613554346857964), np.float64(0.54902428917022), np.float64(0.4444792499861261)]
[np.float64(0.8292015462164367), np.float64(0.5219112132116659), np.float64(0.9938426646299805), np.float64(0.5372630027277865), np.float64(0.4828468428868046), np.float64(1.6425898139157131), np.float64(0.6892401677071763)]


#### Mean values of lower bound and upper bound

In [317]:
mean_values_ci_gpvae_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup , upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.6090733225714396), np.float64(0.41622276170198425), np.float64(0.728375161938024), np.float64(0.4711688994354038), np.float64(0.4221011387863005), np.float64(1.0958070515429665), np.float64(0.5668597088466512)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [318]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_gpvae_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_gpvae_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [319]:
means_bootstraps_gpvae_less_than_65_subgroup,standards_deviations_gpvae_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_less_than_65_subgroup)

print(means_bootstraps_gpvae_less_than_65_subgroup)
print(standards_deviations_gpvae_less_than_65_subgroup)


[np.float64(0.9541384760280642), np.float64(0.48995002310963737), np.float64(1.1451659474922375), np.float64(0.6842474033224641), np.float64(0.507576509610734), np.float64(0.7655724617993821), np.float64(0.7533088284239882)]
[np.float64(0.733037922899086), np.float64(0.14822261346792814), np.float64(0.43319085335782037), np.float64(0.12753912340576318), np.float64(0.0782598314306134), np.float64(0.08024832734321716), np.float64(0.06045738323134545)]


#### Calculating lower bound and upper bound

In [320]:
lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_less_than_65_subgroup, means_bootstraps_gpvae_less_than_65_subgroup, standards_deviations_gpvae_less_than_65_subgroup)

print(lower_bounds_gpvae_less_than_65_subgroup)
print(upper_bounds_gpvae_less_than_65_subgroup)

[np.float64(-0.4825894521561933), np.float64(0.19943903901809662), np.float64(0.2961274764787377), np.float64(0.43427531482735887), np.float64(0.35419005857055597), np.float64(0.6082886303870956), np.float64(0.6348145346910152)]
[np.float64(2.3908664042123218), np.float64(0.7804610072011782), np.float64(1.9942044185057375), np.float64(0.9342194918175695), np.float64(0.660962960650912), np.float64(0.9228562932116686), np.float64(0.8718031221569611)]


In [321]:
lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_less_than_65_subgroup)

print(lower_bounds_gpvae_less_than_65_subgroup)
print(upper_bounds_gpvae_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.266152856269189), np.float64(0.477576491044506), np.float64(0.4698534566457397), np.float64(0.3680750691994851), np.float64(0.6167867437340325), np.float64(0.6377937196504697)]
[np.float64(1.996025239859798), np.float64(0.8296131129143057), np.float64(2.1095147411028314), np.float64(0.966332623569135), np.float64(0.6749266871038238), np.float64(0.9265717834857955), np.float64(0.8760262032992041)]


#### Mean values of lower bound and upper bound

In [322]:
mean_values_ci_gpvae_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup)

print(mean_values_ci_gpvae_less_than_65_subgroup)

[np.float64(0.998012619929899), np.float64(0.5478829845917473), np.float64(1.2935456160736687), np.float64(0.7180930401074374), np.float64(0.5215008781516545), np.float64(0.771679263609914), np.float64(0.7569099614748369)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [323]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [324]:
means_bootstraps_gpvae_ICUType_1_subgroup, standards_deviations_gpvae_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_1_subgroup)

print(means_bootstraps_gpvae_ICUType_1_subgroup)
print(standards_deviations_gpvae_ICUType_1_subgroup)


[np.float64(0.647971142551637), np.float64(1.3109117702464457), np.float64(0.9527436447101069), np.float64(0.33583584680389234), np.float64(0.6999858024671683), np.float64(0.6618415174643225)]
[np.float64(0.17749613484525562), np.float64(0.4047981800926564), np.float64(0.2558986783190476), np.float64(0.03683167713572748), np.float64(0.124535322223324), np.float64(0.05042998064950698)]


#### Calculating lower bound and upper bound

In [325]:
lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_1_subgroup, means_bootstraps_gpvae_ICUType_1_subgroup, standards_deviations_gpvae_ICUType_1_subgroup)

print(lower_bounds_gpvae_ICUType_1_subgroup)
print(upper_bounds_gpvae_ICUType_1_subgroup)

[np.float64(0.3000851108598711), np.float64(0.5175219162574805), np.float64(0.4511914515133728), np.float64(0.2636470861276591), np.float64(0.45590105610636267), np.float64(0.563000571650237)]
[np.float64(0.9958571742434029), np.float64(2.104301624235411), np.float64(1.454295837906841), np.float64(0.4080246074801256), np.float64(0.944070548827974), np.float64(0.760682463278408)]


In [326]:
lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_1_subgroup)

print(lower_bounds_gpvae_ICUType_1_subgroup)
print(upper_bounds_gpvae_ICUType_1_subgroup)

[np.float64(0.4006885896415936), np.float64(0.5911777502043112), np.float64(0.510039203885049), np.float64(0.26777400620496317), np.float64(0.4798981431345316), np.float64(0.5660431229104093)]
[np.float64(1.066454750840184), np.float64(2.1542929834966387), np.float64(1.5178276787005762), np.float64(0.4099803117166312), np.float64(0.964535164070263), np.float64(0.7652582594963171)]


#### Mean values of lower bound and upper bound

In [327]:
mean_values_ci_gpvae_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup)

print(mean_values_ci_gpvae_ICUType_1_subgroup)

[np.float64(0.7335716702408888), np.float64(1.372735366850475), np.float64(1.0139334412928127), np.float64(0.3388771589607972), np.float64(0.7222166536023973), np.float64(0.6656506912033632)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [328]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [329]:
means_bootstraps_gpvae_ICUType_2_subgroup, standards_deviations_gpvae_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_2_subgroup)

print(means_bootstraps_gpvae_ICUType_2_subgroup)
print(standards_deviations_gpvae_ICUType_2_subgroup) 


[np.float64(0.41794327245447443), np.float64(0.4163908500737795), np.float64(0.0), np.float64(0.39894156742501596), np.float64(0.39022895270301644), np.float64(0.5783926320936834)]
[np.float64(0.025688982794294057), np.float64(0.12870736290137288), np.float64(0.0), np.float64(0.03113478789201071), np.float64(0.04955620256549535), np.float64(0.022372866121972573)]


#### Calculating lower bound and upper bound

In [330]:
lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_2_subgroup, means_bootstraps_gpvae_ICUType_2_subgroup, standards_deviations_gpvae_ICUType_2_subgroup)

print(lower_bounds_gpvae_ICUType_2_subgroup)
print(upper_bounds_gpvae_ICUType_2_subgroup)

[np.float64(0.367593791378189), np.float64(0.164129054241962), np.float64(0.0), np.float64(0.33791850449038124), np.float64(0.2931005804640741), np.float64(0.5345426202636808)]
[np.float64(0.4682927535307599), np.float64(0.668652645905597), np.float64(0.0), np.float64(0.4599646303596507), np.float64(0.48735732494195877), np.float64(0.622242643923686)]


In [331]:
lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_2_subgroup)

print(lower_bounds_gpvae_ICUType_2_subgroup)
print(upper_bounds_gpvae_ICUType_2_subgroup)

[np.float64(0.37036516607744074), np.float64(0.11452376477641221), np.float64(0.0), np.float64(0.3366056927854989), np.float64(0.297644702669788), np.float64(0.5351831981357089)]
[np.float64(0.47124905881187745), np.float64(0.6708236791830013), np.float64(0.0), np.float64(0.46003569195119043), np.float64(0.4910302993504988), np.float64(0.6229423624990045)]


#### Mean values of lower bound and upper bound

In [332]:
mean_values_ci_gpvae_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup)

print(mean_values_ci_gpvae_ICUType_2_subgroup)

[np.float64(0.42080711244465907), np.float64(0.3926737219797068), np.float64(0.0), np.float64(0.39832069236834466), np.float64(0.3943375010101434), np.float64(0.5790627803173567)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [333]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [334]:
means_bootstraps_gpvae_ICUType_3_subgroup, standards_deviations_gpvae_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_3_subgroup)

print(means_bootstraps_gpvae_ICUType_3_subgroup)
print(standards_deviations_gpvae_ICUType_3_subgroup)


[np.float64(1.6444095749645153), np.float64(0.569178875511922), np.float64(0.5323719619215365), np.float64(1.1678205368293866), np.float64(0.32657869079095414), np.float64(0.9848962832214279)]
[np.float64(0.4398985064040355), np.float64(0.08652371625649781), np.float64(0.09231597832603668), np.float64(0.44711943716105695), np.float64(0.0314501995697657), np.float64(0.05245346162450552)]


#### Calculating lower bound and upper bound

In [335]:
lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_3_subgroup, means_bootstraps_gpvae_ICUType_3_subgroup, standards_deviations_gpvae_ICUType_3_subgroup)

print(lower_bounds_gpvae_ICUType_3_subgroup)
print(upper_bounds_gpvae_ICUType_3_subgroup)

[np.float64(0.7822243455596435), np.float64(0.39959550784062353), np.float64(0.35143596920492437), np.float64(0.29148254320589506), np.float64(0.26493743232761624), np.float64(0.8820893875729432)]
[np.float64(2.5065948043693873), np.float64(0.7387622431832206), np.float64(0.7133079546381486), np.float64(2.044158530452878), np.float64(0.38821994925429204), np.float64(1.0877031788699125)]


In [336]:
lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_3_subgroup)

print(lower_bounds_gpvae_ICUType_3_subgroup)
print(upper_bounds_gpvae_ICUType_3_subgroup)

[np.float64(0.8767888331778235), np.float64(0.3889976331197694), np.float64(0.4163152626267494), np.float64(0.4674803901010822), np.float64(0.27036091782794824), np.float64(0.8850003009773723)]
[np.float64(2.5919640240518427), np.float64(0.7273941789855666), np.float64(0.669703451290682), np.float64(2.150768078242269), np.float64(0.3938733553212669), np.float64(1.089515985978853)]


#### Mean values of lower bound and upper bound

In [337]:
mean_values_ci_gpvae_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup)

print(mean_values_ci_gpvae_ICUType_3_subgroup)

[np.float64(1.734376428614833), np.float64(0.558195906052668), np.float64(0.5430093569587157), np.float64(1.3091242341716756), np.float64(0.33211713657460756), np.float64(0.9872581434781127)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [338]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_gpvae_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [339]:
means_bootstraps_gpvae_ICUType_4_subgroup, standards_deviations_gpvae_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_4_subgroup)

print(means_bootstraps_gpvae_ICUType_4_subgroup)
print(standards_deviations_gpvae_ICUType_4_subgroup)


[np.float64(0.5415343845159007), np.float64(0.8620458759028283), np.float64(0.6371226508249529), np.float64(0.5771331665541501), np.float64(0.2669926468191781), np.float64(0.6247327563100636)]
[np.float64(0.08291296001571026), np.float64(0.20972708915034832), np.float64(0.2256268458269695), np.float64(0.07279584090282266), np.float64(0.046257430060419644), np.float64(0.026927824028438845)]


#### Calculating lower bound and upper bound

In [340]:
lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_4_subgroup, means_bootstraps_gpvae_ICUType_4_subgroup, standards_deviations_gpvae_ICUType_4_subgroup)

print(lower_bounds_gpvae_ICUType_4_subgroup)
print(upper_bounds_gpvae_ICUType_4_subgroup)

[np.float64(0.3790279690334991), np.float64(0.4509883345857245), np.float64(0.19490215905872127), np.float64(0.43445594016031), np.float64(0.17632974988337516), np.float64(0.5719551910322912)]
[np.float64(0.7040407999983024), np.float64(1.2731034172199323), np.float64(1.0793431425911846), np.float64(0.7198103929479903), np.float64(0.3576555437549811), np.float64(0.6775103215878361)]


In [341]:
lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_4_subgroup)

print(lower_bounds_gpvae_ICUType_4_subgroup)
print(upper_bounds_gpvae_ICUType_4_subgroup)

[np.float64(0.400746992727356), np.float64(0.610237826920164), np.float64(0.18501116021427075), np.float64(0.44579694164299494), np.float64(0.18685943376080993), np.float64(0.5738565206241549)]
[np.float64(0.7193865991945516), np.float64(1.3679670625417122), np.float64(1.1037751466151966), np.float64(0.7299828772538302), np.float64(0.3673500280139027), np.float64(0.679281480020438)]


#### Mean values of lower bound and upper bound

In [342]:
mean_values_ci_gpvae_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup)

print(mean_values_ci_gpvae_ICUType_4_subgroup)

[np.float64(0.5600667959609538), np.float64(0.9891024447309381), np.float64(0.6443931534147337), np.float64(0.5878899094484126), np.float64(0.2771047308873563), np.float64(0.6265690003222965)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [343]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [344]:
means_bootstraps_gpvae_undefined_classification_subgroup, standards_deviations_gpvae_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_undefined_classification_subgroup)

print(means_bootstraps_gpvae_undefined_classification_subgroup)
print(standards_deviations_gpvae_undefined_classification_subgroup)


[np.float64(0.3420571191823156), np.float64(0.48532012308882905), np.float64(0.5487681594379369), np.float64(0.8104390343707566), np.float64(0.5232206593289169), np.float64(0.4409402183673204), np.float64(0.2696675641743906)]
[np.float64(0.04010924423166225), np.float64(0.05048714537781482), np.float64(0.18755234473029045), np.float64(0.12325891228071231), np.float64(0.11706450862927992), np.float64(0.10734583025121357), np.float64(0.006727366405987162)]


#### Calculating lower bound and upper bound

In [345]:
lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_undefined_classification_subgroup, means_bootstraps_gpvae_undefined_classification_subgroup, standards_deviations_gpvae_undefined_classification_subgroup)

print(lower_bounds_gpvae_undefined_classification_subgroup)
print(upper_bounds_gpvae_undefined_classification_subgroup)

[np.float64(0.26344444504113673), np.float64(0.3863671364660741), np.float64(0.181172318550527), np.float64(0.5688560055269787), np.float64(0.29377843854764996), np.float64(0.2305462571843916), np.float64(0.2564821683078511)]
[np.float64(0.4206697933234945), np.float64(0.584273109711584), np.float64(0.9163640003253468), np.float64(1.0520220632145345), np.float64(0.7526628801101839), np.float64(0.6513341795502492), np.float64(0.2828529600409301)]


In [346]:
lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_undefined_classification_subgroup)

print(lower_bounds_gpvae_undefined_classification_subgroup)
print(upper_bounds_gpvae_undefined_classification_subgroup)

[np.float64(0.27367628052300536), np.float64(0.3945783261378865), np.float64(0.1972114982839213), np.float64(0.5985311497377979), np.float64(0.3403055148567253), np.float64(0.2736756668161414), np.float64(0.2566062047494302)]
[np.float64(0.43111222496037194), np.float64(0.5905214511463337), np.float64(0.9411443296678985), np.float64(1.0562454965429924), np.float64(0.7902657294842672), np.float64(0.6851780394476509), np.float64(0.2830929216187784)]


#### Mean values of lower bound and upper bound

In [347]:
mean_values_ci_gpvae_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup)

print(mean_values_ci_gpvae_undefined_classification_subgroup)

[np.float64(0.3523942527416887), np.float64(0.49254988864211013), np.float64(0.5691779139759099), np.float64(0.8273883231403951), np.float64(0.5652856221704963), np.float64(0.47942685313189615), np.float64(0.2698495631841043)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [348]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [349]:
means_bootstraps_gpvae_low_weight_classification_subgroup, standards_deviations_gpvae_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_low_weight_classification_subgroup)

print(means_bootstraps_gpvae_low_weight_classification_subgroup)
print(standards_deviations_gpvae_low_weight_classification_subgroup)


[np.float64(0.35982216955075197), np.float64(1.7206863924775067), np.float64(0.01837046916207931), np.float64(0.0), np.float64(1.4938689641091618), np.float64(0.28088768382125884), np.float64(0.31772621317817695)]
[np.float64(0.2744389705054808), np.float64(1.3298325812438752), np.float64(0.014001248167966803), np.float64(0.0), np.float64(1.2232153422373948), np.float64(0.13381929684828414), np.float64(0.02109427643120123)]


#### Calculating lower bound and upper bound

In [350]:
lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_low_weight_classification_subgroup, means_bootstraps_gpvae_low_weight_classification_subgroup, standards_deviations_gpvae_low_weight_classification_subgroup)

print(lower_bounds_gpvae_low_weight_classification_subgroup)
print(upper_bounds_gpvae_low_weight_classification_subgroup)

[np.float64(-0.1780683285942405), np.float64(-0.885737572228424), np.float64(-0.009071472985743036), np.float64(0.0), np.float64(-0.9035890520129684), np.float64(0.018606681562147565), np.float64(0.2763821910930904)]
[np.float64(0.8977126676957444), np.float64(4.327110357183438), np.float64(0.04581241130990166), np.float64(0.0), np.float64(3.891326980231292), np.float64(0.5431686860803702), np.float64(0.3590702352632635)]


In [351]:
lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_low_weight_classification_subgroup)

print(lower_bounds_gpvae_low_weight_classification_subgroup)
print(upper_bounds_gpvae_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.19080774777701526), np.float64(0.11261259211852517), np.float64(0.27656829360804314)]
[np.float64(0.569138756758905), np.float64(3.611804946896089), np.float64(0.029041669147863172), np.float64(0.0), np.float64(4.350187207370025), np.float64(0.5764182544284093), np.float64(0.35986233454626027)]


#### Mean values of lower bound and upper bound

In [352]:
mean_values_ci_gpvae_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup)

print(mean_values_ci_gpvae_low_weight_classification_subgroup)

[np.float64(0.2845693783794525), np.float64(1.8059024734480444), np.float64(0.014520834573931586), np.float64(0.0), np.float64(2.27049747757352), np.float64(0.34451542327346724), np.float64(0.3182153140771517)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [353]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [354]:
means_bootstraps_gpvae_normal_weight_classification_subgroup, standards_deviations_gpvae_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_normal_weight_classification_subgroup)

print(means_bootstraps_gpvae_normal_weight_classification_subgroup)
print(standards_deviations_gpvae_normal_weight_classification_subgroup)


[np.float64(0.2788390320993726), np.float64(0.5826311789318439), np.float64(0.5306000231089237), np.float64(0.6829394059127972), np.float64(0.5540098952897624), np.float64(0.23577808623873384), np.float64(0.2137000340048897)]
[np.float64(0.05601716229377656), np.float64(0.10811645166639203), np.float64(0.2868824930872985), np.float64(0.15451825318983628), np.float64(0.19153066742536184), np.float64(0.051160811187097366), np.float64(0.008383812962452693)]


#### Calculating lower bound and upper bound

In [355]:
lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_normal_weight_classification_subgroup, means_bootstraps_gpvae_normal_weight_classification_subgroup, standards_deviations_gpvae_normal_weight_classification_subgroup)

print(lower_bounds_gpvae_normal_weight_classification_subgroup)
print(upper_bounds_gpvae_normal_weight_classification_subgroup)

[np.float64(0.1690474114874354), np.float64(0.37072682752945), np.float64(-0.03167933113724242), np.float64(0.3800891947066768), np.float64(0.17861668520113427), np.float64(0.13550473889216913), np.float64(0.1972680625453624)]
[np.float64(0.3886306527113098), np.float64(0.7945355303342378), np.float64(1.0928793773550898), np.float64(0.9857896171189177), np.float64(0.9294031053783905), np.float64(0.33605143358529854), np.float64(0.23013200546441703)]


In [356]:
lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_normal_weight_classification_subgroup)

print(lower_bounds_gpvae_normal_weight_classification_subgroup)
print(upper_bounds_gpvae_normal_weight_classification_subgroup)

[np.float64(0.1843113749217874), np.float64(0.4043378112120809), np.float64(0.0), np.float64(0.39738037215446925), np.float64(0.25550646623866735), np.float64(0.14527077333652166), np.float64(0.19843681185086903)]
[np.float64(0.4044866753662465), np.float64(0.8197679901326286), np.float64(0.8911109597446285), np.float64(1.0047706208552805), np.float64(0.9858004020784817), np.float64(0.34695030192072507), np.float64(0.2315812691279074)]


#### Mean values of lower bound and upper bound

In [357]:
mean_values_ci_gpvae_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup)

print(mean_values_ci_gpvae_normal_weight_classification_subgroup)

[np.float64(0.29439902514401695), np.float64(0.6120529006723547), np.float64(0.44555547987231425), np.float64(0.7010754965048749), np.float64(0.6206534341585745), np.float64(0.24611053762862337), np.float64(0.21500904048938824)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [358]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [359]:
means_bootstraps_gpvae_overweight_classification_subgroup, standards_deviations_gpvae_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_overweight_classification_subgroup)

print(means_bootstraps_gpvae_overweight_classification_subgroup)
print(standards_deviations_gpvae_overweight_classification_subgroup)


[np.float64(0.6337160206380358), np.float64(0.4019553898702843), np.float64(1.3559128592837884), np.float64(1.1334013211894667), np.float64(0.37561656676233945), np.float64(0.5565544287614653), np.float64(0.214048018572171)]
[np.float64(0.14754883207866215), np.float64(0.055874328486758384), np.float64(0.3029911821234176), np.float64(0.5152485041728275), np.float64(0.07113234447007083), np.float64(0.15427225952444992), np.float64(0.005921683933105827)]


#### Calculating lower bound and upper bound

In [360]:
lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_overweight_classification_subgroup, means_bootstraps_gpvae_overweight_classification_subgroup, standards_deviations_gpvae_overweight_classification_subgroup)

print(lower_bounds_gpvae_overweight_classification_subgroup)
print(upper_bounds_gpvae_overweight_classification_subgroup)

[np.float64(0.34452562380290974), np.float64(0.29244371837587746), np.float64(0.7620610546886737), np.float64(0.12353280992258897), np.float64(0.23619973346510376), np.float64(0.25418635627992714), np.float64(0.2024417313354541)]
[np.float64(0.9229064174731618), np.float64(0.5114670613646911), np.float64(1.949764663878903), np.float64(2.1432698324563444), np.float64(0.5150334000595751), np.float64(0.8589225012430035), np.float64(0.22565430580888793)]


In [361]:
lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_overweight_classification_subgroup)

print(lower_bounds_gpvae_overweight_classification_subgroup)
print(upper_bounds_gpvae_overweight_classification_subgroup)

[np.float64(0.4157931427260761), np.float64(0.30296338339721923), np.float64(0.7438432847606096), np.float64(0.5368283997580229), np.float64(0.25806580132307677), np.float64(0.2869753352111305), np.float64(0.20260822167247478)]
[np.float64(0.973875000328913), np.float64(0.5211022550839362), np.float64(1.9052202326942167), np.float64(2.3068001475218898), np.float64(0.5338686403075361), np.float64(0.8925268655549834), np.float64(0.22567650936001737)]


#### Mean values of lower bound and upper bound

In [362]:
mean_values_ci_gpvae_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup)

print(mean_values_ci_gpvae_overweight_classification_subgroup)

[np.float64(0.6948340715274945), np.float64(0.41203281924057766), np.float64(1.3245317587274132), np.float64(1.4218142736399564), np.float64(0.39596722081530644), np.float64(0.5897511003830569), np.float64(0.21414236551624607)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [363]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [364]:
means_bootstraps_gpvae_obs1_classification_subgroup, standards_deviations_gpvae_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs1_classification_subgroup)

print(means_bootstraps_gpvae_obs1_classification_subgroup)
print(standards_deviations_gpvae_obs1_classification_subgroup)


[np.float64(0.24883366567057733), np.float64(0.35780846494064744), np.float64(0.29900311311799693), np.float64(0.5051498971812931), np.float64(0.418538090020395), np.float64(0.29008887236735187), np.float64(0.3052290223458991)]
[np.float64(0.07274376114425882), np.float64(0.041795048687896866), np.float64(0.2097041263248687), np.float64(0.22205194847051696), np.float64(0.112901389283256), np.float64(0.045555983153380095), np.float64(0.01068984112255131)]


#### Calculating lower bound and upper bound

In [365]:
lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs1_classification_subgroup, means_bootstraps_gpvae_obs1_classification_subgroup, standards_deviations_gpvae_obs1_classification_subgroup)

print(lower_bounds_gpvae_obs1_classification_subgroup)
print(upper_bounds_gpvae_obs1_classification_subgroup)

[np.float64(0.10625851372784587), np.float64(0.27589167478027155), np.float64(-0.11200942188818358), np.float64(0.06993607548213593), np.float64(0.1972554332206768), np.float64(0.20080078610641344), np.float64(0.2842773187452433)]
[np.float64(0.3914088176133088), np.float64(0.43972525510102334), np.float64(0.7100156481241775), np.float64(0.9403637188804503), np.float64(0.6398207468201131), np.float64(0.3793769586282903), np.float64(0.3261807259465549)]


In [366]:
lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs1_classification_subgroup)

print(lower_bounds_gpvae_obs1_classification_subgroup)
print(upper_bounds_gpvae_obs1_classification_subgroup)

[np.float64(0.12653426283784666), np.float64(0.27782518472963164), np.float64(0.0), np.float64(0.0), np.float64(0.25211093524760375), np.float64(0.20546832362401582), np.float64(0.2847696734096095)]
[np.float64(0.4105533068361425), np.float64(0.4404350103635942), np.float64(0.740125445023434), np.float64(0.7107154921295015), np.float64(0.6729622660164267), np.float64(0.3831318584058979), np.float64(0.326235623124969)]


#### Mean values of lower bound and upper bound

In [367]:
mean_values_ci_gpvae_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup)

print(mean_values_ci_gpvae_obs1_classification_subgroup)

[np.float64(0.2685437848369946), np.float64(0.35913009754661296), np.float64(0.370062722511717), np.float64(0.35535774606475073), np.float64(0.46253660063201524), np.float64(0.29430009101495685), np.float64(0.30550264826728923)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [368]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_gpvae_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [369]:
means_bootstraps_gpvae_obs2_classification_subgroup, standards_deviations_gpvae_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs2_classification_subgroup)

print(means_bootstraps_gpvae_obs2_classification_subgroup)
print(standards_deviations_gpvae_obs2_classification_subgroup)


[np.float64(1.2295954167318155), np.float64(0.34442750113258114), np.float64(0.1732814544970187), np.float64(0.0), np.float64(0.4088261420778865), np.float64(0.7727092147825283), np.float64(0.31946213528168343)]
[np.float64(0.899230702765293), np.float64(0.04331723972957595), np.float64(0.1320051735437259), np.float64(0.0), np.float64(0.08861566323598369), np.float64(0.3039682964989979), np.float64(0.021538435046573844)]


#### Calculating lower bound and upper bound

In [370]:
lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs2_classification_subgroup, means_bootstraps_gpvae_obs2_classification_subgroup, standards_deviations_gpvae_obs2_classification_subgroup)

print(lower_bounds_gpvae_obs2_classification_subgroup)
print(upper_bounds_gpvae_obs2_classification_subgroup)

[np.float64(-0.5328643744808013), np.float64(0.25952727135292475), np.float64(-0.08544393142164364), np.float64(0.0), np.float64(0.2351426336692283), np.float64(0.17694230120249976), np.float64(0.27724757830704344)]
[np.float64(2.992055207944432), np.float64(0.4293277309122375), np.float64(0.43200684041568105), np.float64(0.0), np.float64(0.5825096504865447), np.float64(1.3684761283625568), np.float64(0.36167669225632343)]


In [371]:
lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs2_classification_subgroup)

print(lower_bounds_gpvae_obs2_classification_subgroup)
print(upper_bounds_gpvae_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.2664236746833256), np.float64(0.0), np.float64(0.0), np.float64(0.2694019027212193), np.float64(0.2654285990459724), np.float64(0.2787589025828395)]
[np.float64(3.1824607622085654), np.float64(0.4359315545079977), np.float64(0.2738425093018167), np.float64(0.0), np.float64(0.6083622148703423), np.float64(1.4250064820421084), np.float64(0.3624556821804304)]


#### Mean values of lower bound and upper bound

In [372]:
mean_values_ci_gpvae_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup)

print(mean_values_ci_gpvae_obs2_classification_subgroup)

[np.float64(1.5912303811042827), np.float64(0.35117761459566166), np.float64(0.13692125465090835), np.float64(0.0), np.float64(0.4388820587957808), np.float64(0.8452175405440404), np.float64(0.32060729238163493)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [373]:
indices = [32, 0, 6, 31, 5, 1, 35]


results_bootstrap_gpvae_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [374]:
means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs3_classification_subgroup)

print(means_bootstraps_gpvae_obs3_classification_subgroup)
print(standards_deviations_gpvae_obs3_classification_subgroup)


[np.float64(1.082904634415875), np.float64(0.2543884757819845), np.float64(0.0), np.float64(0.16815668345211396), np.float64(0.470607312679061), np.float64(1.587710283434484), np.float64(0.5821759109812601)]
[np.float64(0.7599439230364026), np.float64(0.04234700526501501), np.float64(0.0), np.float64(0.12935975948620154), np.float64(0.1858509235236118), np.float64(1.0923134185816146), np.float64(0.0349145802865942)]


#### Calculating lower bound and upper bound

In [375]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs3_classification_subgroup, means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

[np.float64(-0.4065580850055528), np.float64(0.171389870609427), np.float64(0.0), np.float64(-0.08538378618960465), np.float64(0.10634619607927392), np.float64(-0.5531846768153053), np.float64(0.5137445910842033)]
[np.float64(2.572367353837303), np.float64(0.33738708095454195), np.float64(0.0), np.float64(0.4216971530938326), np.float64(0.834868429278848), np.float64(3.7286052436842736), np.float64(0.6506072308783168)]


In [376]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

[np.float64(0.18069389887825504), np.float64(0.1706558802364761), np.float64(0.0), np.float64(0.0), np.float64(0.16616333645478304), np.float64(0.0), np.float64(0.5141726327707242)]
[np.float64(2.8033925660574868), np.float64(0.33826795848343816), np.float64(0.0), np.float64(0.26767070234695617), np.float64(0.8760385102660416), np.float64(3.916981270671261), np.float64(0.6515707115363911)]


#### Mean values of lower bound and upper bound

In [377]:
mean_values_ci_gpvae_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup)

print(mean_values_ci_gpvae_obs3_classification_subgroup)

[np.float64(1.492043232467871), np.float64(0.2544619193599571), np.float64(0.0), np.float64(0.13383535117347808), np.float64(0.5211009233604124), np.float64(1.9584906353356304), np.float64(0.5828716721535576)]


### Confidence Interval (Obesity subgroup)

In [81]:
indices = [32, 0, 6, 31, 5, 1, 35]


results_bootstrap_gpvae_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



In [82]:
means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs3_classification_subgroup)

print(means_bootstraps_gpvae_obs3_classification_subgroup)
print(standards_deviations_gpvae_obs3_classification_subgroup)


[np.float64(0.27555636829615554), np.float64(0.4163626611692036), np.float64(0.9744375087984748), np.float64(0.6247616893293524), np.float64(0.9544883898988004), np.float64(0.42545585027276434), np.float64(0.19481102984534043)]
[np.float64(0.04419587100126069), np.float64(0.17489012918527883), np.float64(0.5599887970201437), np.float64(0.0892405810214106), np.float64(0.2945611621662042), np.float64(0.22617651193417881), np.float64(0.005944216965552449)]


In [ ]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

[np.float64(0.19414765647825802), np.float64(0.20029027714924186), np.float64(0.0), np.float64(0.46471271158064226), np.float64(0.44761681301701794), np.float64(0.16288975410409948), np.float64(0.18330931707311895)]
[np.float64(0.3671560473110876), np.float64(0.8326562128898994), np.float64(2.008021449480889), np.float64(0.8060316611288288), np.float64(1.5891190064067173), np.float64(0.9467590081779655), np.float64(0.20667332301290167)]


In [84]:
mean_values_ci_gpvae_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup)

print(mean_values_ci_gpvae_obs3_classification_subgroup)

[np.float64(0.2806518518946728), np.float64(0.5164732450195706), np.float64(1.0040107247404444), np.float64(0.6353721863547356), np.float64(1.0183679097118676), np.float64(0.5548243811410325), np.float64(0.1949913200430103)]


## MRNN

### Calculate MAE and AE

In [74]:
testing_mae_mrnn_variables_standard, testing_ae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for MRNN model

#### Difference between gender group

In [379]:
diff_mrnn_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 1, 2, variables)
diff_mrnn_gender_subgroup

[('Cholesterol', 0.460074799665683),
 ('SaO2', 0.23089926175071557),
 ('Weight', 0.17818464548563684),
 ('AST', 0.17410829776911274),
 ('PaCO2', 0.1296286552649416)]

#### Difference between age group

In [380]:
diff_mrnn_age_subgroup = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 4, 5, variables)
diff_mrnn_age_subgroup

[('Cholesterol', 0.5264706084970757),
 ('ALP', 0.4211803944022994),
 ('SaO2', 0.2121897999027318),
 ('TroponinI', 0.1882160203831248),
 ('Bilirubin', 0.14052047553207314)]

#### Difference between ICUType group

In [381]:
diff_mrnn_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 7, variables)
diff_mrnn_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 8, variables)
diff_mrnn_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 9, variables)
diff_mrnn_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 7, 8, variables)
diff_mrnn_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 7, 9, variables)
diff_mrnn_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 8, 9, variables)

In [382]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_mrnn_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_mrnn_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_mrnn_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.901031992086054), ('TroponinI', 0.8885610682206686), ('TroponinT', 0.2790266120032721), ('AST', 0.2552394500943449), ('HR', 0.2447531820372948)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 1.126906437286816), ('ALP', 0.8268337264809685), ('TroponinI', 0.6275078282113778), ('PaCO2', 0.31668859057403287), ('K', 0.246514642989757)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.47523007256967253), ('TroponinT', 0.28805061730511766), ('Albumin', 0.20611631660536978), ('Glucose', 0.20571705278816577), ('Cholesterol', 0.20496775474055506)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.3273285735353169), ('Cholesterol', 0.7254168125829273), ('ALP', 0.6559594642969739), ('HCO3', 0.44461402558334706), ('PaCO2', 0.4402514604811403)]
----------------------------
Diferença(MAE) ICUType 2 e 

In [383]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[24])

Top 5 variáveis ICUType
-----------------------
SaO2
Cholesterol
TroponinI
ALP
PaCO2


#### Difference between BMI group

In [384]:
diff_mrnn_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 11, variables)
diff_mrnn_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 12, variables)
diff_mrnn_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 13, variables)
diff_mrnn_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 14, variables)
diff_mrnn_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 15, variables)
diff_mrnn_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 16, variables)

diff_mrnn_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 12, variables)
diff_mrnn_classification_low_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 13, variables)
diff_mrnn_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 14, variables)
diff_mrnn_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 15, variables)
diff_mrnn_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 16, variables)

diff_mrnn_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 13, variables)
diff_mrnn_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 14, variables)
diff_mrnn_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 15, variables)
diff_mrnn_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 16, variables)

diff_mrnn_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 14, variables)
diff_mrnn_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 15, variables)
diff_mrnn_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 16, variables)


diff_mrnn_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 14, 15, variables)
diff_mrnn_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 14, 16, variables)

diff_mrnn_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 15, 16, variables)

In [385]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_mrnn_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_mrnn_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_mrnn_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_mrnn_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_mrnn_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_mrnn_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.7276133849206046), ('Bilirubin', 0.991431300323627), ('TroponinI', 0.760530696557348), ('Weight', 0.5541701206564827), ('Cholesterol', 0.4686585717311543)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 0.3793471669959604), ('PaCO2', 0.2607229460061936), ('SaO2', 0.23204288750553248), ('TroponinI', 0.19152385401179972), ('HCO3', 0.18600797150566883)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.73118655525492), ('TroponinI', 0.4223997426958679), ('Weight', 0.3506808770526928), ('PaCO2', 0.227091647825451), ('SaO2', 0.21374411070461574)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.885811109436305), ('PaCO2', 0.2961064251653872), ('HCO3', 0.2589322197694509), ('SaO2', 0.25878672579612844), ('Albumin',

In [386]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_mrnn_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_mrnn_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_mrnn_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_mrnn_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_mrnn_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.6166555970496364), ('Bilirubin', 0.9207620408895874), ('Cholesterol', 0.8480057387271147), ('TroponinI', 0.5690068425455482), ('HCO3', 0.536785762597226)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.800473096976984), ('Cholesterol', 1.1998451269860744), ('TroponinI', 1.1829304392532158), ('Bilirubin', 1.1149758410790835), ('Weight', 0.9048509977091755)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.8393424639406553), ('Bilirubin', 1.1198139288037439), ('Ph', 0.8807265506509955), ('Weight', 0.7823811039519852), ('TroponinI', 0.6293802953922413)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.7016018916185272), ('Bilirubin', 1.0552999603327775), ('TroponinT', 0.9759888694076897), ('HCO3', 0.633904112234476), ('Weight', 0.5288315187720876

In [387]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_mrnn_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_mrnn_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_mrnn_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_mrnn_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('TroponinI', 0.6139235967076676), ('Weight', 0.39083485382314354), ('Cholesterol', 0.35183938825895966), ('ALT', 0.25331368596067866), ('Bilirubin', 0.1942138001894962)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8908243808825946), ('Cholesterol', 0.3310031978642243), ('Weight', 0.26836496006595323), ('ALP', 0.22268686689101885), ('Albumin', 0.19923420837815708)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 0.9827645651094227), ('Cholesterol', 0.8717828192132384), ('TroponinI', 0.5690068425455482), ('ALT', 0.28712642112221703), ('Creatinine', 0.20622605478110573)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('ALT', 1.098237131073044), ('Cholesterol', 0.953380515668881), ('Weight', 0.8645236401093375), ('TroponinT', 0.5796770841475586), 

In [388]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_mrnn_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_mrnn_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_mrnn_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Ph', 0.8888769180304672), ('Cholesterol', 0.6828425861231839), ('TroponinI', 0.5535501438609746), ('ALT', 0.23397502450091273), ('TroponinT', 0.23237983370993043)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('Cholesterol', 1.223622207472198), ('TroponinI', 1.1829304392532158), ('TroponinT', 0.8424714490327402), ('Weight', 0.376019478937088), ('RespRate', 0.20419276783614382)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.3052199039278407), ('Weight', 1.255358493932481), ('ALT', 0.8449234451123655), ('TroponinI', 0.8326977075447959), ('TroponinT', 0.4393839680708761)]
----------------------------


In [389]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_mrnn_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_mrnn_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.0748512827426706), ('Ph', 0.8644289969648554), ('TroponinI', 0.6293802953922413), ('Cholesterol', 0.5407796213490141), ('ALT', 0.2677877596624511)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Weight', 1.1328886001752907), ('ALT', 1.0788984696132782), ('Ph', 0.8845028557366159), ('TroponinT', 0.6717638017808065), ('Cholesterol', 0.6223773178046568)]
----------------------------


In [390]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_mrnn_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('Weight', 0.879339014995393), ('ALT', 0.8111107099508271), ('TroponinT', 0.4030874809618641), ('HCO3', 0.4017804085595853), ('TroponinI', 0.35023273170841995)]
----------------------------


In [391]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[6])
print(variables[0])
print(variables[31])
print(variables[35])
print(variables[32])

Top 5 variáveis BMI
-----------------------
Cholesterol
ALP
TroponinI
Weight
TroponinT


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [392]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_mrnn_female = []

for i in indices:
       results_bootstrap_mrnn_female.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [393]:
means_bootstraps_mrnn_female,standards_deviations_mrnn_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_female)

print(means_bootstraps_mrnn_female)
print(standards_deviations_mrnn_female)


[np.float64(1.2935755664536321), np.float64(0.6164434389513259), np.float64(0.6773026444534478), np.float64(0.6870786909727755), np.float64(0.5948582802438828), np.float64(0.5839452342568027), np.float64(0.5567168169671148), np.float64(0.7504470639370755), np.float64(0.5741275830245283), np.float64(0.5866013448369327), np.float64(0.8422595413325532), np.float64(0.8398711298558051)]
[np.float64(0.3348201581771101), np.float64(0.06944650222593989), np.float64(0.14478900510714163), np.float64(0.03417395759481338), np.float64(0.009064407946767905), np.float64(0.08534796815501383), np.float64(0.022372073254679395), np.float64(0.036103980569769566), np.float64(0.05611758444016825), np.float64(0.08618076338596993), np.float64(0.011422593458141908), np.float64(0.03226758107662835)]


#### Calculating lower bound and upper bound

In [394]:
lower_bounds_mrnn_female, upper_bounds_mrnn_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_female, means_bootstraps_mrnn_female,standards_deviations_mrnn_female)

print(lower_bounds_mrnn_female)
print(upper_bounds_mrnn_female)

[np.float64(0.6373401151284923), np.float64(0.4803307957362031), np.float64(0.39352140908606426), np.float64(0.6200989648777423), np.float64(0.577092367127039), np.float64(0.41666629051930415), np.float64(0.5128683591284514), np.float64(0.6796845623217933), np.float64(0.4641391386224132), np.float64(0.41769015244026353), np.float64(0.8198716695445523), np.float64(0.7766278330773874)]
[np.float64(1.949811017778772), np.float64(0.7525560821664488), np.float64(0.9610838798208313), np.float64(0.7540584170678087), np.float64(0.6126241933607265), np.float64(0.7512241779943012), np.float64(0.6005652748057783), np.float64(0.8212095655523577), np.float64(0.6841160274266433), np.float64(0.7555125372336019), np.float64(0.8646474131205542), np.float64(0.9031144266342229)]


In [395]:
lower_bounds_mrnn_female, upper_bounds_mrnn_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_female)

print(lower_bounds_mrnn_female)
print(upper_bounds_mrnn_female)

[np.float64(0.6126520319937722), np.float64(0.47409563271282296), np.float64(0.4323250157365883), np.float64(0.6222426612168049), np.float64(0.5771798590122321), np.float64(0.4437001845782444), np.float64(0.5157225624973552), np.float64(0.6822838454699156), np.float64(0.4784126106859384), np.float64(0.44340788766262024), np.float64(0.8199168501473644), np.float64(0.7781097414786914)]
[np.float64(2.0159011400360725), np.float64(0.7487762509348388), np.float64(0.9954092308364325), np.float64(0.758028411381523), np.float64(0.6128677582362574), np.float64(0.7729790550788368), np.float64(0.6038603980359288), np.float64(0.8237995923141855), np.float64(0.6974089157890936), np.float64(0.7743290907046212), np.float64(0.8651446658608004), np.float64(0.9041361461481169)]


#### Mean values of lower bound and upper bound


In [396]:
mean_values_ci_mrnn_female = toolkits.calc_mean_values_ci(lower_bounds_mrnn_female, upper_bounds_mrnn_female)

print(mean_values_ci_mrnn_female)

[np.float64(1.3142765860149224), np.float64(0.6114359418238309), np.float64(0.7138671232865104), np.float64(0.6901355362991639), np.float64(0.5950238086242448), np.float64(0.6083396198285406), np.float64(0.5597914802666419), np.float64(0.7530417188920506), np.float64(0.587910763237516), np.float64(0.6088684891836207), np.float64(0.8425307580040824), np.float64(0.8411229438134041)]


### Confidence Interval (General)(variable)

#### Distribution Bootstrap per variable

In [32]:
results_bootstrap_mrnn_general = []

for i in range(len(variables)):
       results_bootstrap_mrnn_general.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 0, i, 9000))

#### Calculating lower bound and upper bound

In [33]:
lower_bounds_mrnn_general, upper_bounds_mrnn_general = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_general)

print(lower_bounds_mrnn_general)
print(upper_bounds_mrnn_general)

[np.float64(0.465647881384931), np.float64(0.4098742560247144), np.float64(0.4628455448174591), np.float64(0.6429093333732719), np.float64(0.6413466522071308), np.float64(0.5049983752809665), np.float64(0.6690266243085883), np.float64(0.5743647097914867), np.float64(0.6605854660481888), np.float64(0.793250463035465), np.float64(0.8216029559185332), np.float64(0.6012753452050034), np.float64(0.6999832034384326), np.float64(0.7456455841579578), np.float64(0.7848713155459872), np.float64(0.6968839131318737), np.float64(0.6652487519370544), np.float64(0.5885648206475895), np.float64(0.11063587247608093), np.float64(0.5910855764218137), np.float64(0.6626921700977987), np.float64(0.6176256543009411), np.float64(0.7007284484859146), np.float64(0.6699547151691615), np.float64(0.6913894388579308), np.float64(0.7391842763585581), np.float64(0.723451758589465), np.float64(0.7835935487759038), np.float64(0.35574677147497863), np.float64(0.6953940718629048), np.float64(0.5020880008313511), np.float

#### Mean values of lower bound and upper bound


In [34]:
mean_values_ci_mrnn_general = toolkits.calc_mean_values_ci(lower_bounds_mrnn_general, upper_bounds_mrnn_general)

print(mean_values_ci_mrnn_general)

[np.float64(0.6338232541766491), np.float64(0.5152443059521685), np.float64(0.6053927014400157), np.float64(0.723952578143267), np.float64(0.6760705094101654), np.float64(0.6039486679976045), np.float64(0.9316277025006986), np.float64(0.6213450094765118), np.float64(0.674775610599583), np.float64(0.8204657616144571), np.float64(0.8382466158097566), np.float64(0.6449009647749766), np.float64(0.7455523212956321), np.float64(0.7805123381007555), np.float64(0.796371567245254), np.float64(0.7401858780524548), np.float64(0.7148992650103607), np.float64(0.6023836233882192), np.float64(0.11193191178785336), np.float64(0.651290202845989), np.float64(0.677857914259079), np.float64(0.632263072657384), np.float64(0.7166239835416021), np.float64(0.7130135135035587), np.float64(0.7295331076584102), np.float64(0.7733025153484592), np.float64(0.7662224429395088), np.float64(0.8093037258489179), np.float64(0.42614903189694553), np.float64(0.7101102215540709), np.float64(0.520413058138714), np.float64(0

### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [397]:
indices = [6, 31, 28, 16, 33, 2, 7, 15, 5, 2, 35, 24]

results_bootstrap_mrnn_male = []

for i in indices:
       results_bootstrap_mrnn_male.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [398]:
means_bootstraps_mrnn_male,standards_deviations_mrnn_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_male)

print(means_bootstraps_mrnn_male)
print(standards_deviations_mrnn_male)


[np.float64(0.843192574716056), np.float64(0.668568255170694), np.float64(0.4492868636799215), np.float64(0.7338100541214645), np.float64(0.637105204616174), np.float64(0.41058765459051233), np.float64(0.6407296710635678), np.float64(0.6907439871649839), np.float64(0.5063212250506773), np.float64(0.41077536732311426), np.float64(0.6641424102935869), np.float64(0.7097950356298286)]
[np.float64(0.17814377509333526), np.float64(0.10364648396448216), np.float64(0.0364612662497388), np.float64(0.03652193207443982), np.float64(0.012651710373960564), np.float64(0.017616312301926715), np.float64(0.038040418694449706), np.float64(0.02679927511702984), np.float64(0.03590502969836963), np.float64(0.017568824787085713), np.float64(0.011397749869035587), np.float64(0.026005992094796697)]


#### Calculating lower bound and upper bound

In [399]:
lower_bounds_mrnn_male, upper_bounds_mrnn_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_male, means_bootstraps_mrnn_male,standards_deviations_mrnn_male)

print(lower_bounds_mrnn_male)
print(upper_bounds_mrnn_male)

[np.float64(0.4940371914631154), np.float64(0.4654248794761007), np.float64(0.37782409499970765), np.float64(0.6622283826097443), np.float64(0.6123083079403796), np.float64(0.3760603169383261), np.float64(0.5661718204656222), np.float64(0.6382183731238249), np.float64(0.43594865997803184), np.float64(0.3763411034897317), np.float64(0.641803231045481), np.float64(0.6588242277417937)]
[np.float64(1.1923479579689966), np.float64(0.8717116308652872), np.float64(0.5207496323601354), np.float64(0.8053917256331847), np.float64(0.6619021012919685), np.float64(0.44511499224269857), np.float64(0.7152875216615134), np.float64(0.7432696012061428), np.float64(0.5766937901233229), np.float64(0.4452096311564968), np.float64(0.6864815895416928), np.float64(0.7607658435178635)]


In [400]:
lower_bounds_mrnn_male, upper_bounds_mrnn_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_male)

print(lower_bounds_mrnn_male)
print(upper_bounds_mrnn_male)

[np.float64(0.4991894056308096), np.float64(0.46620400437360143), np.float64(0.383539541408955), np.float64(0.6650980650926085), np.float64(0.6135516384217858), np.float64(0.37855526823129615), np.float64(0.5707762780527199), np.float64(0.6387407497127994), np.float64(0.44272037855640506), np.float64(0.37872429283426834), np.float64(0.6421288395215133), np.float64(0.6604648874871495)]
[np.float64(1.2081729248187498), np.float64(0.8610184344452578), np.float64(0.5256279646644721), np.float64(0.8086929820598046), np.float64(0.6629398481038187), np.float64(0.4476879639204266), np.float64(0.7198204605938501), np.float64(0.7437561968936302), np.float64(0.5847929971006726), np.float64(0.4468672984106811), np.float64(0.6867581547311782), np.float64(0.7636419167633988)]


#### Mean values of lower bound and upper bound


In [401]:
mean_values_ci_mrnn_male = toolkits.calc_mean_values_ci(lower_bounds_mrnn_male, upper_bounds_mrnn_male)

print(mean_values_ci_mrnn_male)

[np.float64(0.8536811652247798), np.float64(0.6636112194094297), np.float64(0.4545837530367136), np.float64(0.7368955235762065), np.float64(0.6382457432628023), np.float64(0.4131216160758614), np.float64(0.645298369323285), np.float64(0.6912484733032147), np.float64(0.5137566878285388), np.float64(0.4127957956224747), np.float64(0.6644434971263458), np.float64(0.7120534021252741)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [402]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [403]:
means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup,standards_deviations_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup)
print(standards_deviations_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.6685860079224264), np.float64(0.5272663685400262), np.float64(0.7695863874270902), np.float64(0.44155316627573615), np.float64(0.48557998905047123), np.float64(0.934827689578139), np.float64(0.6820614395117011)]
[np.float64(0.10018205121265905), np.float64(0.05013941941002657), np.float64(0.13086064525354313), np.float64(0.036120230106595166), np.float64(0.01922782751940422), np.float64(0.2804046337041767), np.float64(0.06877446561494749)]


#### Calculating lower bound and upper bound


In [404]:
lower_bounds_mrnn_more_than_or_equal_to_65_subgroup, upper_bounds_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup, means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup,standards_deviations_mrnn_more_than_or_equal_to_65_subgroup)

print(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup)
print(upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)


[np.float64(0.47223279564826737), np.float64(0.4289949122906256), np.float64(0.5131042357364732), np.float64(0.37075881615351025), np.float64(0.4478941396114908), np.float64(0.38524470641980657), np.float64(0.5472659638504156)]
[np.float64(0.8649392201965854), np.float64(0.6255378247894268), np.float64(1.0260685391177071), np.float64(0.512347516397962), np.float64(0.5232658384894516), np.float64(1.4844106727364714), np.float64(0.8168569151729865)]


In [405]:
lower_bounds_mrnn_more_than_or_equal_to_65_subgroup, upper_bounds_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup)

print(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup)
print(upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.4699906828380936), np.float64(0.44279438851118996), np.float64(0.5418240742184548), np.float64(0.3749656354219161), np.float64(0.44722095873791734), np.float64(0.4724457637819884), np.float64(0.5513685298243582)]
[np.float64(0.8653160622719036), np.float64(0.6373761340725912), np.float64(1.0537990044224066), np.float64(0.5154765764513622), np.float64(0.5230084420416287), np.float64(1.5622426368143583), np.float64(0.8202436064727875)]


#### Mean values of lower bound and upper bound

In [406]:
mean_values_ci_mrnn_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup , upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.6676533725549986), np.float64(0.5400852612918906), np.float64(0.7978115393204307), np.float64(0.44522110593663916), np.float64(0.485114700389773), np.float64(1.0173442002981734), np.float64(0.6858060681485729)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [407]:
indices = [6, 32, 0, 28, 5, 31, 3]

results_bootstrap_mrnn_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_mrnn_less_than_65_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [408]:
means_bootstraps_mrnn_less_than_65_subgroup,standards_deviations_mrnn_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_less_than_65_subgroup)

print(means_bootstraps_mrnn_less_than_65_subgroup)
print(standards_deviations_mrnn_less_than_65_subgroup)


[np.float64(1.0125284495848363), np.float64(0.48691819213338233), np.float64(1.1932477112152653), np.float64(0.6511956583092404), np.float64(0.6256062744386217), np.float64(0.7499703450677268), np.float64(0.7791244196185086)]
[np.float64(0.8325073052733275), np.float64(0.054879287380068906), np.float64(0.4307696183017506), np.float64(0.13903843147349368), np.float64(0.07340469975151238), np.float64(0.056076356745105624), np.float64(0.061780433705196015)]


#### Calculating lower bound and upper bound

In [409]:
lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_less_than_65_subgroup, means_bootstraps_mrnn_less_than_65_subgroup, standards_deviations_mrnn_less_than_65_subgroup)

print(lower_bounds_mrnn_less_than_65_subgroup)
print(upper_bounds_mrnn_less_than_65_subgroup)

[np.float64(-0.6191558856173778), np.float64(0.37935676537122376), np.float64(0.348954773709768), np.float64(0.3786853401542525), np.float64(0.4817357066296812), np.float64(0.6400627054631), np.float64(0.65803699460706)]
[np.float64(2.6442127847870505), np.float64(0.5944796188955409), np.float64(2.0375406487207623), np.float64(0.9237059764642284), np.float64(0.7694768422475622), np.float64(0.8598779846723535), np.float64(0.9002118446299573)]


In [410]:
lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_less_than_65_subgroup)

print(lower_bounds_mrnn_less_than_65_subgroup)
print(upper_bounds_mrnn_less_than_65_subgroup)

[np.float64(0.0), np.float64(0.3920449890748008), np.float64(0.5358780023016734), np.float64(0.4153319586478943), np.float64(0.49591883952098054), np.float64(0.6242850411015622), np.float64(0.6601411430538056)]
[np.float64(2.2237061316119653), np.float64(0.6075882032367239), np.float64(2.1480564707077767), np.float64(0.9573257099336568), np.float64(0.783888908685614), np.float64(0.8406452405450803), np.float64(0.902299627619846)]


#### Mean values of lower bound and upper bound

In [411]:
mean_values_ci_mrnn_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup)

print(mean_values_ci_mrnn_less_than_65_subgroup)

[np.float64(1.1118530658059826), np.float64(0.49981659615576235), np.float64(1.341967236504725), np.float64(0.6863288342907756), np.float64(0.6399038741032973), np.float64(0.7324651408233213), np.float64(0.7812203853368258)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [412]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_mrnn_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_1_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [413]:
means_bootstraps_mrnn_ICUType_1_subgroup, standards_deviations_mrnn_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_1_subgroup)

print(means_bootstraps_mrnn_ICUType_1_subgroup)
print(standards_deviations_mrnn_ICUType_1_subgroup)


[np.float64(0.5943122351342515), np.float64(1.296183923906161), np.float64(0.90119808488499), np.float64(0.3888843189711193), np.float64(0.7059273759600687), np.float64(0.7213178588104578)]
[np.float64(0.18869648636668113), np.float64(0.3713588540235834), np.float64(0.24203709860376785), np.float64(0.04280483855469153), np.float64(0.11865358177680002), np.float64(0.053431654944320076)]


#### Calculating lower bound and upper bound

In [414]:
lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_1_subgroup, means_bootstraps_mrnn_ICUType_1_subgroup, standards_deviations_mrnn_ICUType_1_subgroup)

print(lower_bounds_mrnn_ICUType_1_subgroup)
print(upper_bounds_mrnn_ICUType_1_subgroup)

[np.float64(0.22447391784630316), np.float64(0.5683339446798702), np.float64(0.4268140886990352), np.float64(0.30498837703987236), np.float64(0.4733706290408626), np.float64(0.616593739485219)]
[np.float64(0.9641505524221998), np.float64(2.0240339031324517), np.float64(1.3755820810709447), np.float64(0.47278026090236625), np.float64(0.9384841228792749), np.float64(0.8260419781356967)]


In [415]:
lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_1_subgroup)

print(lower_bounds_mrnn_ICUType_1_subgroup)
print(upper_bounds_mrnn_ICUType_1_subgroup)

[np.float64(0.3331194822164736), np.float64(0.6397649554169416), np.float64(0.47440347159410723), np.float64(0.30344414263831104), np.float64(0.49718343591820835), np.float64(0.6194561846042215)]
[np.float64(1.0307236660324435), np.float64(2.0993957435220927), np.float64(1.4194438922686403), np.float64(0.4732330650906346), np.float64(0.9579007344805754), np.float64(0.8292213870649836)]


#### Mean values of lower bound and upper bound

In [416]:
mean_values_ci_mrnn_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup)

print(mean_values_ci_mrnn_ICUType_1_subgroup)

[np.float64(0.6819215741244585), np.float64(1.3695803494695171), np.float64(0.9469236819313738), np.float64(0.38833860386447283), np.float64(0.7275420851993919), np.float64(0.7243387858346025)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [417]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_mrnn_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_2_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [418]:
means_bootstraps_mrnn_ICUType_2_subgroup, standards_deviations_mrnn_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_2_subgroup)

print(means_bootstraps_mrnn_ICUType_2_subgroup)
print(standards_deviations_mrnn_ICUType_2_subgroup) 


[np.float64(0.3916590391244182), np.float64(0.4072828872551072), np.float64(0.0), np.float64(0.5591322006578231), np.float64(0.425918563477567), np.float64(0.598082443704859)]
[np.float64(0.025942388243406177), np.float64(0.1335303132323623), np.float64(0.0), np.float64(0.021966787016863255), np.float64(0.03607830122645071), np.float64(0.022204161183700815)]


#### Calculating lower bound and upper bound

In [419]:
lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_2_subgroup, means_bootstraps_mrnn_ICUType_2_subgroup, standards_deviations_mrnn_ICUType_2_subgroup)

print(lower_bounds_mrnn_ICUType_2_subgroup)
print(upper_bounds_mrnn_ICUType_2_subgroup)

[np.float64(0.34081289249438673), np.float64(0.14556828247532488), np.float64(0.0), np.float64(0.516078089248709), np.float64(0.35520639245033636), np.float64(0.5545630874778831)]
[np.float64(0.44250518575444964), np.float64(0.6689974920348896), np.float64(0.0), np.float64(0.6021863120669372), np.float64(0.49663073450479767), np.float64(0.6416017999318349)]


In [420]:
lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_2_subgroup)

print(lower_bounds_mrnn_ICUType_2_subgroup)
print(upper_bounds_mrnn_ICUType_2_subgroup)

[np.float64(0.343457425628546), np.float64(0.10866928622836351), np.float64(0.0), np.float64(0.5143712942416259), np.float64(0.35884361062116316), np.float64(0.5550634735484589)]
[np.float64(0.44405091119660894), np.float64(0.6781899676363381), np.float64(0.0), np.float64(0.6006455414289316), np.float64(0.4985700143566348), np.float64(0.6417498219281579)]


#### Mean values of lower bound and upper bound

In [421]:
mean_values_ci_mrnn_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup)

print(mean_values_ci_mrnn_ICUType_2_subgroup)

[np.float64(0.3937541684125775), np.float64(0.3934296269323508), np.float64(0.0), np.float64(0.5575084178352787), np.float64(0.42870681248889897), np.float64(0.5984066477383083)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [422]:
indices = [28, 31, 6, 0, 32, 24]

results_bootstrap_mrnn_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_3_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [423]:
means_bootstraps_mrnn_ICUType_3_subgroup, standards_deviations_mrnn_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_3_subgroup)

print(means_bootstraps_mrnn_ICUType_3_subgroup)
print(standards_deviations_mrnn_ICUType_3_subgroup)


[np.float64(1.7201421783191229), np.float64(0.6712263138326279), np.float64(0.7099729443571801), np.float64(1.2177127705892345), np.float64(0.5233024478153722), np.float64(1.0373984783765282)]
[np.float64(0.4633041168516057), np.float64(0.07452755651443149), np.float64(0.2467238749310891), np.float64(0.44616363311206336), np.float64(0.02427799239644668), np.float64(0.05240905524160113)]


#### Calculating lower bound and upper bound

In [424]:
lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_3_subgroup, means_bootstraps_mrnn_ICUType_3_subgroup, standards_deviations_mrnn_ICUType_3_subgroup)

print(lower_bounds_mrnn_ICUType_3_subgroup)
print(upper_bounds_mrnn_ICUType_3_subgroup)

[np.float64(0.8120827954008389), np.float64(0.5251549872085687), np.float64(0.2264030353660807), np.float64(0.34324811847804804), np.float64(0.4757184571013994), np.float64(0.9346786176392199)]
[np.float64(2.6282015612374066), np.float64(0.8172976404566871), np.float64(1.1935428533482795), np.float64(2.092177422700421), np.float64(0.570886438529345), np.float64(1.1401183391138365)]


In [425]:
lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_3_subgroup)

print(lower_bounds_mrnn_ICUType_3_subgroup)
print(upper_bounds_mrnn_ICUType_3_subgroup)

[np.float64(0.9013282987761839), np.float64(0.5042459853271846), np.float64(0.07943431525266052), np.float64(0.523540281662038), np.float64(0.47710069473278066), np.float64(0.9355301158593502)]
[np.float64(2.7268963440548806), np.float64(0.7919135003794507), np.float64(1.1458615561271466), np.float64(2.201580622231096), np.float64(0.5720822825126277), np.float64(1.1436354969336824)]


#### Mean values of lower bound and upper bound

In [426]:
mean_values_ci_mrnn_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup)

print(mean_values_ci_mrnn_ICUType_3_subgroup)

[np.float64(1.8141123214155321), np.float64(0.6480797428533176), np.float64(0.6126479356899035), np.float64(1.362560451946567), np.float64(0.5245914886227042), np.float64(1.0395828063965162)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [427]:
indices = [28, 31, 6, 0, 32, 24]


results_bootstrap_mrnn_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_4_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [428]:
means_bootstraps_mrnn_ICUType_4_subgroup, standards_deviations_mrnn_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_4_subgroup)

print(means_bootstraps_mrnn_ICUType_4_subgroup)
print(standards_deviations_mrnn_ICUType_4_subgroup)


[np.float64(0.5129584753292196), np.float64(0.8049779680766729), np.float64(0.6932415830956783), np.float64(0.5925069675680987), np.float64(0.41812632517087545), np.float64(0.67093157463133)]
[np.float64(0.10738153608918637), np.float64(0.18972691290936852), np.float64(0.2824699624350711), np.float64(0.06466034510293105), np.float64(0.038938005126450716), np.float64(0.02695168857050118)]


#### Calculating lower bound and upper bound

In [429]:
lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_4_subgroup, means_bootstraps_mrnn_ICUType_4_subgroup, standards_deviations_mrnn_ICUType_4_subgroup)

print(lower_bounds_mrnn_ICUType_4_subgroup)
print(upper_bounds_mrnn_ICUType_4_subgroup)

[np.float64(0.30249453198982623), np.float64(0.4331200518763431), np.float64(0.1396106300085569), np.float64(0.465775019938423), np.float64(0.341809237493196), np.float64(0.6181072357106079)]
[np.float64(0.7234224186686129), np.float64(1.1768358842770026), np.float64(1.2468725361827997), np.float64(0.7192389151977745), np.float64(0.49444341284855486), np.float64(0.7237559135520522)]


In [430]:
lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_4_subgroup)

print(lower_bounds_mrnn_ICUType_4_subgroup)
print(upper_bounds_mrnn_ICUType_4_subgroup)

[np.float64(0.33563338105917867), np.float64(0.5495089491374031), np.float64(0.24243039744972775), np.float64(0.47460904439599005), np.float64(0.33886485296293595), np.float64(0.6189131433405981)]
[np.float64(0.7545120382427323), np.float64(1.266807378181786), np.float64(1.3201168843565492), np.float64(0.726770121225473), np.float64(0.49087161449270034), np.float64(0.7253197030826162)]


#### Mean values of lower bound and upper bound

In [431]:
mean_values_ci_mrnn_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup)

print(mean_values_ci_mrnn_ICUType_4_subgroup)

[np.float64(0.5450727096509554), np.float64(0.9081581636595946), np.float64(0.7812736409031384), np.float64(0.6006895828107315), np.float64(0.4148682337278181), np.float64(0.6721164232116071)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [432]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_undefined_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [433]:
means_bootstraps_mrnn_undefined_classification_subgroup, standards_deviations_mrnn_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_undefined_classification_subgroup)

print(means_bootstraps_mrnn_undefined_classification_subgroup)
print(standards_deviations_mrnn_undefined_classification_subgroup)


[np.float64(0.4834588311090102), np.float64(0.5338851054050414), np.float64(0.5737546291785596), np.float64(0.7572050543606113), np.float64(0.6348404046401104), np.float64(0.5221624689701393), np.float64(0.7690207928427124)]
[np.float64(0.035947440034432995), np.float64(0.04796417009426909), np.float64(0.15997792293161645), np.float64(0.12243525026512263), np.float64(0.11238341168204886), np.float64(0.09257063040519851), np.float64(0.01348167406637216)]


#### Calculating lower bound and upper bound

In [434]:
lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_undefined_classification_subgroup, means_bootstraps_mrnn_undefined_classification_subgroup, standards_deviations_mrnn_undefined_classification_subgroup)

print(lower_bounds_mrnn_undefined_classification_subgroup)
print(upper_bounds_mrnn_undefined_classification_subgroup)

[np.float64(0.4130031433051083), np.float64(0.43987705947192085), np.float64(0.2602036619110669), np.float64(0.5172363734028228), np.float64(0.41457296528355664), np.float64(0.3407273673497817), np.float64(0.7425971972213153)]
[np.float64(0.5539145189129122), np.float64(0.6278931513381619), np.float64(0.8873055964460522), np.float64(0.9971737353183997), np.float64(0.8551078439966642), np.float64(0.7035975705904969), np.float64(0.7954443884641095)]


In [435]:
lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_undefined_classification_subgroup)

print(lower_bounds_mrnn_undefined_classification_subgroup)
print(upper_bounds_mrnn_undefined_classification_subgroup)

[np.float64(0.4263038813346761), np.float64(0.4470550187520496), np.float64(0.2800241680793975), np.float64(0.5133303592341026), np.float64(0.46736221029321007), np.float64(0.39054171908927215), np.float64(0.7430997496288949)]
[np.float64(0.5643525118998892), np.float64(0.6341258928299868), np.float64(0.9176472282782503), np.float64(0.9896050981839731), np.float64(0.8866372098312039), np.float64(0.7372734132646498), np.float64(0.7957462828589899)]


#### Mean values of lower bound and upper bound

In [436]:
mean_values_ci_mrnn_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup)

print(mean_values_ci_mrnn_undefined_classification_subgroup)

[np.float64(0.49532819661728267), np.float64(0.5405904557910182), np.float64(0.5988356981788239), np.float64(0.7514677287090379), np.float64(0.6769997100622069), np.float64(0.563907566176961), np.float64(0.7694230162439424)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [437]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_low_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [438]:
means_bootstraps_mrnn_low_weight_classification_subgroup, standards_deviations_mrnn_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_low_weight_classification_subgroup)

print(means_bootstraps_mrnn_low_weight_classification_subgroup)
print(standards_deviations_mrnn_low_weight_classification_subgroup)


[np.float64(0.2742149593914782), np.float64(1.9459670267182232), np.float64(0.06753352371113829), np.float64(0.0), np.float64(1.6205607563381432), np.float64(0.43983313740810515), np.float64(1.3235392557014443)]
[np.float64(0.20929567175877506), np.float64(1.4741325422104914), np.float64(0.050552479388368335), np.float64(0.0), np.float64(1.1460472640197181), np.float64(0.05103390816270178), np.float64(0.05072339486945789)]


#### Calculating lower bound and upper bound

In [439]:
lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_low_weight_classification_subgroup, means_bootstraps_mrnn_low_weight_classification_subgroup, standards_deviations_mrnn_low_weight_classification_subgroup)

print(lower_bounds_mrnn_low_weight_classification_subgroup)
print(upper_bounds_mrnn_low_weight_classification_subgroup)

[np.float64(-0.1359970193758378), np.float64(-0.943279664452811), np.float64(-0.031547515219267075), np.float64(0.0), np.float64(-0.625650605721171), np.float64(0.33980851541888496), np.float64(1.2241232285837031)]
[np.float64(0.6844269381587942), np.float64(4.835213717889257), np.float64(0.16661456264154365), np.float64(0.0), np.float64(3.8667721183974573), np.float64(0.5398577593973253), np.float64(1.4229552828191854)]


In [440]:
lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_low_weight_classification_subgroup)

print(lower_bounds_mrnn_low_weight_classification_subgroup)
print(upper_bounds_mrnn_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.4383364553534848), np.float64(0.3575160340998334), np.float64(1.2209718070276419)]
[np.float64(0.4339607234964589), np.float64(4.032034724589697), np.float64(0.10537477694183658), np.float64(0.0), np.float64(4.250518118450101), np.float64(0.49951248070860665), np.float64(1.4225176222540652)]


#### Mean values of lower bound and upper bound

In [441]:
mean_values_ci_mrnn_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup)

print(mean_values_ci_mrnn_low_weight_classification_subgroup)

[np.float64(0.21698036174822946), np.float64(2.0160173622948485), np.float64(0.05268738847091829), np.float64(0.0), np.float64(2.344427286901793), np.float64(0.42851425740422), np.float64(1.3217447146408534)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [442]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_normal_weight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [443]:
means_bootstraps_mrnn_normal_weight_classification_subgroup, standards_deviations_mrnn_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_normal_weight_classification_subgroup)

print(means_bootstraps_mrnn_normal_weight_classification_subgroup)
print(standards_deviations_mrnn_normal_weight_classification_subgroup)


[np.float64(0.4263621519371738), np.float64(0.646458841174411), np.float64(0.8314887990910419), np.float64(0.5699137016310701), np.float64(0.705793005337667), np.float64(0.39751595968784265), np.float64(0.8094118709126088)]
[np.float64(0.04495926815170688), np.float64(0.10165939135263614), np.float64(0.38592353741859636), np.float64(0.10993289651872548), np.float64(0.16873074462490006), np.float64(0.026454342049380443), np.float64(0.013641674636157602)]


#### Calculating lower bound and upper bound

In [444]:
lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_normal_weight_classification_subgroup, means_bootstraps_mrnn_normal_weight_classification_subgroup, standards_deviations_mrnn_normal_weight_classification_subgroup)

print(lower_bounds_mrnn_normal_weight_classification_subgroup)
print(upper_bounds_mrnn_normal_weight_classification_subgroup)

[np.float64(0.33824360558854966), np.float64(0.4472100954329815), np.float64(0.07509256496429717), np.float64(0.3544491837381995), np.float64(0.3750868227882375), np.float64(0.34566640203635346), np.float64(0.7826746799369264)]
[np.float64(0.514480698285798), np.float64(0.8457075869158405), np.float64(1.5878850332177867), np.float64(0.7853782195239407), np.float64(1.0364991878870964), np.float64(0.44936551733933183), np.float64(0.8361490618882912)]


In [445]:
lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_normal_weight_classification_subgroup)

print(lower_bounds_mrnn_normal_weight_classification_subgroup)
print(upper_bounds_mrnn_normal_weight_classification_subgroup)

[np.float64(0.34290042744646426), np.float64(0.48431375038167185), np.float64(0.0), np.float64(0.351740472150374), np.float64(0.44627917028178504), np.float64(0.3421904179213616), np.float64(0.7825763113881157)]
[np.float64(0.5174341815635826), np.float64(0.8740742958220539), np.float64(1.2552017600298466), np.float64(0.7827012442511738), np.float64(1.0907357640575552), np.float64(0.44633988166304134), np.float64(0.8369819015799299)]


#### Mean values of lower bound and upper bound

In [446]:
mean_values_ci_mrnn_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup)

print(mean_values_ci_mrnn_normal_weight_classification_subgroup)

[np.float64(0.4301673045050234), np.float64(0.6791940231018628), np.float64(0.6276008800149233), np.float64(0.567220858200774), np.float64(0.7685074671696701), np.float64(0.3942651497922015), np.float64(0.8097791064840227)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [447]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_overweight_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [448]:
means_bootstraps_mrnn_overweight_classification_subgroup, standards_deviations_mrnn_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_overweight_classification_subgroup)

print(means_bootstraps_mrnn_overweight_classification_subgroup)
print(standards_deviations_mrnn_overweight_classification_subgroup)


[np.float64(0.5687296858733425), np.float64(0.46048855694070373), np.float64(1.2963966354597518), np.float64(1.1794300213081532), np.float64(0.5118163712538857), np.float64(0.6502465333656093), np.float64(0.4183955421148772)]
[np.float64(0.1424406140788189), np.float64(0.04704308504239986), np.float64(0.2550406601338593), np.float64(0.45394110874116217), np.float64(0.055238044963513054), np.float64(0.15567730445430072), np.float64(0.009740406739071887)]


#### Calculating lower bound and upper bound

In [449]:
lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_overweight_classification_subgroup, means_bootstraps_mrnn_overweight_classification_subgroup, standards_deviations_mrnn_overweight_classification_subgroup)

print(lower_bounds_mrnn_overweight_classification_subgroup)
print(upper_bounds_mrnn_overweight_classification_subgroup)

[np.float64(0.2895512123430885), np.float64(0.36828580453594506), np.float64(0.7965261270040673), np.float64(0.28972179707329504), np.float64(0.403551792548996), np.float64(0.34512462342490297), np.float64(0.39930469571152505)]
[np.float64(0.8479081594035964), np.float64(0.5526913093454624), np.float64(1.7962671439154363), np.float64(2.0691382455430114), np.float64(0.6200809499587754), np.float64(0.9553684433063157), np.float64(0.4374863885182293)]


In [450]:
lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_overweight_classification_subgroup)

print(lower_bounds_mrnn_overweight_classification_subgroup)
print(upper_bounds_mrnn_overweight_classification_subgroup)

[np.float64(0.35425209969664206), np.float64(0.37523398603048347), np.float64(0.7784886463716818), np.float64(0.6119497418486745), np.float64(0.4225049928694508), np.float64(0.39352243769994766), np.float64(0.3996233553463024)]
[np.float64(0.8917709256209447), np.float64(0.5587809122939364), np.float64(1.7541582786245802), np.float64(2.218829956583898), np.float64(0.6359906691208814), np.float64(0.9902873998296422), np.float64(0.43702309337471607)]


#### Mean values of lower bound and upper bound

In [451]:
mean_values_ci_mrnn_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup)

print(mean_values_ci_mrnn_overweight_classification_subgroup)

[np.float64(0.6230115126587934), np.float64(0.46700744916220993), np.float64(1.266323462498131), np.float64(1.4153898492162862), np.float64(0.5292478309951661), np.float64(0.691904918764795), np.float64(0.4183232243605092)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [452]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs1_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [453]:
means_bootstraps_mrnn_obs1_classification_subgroup, standards_deviations_mrnn_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs1_classification_subgroup)

print(means_bootstraps_mrnn_obs1_classification_subgroup)
print(standards_deviations_mrnn_obs1_classification_subgroup)


[np.float64(0.3351551287944784), np.float64(0.4229640533709728), np.float64(0.5906922871369863), np.float64(0.5427039503003247), np.float64(0.5069558136935141), np.float64(0.41724289027889583), np.float64(0.5410437616636001)]
[np.float64(0.051726865655468225), np.float64(0.04197461098231375), np.float64(0.3107552056173961), np.float64(0.27098536668459516), np.float64(0.09252744137522277), np.float64(0.04164542531375808), np.float64(0.01906373953681107)]


#### Calculating lower bound and upper bound

In [454]:
lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs1_classification_subgroup, means_bootstraps_mrnn_obs1_classification_subgroup, standards_deviations_mrnn_obs1_classification_subgroup)

print(lower_bounds_mrnn_obs1_classification_subgroup)
print(upper_bounds_mrnn_obs1_classification_subgroup)

[np.float64(0.23377233507661882), np.float64(0.3406953275805584), np.float64(-0.018376723881449175), np.float64(0.011582391261137914), np.float64(0.32560536101643617), np.float64(0.3356193565430773), np.float64(0.5036795187607981)]
[np.float64(0.436537922512338), np.float64(0.5052327791613872), np.float64(1.1997612981554218), np.float64(1.0738255093395115), np.float64(0.6883062663705919), np.float64(0.49886642401471437), np.float64(0.5784080045664022)]


In [455]:
lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs1_classification_subgroup)

print(lower_bounds_mrnn_obs1_classification_subgroup)
print(upper_bounds_mrnn_obs1_classification_subgroup)

[np.float64(0.23060395788572963), np.float64(0.33986682438817417), np.float64(0.0), np.float64(0.0), np.float64(0.3562788871155775), np.float64(0.3393535479416052), np.float64(0.5043022495098985)]
[np.float64(0.4336221147821313), np.float64(0.5037857473264438), np.float64(1.218276945248262), np.float64(0.8630806178451833), np.float64(0.7067325651109874), np.float64(0.5063421703125072), np.float64(0.578400848522646)]


#### Mean values of lower bound and upper bound

In [456]:
mean_values_ci_mrnn_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup)

print(mean_values_ci_mrnn_obs1_classification_subgroup)

[np.float64(0.33211303633393047), np.float64(0.42182628585730897), np.float64(0.609138472624131), np.float64(0.43154030892259165), np.float64(0.5315057261132824), np.float64(0.4228478591270562), np.float64(0.5413515490162722)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [457]:
indices = [32, 0, 6, 31, 5, 1, 35]
results_bootstrap_mrnn_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs2_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [458]:
means_bootstraps_mrnn_obs2_classification_subgroup, standards_deviations_mrnn_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs2_classification_subgroup)

print(means_bootstraps_mrnn_obs2_classification_subgroup)
print(standards_deviations_mrnn_obs2_classification_subgroup) 


[np.float64(1.3388923242492), np.float64(0.5607931709304527), np.float64(0.05189613494580122), np.float64(0.0), np.float64(0.5726815182613774), np.float64(0.6833419537173094), np.float64(0.7948349604749801)]
[np.float64(0.9082803746120481), np.float64(0.05168739719568244), np.float64(0.03926061950882545), np.float64(0.0), np.float64(0.06076866739687586), np.float64(0.24872512458782148), np.float64(0.04293914890512166)]


#### Calculating lower bound and upper bound

In [459]:
lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs2_classification_subgroup, means_bootstraps_mrnn_obs2_classification_subgroup, standards_deviations_mrnn_obs2_classification_subgroup)

print(lower_bounds_mrnn_obs2_classification_subgroup)
print(upper_bounds_mrnn_obs2_classification_subgroup)

[np.float64(-0.44130449785496295), np.float64(0.4594877339722985), np.float64(-0.025053265302227297), np.float64(0.0), np.float64(0.4535771187750073), np.float64(0.19584966747494148), np.float64(0.7106757750941392)]
[np.float64(3.119089146353363), np.float64(0.6620986078886069), np.float64(0.12884553519382974), np.float64(0.0), np.float64(0.6917859177477474), np.float64(1.1708342399596774), np.float64(0.8789941458558211)]


In [460]:
lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs2_classification_subgroup)

print(lower_bounds_mrnn_obs2_classification_subgroup)
print(upper_bounds_mrnn_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.4635676284261475), np.float64(0.0), np.float64(0.0), np.float64(0.4805169751861541), np.float64(0.2845622865901363), np.float64(0.7113659581732991)]
[np.float64(3.2940803837237835), np.float64(0.6683807845048232), np.float64(0.08159769645569703), np.float64(0.0), np.float64(0.7102575565629949), np.float64(1.2226766290149655), np.float64(0.88109777481901)]


#### Mean values of lower bound and upper bound

In [461]:
mean_values_ci_mrnn_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup)

print(mean_values_ci_mrnn_obs2_classification_subgroup)

[np.float64(1.6470401918618918), np.float64(0.5659742064654854), np.float64(0.040798848227848517), np.float64(0.0), np.float64(0.5953872658745745), np.float64(0.7536194578025509), np.float64(0.7962318664961545)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



KeyboardInterrupt: 

#### Calculating the mean and standard deviation of the bootstrap distribution

In [463]:
means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs3_classification_subgroup)

print(means_bootstraps_mrnn_obs3_classification_subgroup)
print(standards_deviations_mrnn_obs3_classification_subgroup)


[np.float64(1.00414169046409), np.float64(0.3486289345447676), np.float64(0.0), np.float64(0.21776692962675417), np.float64(0.4408133808566669), np.float64(1.4254427330239856), np.float64(1.6733769546848565)]
[np.float64(0.7446954298067441), np.float64(0.04142255324414479), np.float64(0.0), np.float64(0.1698430775742359), np.float64(0.09930024806338666), np.float64(1.0025793427577305), np.float64(0.06505829337098328)]


#### Calculating lower bound and upper bound

In [464]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs3_classification_subgroup, means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

[np.float64(-0.45543453140870427), np.float64(0.26744222203855106), np.float64(0.0), np.float64(-0.11511938544219072), np.float64(0.24618847099653576), np.float64(-0.5395766704250045), np.float64(1.5458650427820884)]
[np.float64(2.463717912336884), np.float64(0.4298156470509842), np.float64(0.0), np.float64(0.5506532446956991), np.float64(0.635438290716798), np.float64(3.3904621364729755), np.float64(1.8008888665876246)]


In [465]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

[np.float64(0.11099036475633307), np.float64(0.2704979384867315), np.float64(0.0), np.float64(0.0), np.float64(0.23473926210624346), np.float64(0.0), np.float64(1.5473400640998762)]
[np.float64(2.7370165042499703), np.float64(0.43351364400095094), np.float64(0.0), np.float64(0.3502327317086535), np.float64(0.6214057328979435), np.float64(3.597617080569797), np.float64(1.8039514921318378)]


#### Mean values of lower bound and upper bound

In [466]:
mean_values_ci_mrnn_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup)

print(mean_values_ci_mrnn_obs3_classification_subgroup)

[np.float64(1.4240034345031518), np.float64(0.35200579124384124), np.float64(0.0), np.float64(0.17511636585432674), np.float64(0.42807249750209353), np.float64(1.7988085402848986), np.float64(1.675645778115857)]


### Confidence Interval (Obesity subgroup)

In [77]:
indices = [32, 0, 6, 31, 5, 1, 35]

results_bootstrap_mrnn_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs3_classification_subgroup.append(toolkits.bootstrap_v2(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



In [78]:
means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs3_classification_subgroup)

print(means_bootstraps_mrnn_obs3_classification_subgroup)
print(standards_deviations_mrnn_obs3_classification_subgroup)


[np.float64(0.3696615213246427), np.float64(0.46338850275121957), np.float64(1.1955716515985657), np.float64(0.6921596815533241), np.float64(0.9763306584473604), np.float64(0.6720883522581013), np.float64(0.7918787809088241)]
[np.float64(0.042544366062449795), np.float64(0.17091797408700526), np.float64(0.5867888025972247), np.float64(0.1216284117017402), np.float64(0.2757890388652896), np.float64(0.2357263714549806), np.float64(0.013936800453628499)]


In [79]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

[np.float64(0.28119260502214743), np.float64(0.2379884195839537), np.float64(0.0), np.float64(0.43834525944984737), np.float64(0.5003271685328841), np.float64(0.41466908814978265), np.float64(0.7644654328319695)]
[np.float64(0.4517509628720904), np.float64(0.8565371354588842), np.float64(2.2253978902442535), np.float64(0.9143463288515988), np.float64(1.57907732480555), np.float64(1.2302365620751106), np.float64(0.8188392881330487)]


In [80]:
mean_values_ci_mrnn_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup)

print(mean_values_ci_mrnn_obs3_classification_subgroup)

[np.float64(0.3664717839471189), np.float64(0.5472627775214189), np.float64(1.1126989451221267), np.float64(0.6763457941507232), np.float64(1.0397022466692172), np.float64(0.8224528251124466), np.float64(0.7916523604825092)]


# Results mean absolute error

In [ ]:
subgroups = ["General", "Female", "Male", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [13]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35549184664706085
ALT : 0.34616397235246166
AST : 0.283203929613111
Albumin : 0.5865378684716107
BUN : 0.23549456829210566
Bilirubin : 0.29213390018033225
Cholesterol : 0.8619142944054452
Creatinine : 0.2272360108358054
DiasABP : 0.25696352027850733
FiO2 : 0.4843457493583031
GCS : 0.27782594851598286
Glucose : 0.5574566141755575
HCO3 : 0.42235495223366226
HCT : 0.5478476366595461
HR : 0.2786515469045242
K : 0.6173738181988982
Lactate : 0.3602081488093162
MAP : 0.276148040474521
MechVent : 0.016131509283529688
Mg : 0.5428868319523208
NIDiasABP : 0.23805346595387422
NIMAP : 0.17731126967659114
NISysABP : 0.3059993623684565
Na : 0.5094466185029012
PaCO2 : 0.5448751678244528
PaO2 : 0.5044175503727509
Platelets : 0.37910753219049526
RespRate : 0.4975549864022565
SaO2 : 0.395203639666887
SysABP : 0.2876315375696396
Temp : 0.3654739421516718
TroponinI : 0.5742820858879293
TroponinT : 0.562502744207675
Urine : 0.4555801478350003
WBC : 0.15

#### BRITS

<h4>Standard Scaler (C/Normalização)</h4>

In [47]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 0.4591248631105258
ALT : 0.2098298479270722
AST : 0.27798859929904113
Albumin : 0.6778925772529109
BUN : 0.39521960833661646
Bilirubin : 0.3833806392125974
Cholesterol : 0.9463970561439892
Creatinine : 0.3444718062995419
DiasABP : 0.20806233248448402
FiO2 : 0.34404522156046397
GCS : 0.21322463397612945
Glucose : 0.5644091100017328
HCO3 : 0.5047606820899914
HCT : 0.6338702223060172
HR : 0.2431173715923441
K : 0.6534995938478387
Lactate : 0.4167839664888235
MAP : 0.1564062923159312
MechVent : 0.007398844359593802
Mg : 0.5955810654854342
NIDiasABP : 0.15725503986364742
NIMAP : 0.09992748699932849
NISysABP : 0.2543230216923866
Na : 0.645442078485974
PaCO2 : 1.1895743136362507
PaO2 : 0.4776378612780617
Platelets : 0.6445916196510731
RespRate : 0.475295251812804
SaO2 : 0.348066606252948
SysABP : 0.25398352012962805
Temp : 0.240561647460938
TroponinI : 0.667214116433886
TroponinT : 0.29949287221387894
Urine : 0.3333883323992429
WBC : 0.1101